# take mature cells from Calderon umap and subset, rename cells and find nearest neighbors

In [2]:
suppressPackageStartupMessages({
    library(monocle3)
    library(tidyverse)
    library(ggplot2)
    library(reshape2)
    library(viridis)
})

Warning message:
“package ‘S4Vectors’ was built under R version 4.1.3”


In [3]:
sessionInfo()

R version 4.1.2 (2021-11-01)
Platform: x86_64-apple-darwin17.0 (64-bit)
Running under: macOS Big Sur 10.16

Matrix products: default
BLAS:   /Library/Frameworks/R.framework/Versions/4.1/Resources/lib/libRblas.0.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/4.1/Resources/lib/libRlapack.dylib

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] viridis_0.6.3               viridisLite_0.4.2          
 [3] reshape2_1.4.4              forcats_0.5.2              
 [5] stringr_1.5.0               dplyr_1.1.2                
 [7] purrr_1.0.1                 readr_2.1.3                
 [9] tidyr_1.3.0                 tibble_3.2.1               
[11] ggplot2_3.4.2               tidyverse_1.3.2            
[13] monocle3_1.3.1              SingleCellExperiment_1.16.0
[15] SummarizedExperiment_1.24.0 GenomicRanges_1.46.1     

In [4]:
library(philentropy)
library(cowplot)
library(ggrepel)


Attaching package: ‘philentropy’


The following object is masked from ‘package:SummarizedExperiment’:

    distance


The following object is masked from ‘package:GenomicRanges’:

    distance


The following object is masked from ‘package:IRanges’:

    distance




In [5]:
load("data/utility.functions.RData")
load("data/moreUtilities.RData")

In [6]:
load("data/utility.functions.3.RData")

In [ ]:
# these are the cells that were subsetted
# probably should have added cluster 19, 20 the cells at lower left.  Could redo this if it looks like it is working

## commands from old notebook

In [1]:
peedeef = ggplot(data_df_100 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = data_df_100$cluster %in% c(1, 16, 18,22, 23, 5, 8, 21, 11, 15, 9, 12)), size =  0.02, stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(data_df_100, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [2]:
peedeef = ggplot(data_df_100 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = data_df_100$inferred_time_window), size =  0.02, stroke = 0.005) +
# scale_color_manual(values = c("grey70", "firebrick")) +
#      labs(color = cell_type) +
guides(color = guide_legend(override.aes = list(size = 4))) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(data_df_100, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [3]:
peedeef = ggplot(data_df_100 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = data_df_100$inferred_time_window == "0-2"), size =  0.1, stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(data_df_100, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [955]:
cds_sub = cds_pcg[, data_df$cluster %in% c(1, 16, 18,22, 23, 5, 8, 21, 11, 15, 9, 12) ]

In [956]:
dim(cds_sub)

[1]  13938 238445

In [957]:
data_df  %>% filter(cluster %in% c(1, 16, 18,22, 23, 5, 8, 21, 11, 15, 9, 12))  %>% dim

[1] 238445     32

In [4]:
peedeef = ggplot(df_sub , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub$inferred_time_window == "12-14"), size =  0.1, stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


### remove steps of making different PCAs since went forward with PCA 300

In [8]:
# corrected plot expr for more than one gene
plot.expr_clus.br2 = function (cds, gene_short_name, coexpr = F, ncol = NULL, x = 1, 
    y = 2, size_pos = NULL, stroke = NULL, non.expr.alpha = 1/3, 
    color.pal = c("grey85", "red", "orange", "yellow"), sample.n = F, 
    label_size = NULL, limit = NULL, size_neg = NULL) 
{
    if (sample.n != F) {
        set.seed(42)
        cds = cds[, sample(1:ncol(cds), sample.n)]
    }
    S_matrix <- reducedDims(cds)[["UMAP"]]
    data_df <- data.frame(S_matrix)
    colnames(data_df) <- c("umap_1", "umap_2")
    data_df$cluster = as.vector(clusters(cds))
    data_df$sample_name <- row.names(data_df)
    data_df$Size_Factor <- colData(cds)$Size_Factor
    data_df$lineage <- colData(cds)$lineage
    data_df$species <- colData(cds)$species
    if (length(nchar(gene_short_name)) > 1) {
        label_df <- data_df %>% group_by(cluster) %>% summarize(umap_1 = median(umap_1), 
            umap_2 = median(umap_2))
        a <- as.data.frame(as.matrix(t(counts(cds)[gene_short_name, 
            ])))
        a <- a/data_df$Size_Factor
        a$min <- apply(a, 1, min)
        data_df$min <- a$min
        if (is.null(size_pos)) {
            size_pos = 0.1
        }
        if (is.null(stroke)) {
            stroke = 0.4
        }
        if (is.null(label_size)) {
            label_size = 4
        }
        if (is.null(limit)) {
            limit = 1
        }
        if (is.null(size_neg)) {
            size_neg = 0.1
        }
        plot = ggplot(data_df, alpha = min > 0) + geom_point(aes(x = umap_1, 
            y = umap_2, color = log2(min + 1)), size = ifelse(data_df$min > 
            limit, size_pos, size_neg), stroke = stroke) + scale_color_gradientn(colors = color.pal) + 
            scale_alpha_manual(values = c(non.expr.alpha, 1)) + 
            guides(alpha = F, color = guide_colorbar(title = "Minimum\nnormalized\nlog2 expression\nfor specified genes")) + 
            geom_text(data = label_df, aes(label = cluster, x = umap_1 + 
                0.25, y = umap_2 + 0.25), size = label_size) + 
            monocle3:::monocle_theme_opts()
        return(plot)
    }
    if (length(nchar(gene_short_name)) == 1) {
        label_df <- data_df %>% group_by(cluster) %>% summarize(umap_1 = median(umap_1), 
            umap_2 = median(umap_2))
        data_df$tmp.expr = counts(cds)[gene_short_name, ]/data_df$Size_Factor
        if (is.null(size_pos)) {
            size_pos = 0
        }
        if (is.null(stroke)) {
            stroke = 0.4
        }
        if (is.null(label_size)) {
            label_size = 4
        }
        if (is.null(limit)) {
            limit = 1
        }
        if (is.null(size_neg)) {
            size_neg = 0.1
        }
        plot = ggplot(data_df) + geom_point(aes(x = umap_1, y = umap_2, 
            color = log2(tmp.expr/Size_Factor + 1), alpha = tmp.expr > 
                0), size = ifelse(data_df$tmp.exp > limit, size_pos, 
            size_neg), stroke = stroke) + scale_color_gradientn(colors = color.pal) + 
            scale_alpha_manual(values = c(non.expr.alpha, 1)) + 
            guides(alpha = F, color = guide_colorbar(title = paste(gene_short_name, 
                "\nnormalized\nlog2 expression", sep = ""))) + 
            geom_text(data = label_df, aes(label = cluster, x = umap_1 + 
                0.25, y = umap_2 + 0.25), size = label_size) + 
            monocle3:::monocle_theme_opts()
        return(plot)
    }
}

## Try 300 PCAs

In [1155]:
cds_sub <- preprocess_cds(cds_sub, num_dim = 300)

In [1156]:
cds_sub = reduce_dimension(cds_sub)

No preprocess_method specified, using preprocess_method = 'PCA'



In [5]:
plot_cells(cds_sub, cell_size = 0.04,  color_cells_by = "annot")

ERROR: Error in plot_cells(cds_sub, cell_size = 0.04, color_cells_by = "annot"): could not find function "plot_cells"


In [1158]:
cds_sub = cluster_cells(cds_sub, resolution=4e-4)

In [6]:
plot_cells(cds_sub, cell_size = 0.04)

ERROR: Error in plot_cells(cds_sub, cell_size = 0.04): could not find function "plot_cells"


In [9]:
cds_sub = readRDS( "data/cds_fly_sub_PCA300_older_cells.rds")

In [51]:
df_sub300 = make_df(cds_sub)

In [17]:
options(repr.plot.width=14, repr.plot.height=8)

In [7]:
plot_cells(cds_sub, cell_size = 0.04)

ERROR: Error in plot_cells(cds_sub, cell_size = 0.04): could not find function "plot_cells"


## Pick up here to redo

## Use PCA to find singleton cells and remove

In [18]:
cells <- colData(cds_sub)

In [19]:
cellTypes <- cells$annot

In [20]:
cellIDs <- cells$cell

In [21]:
pca <- cds_sub@int_colData$reducedDims$PCA

In [22]:
library(RANN)

In [23]:
nnMatches <- function(ids,labels,df,k=10){
   k1 <- k + 1  # need one more since always finds itself
   rann <- RANN::nn2(df,query=df,k1)
   df <- as.data.frame(rann$nn.idx)
   df <- mutate(df,ID=ids,.before=1)  # add the ids to the df
   df <- mutate(df,Label=labels[V1],.before=2) %>% select(-V1)  # put the labels into the df
   df <- pivot_longer(df,cols=3:k1,names_to = "NN")
   df <- mutate(df,nnLabel=labels[value])  # label the nearest neighbors
   df <- select(df,-NN,-value)
   res <- group_by(df,ID) %>% summarise(nk=sum (Label==nnLabel)  )  # count the matches in the nearest neighors
   return(res)
}

### Do individual steps of Lou's function above

In [24]:
k = 20

In [25]:
k1 <- k + 1

In [26]:
df = pca

In [27]:
rann <- RANN::nn2(df,query=df,k1)

In [28]:
saveRDS(rann, "data/fly_rann_PCA300.rds")

In [29]:
ids = cellIDs

In [30]:
df <- as.data.frame(rann$nn.idx)

In [31]:
head(df)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,1084,117474,33297,2934,2355,854,2437,4221,63518,⋯,116766,48132,20116,115992,34544,31818,50700,143608,571,3626
2,2,51177,19874,69268,44378,67519,212700,189174,131569,24279,⋯,20273,130972,9391,3473,109102,132457,130068,46597,53857,183430
3,3,137166,35175,4381,123690,99621,35423,123344,108376,126736,⋯,123710,47008,134782,136306,137709,123045,33837,32261,136304,135305
4,4,68440,71253,67238,73868,67236,71880,64019,64022,66768,⋯,64612,63786,70436,65552,2853,65551,67168,39930,28538,74114
5,5,137728,19780,120926,91533,5730,144152,143823,35446,54161,⋯,51952,4980,134914,19964,132236,52185,34199,214925,117952,68538
6,6,36849,1387,2385,63108,36476,2887,67810,3884,35089,⋯,5792,75285,2730,19767,34789,35981,1882,64621,49619,781


In [32]:
df <- mutate(df,ID=ids,.before=1)

In [33]:
head(df)

,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,⋯,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,1,1084,117474,33297,2934,2355,854,2437,4221,⋯,116766,48132,20116,115992,34544,31818,50700,143608,571,3626
2,exp1_idx1AGACCATATC_idx2AACGAGGCTA.ACGAGGTTTTTAATTATTG,2,51177,19874,69268,44378,67519,212700,189174,131569,⋯,20273,130972,9391,3473,109102,132457,130068,46597,53857,183430
3,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCAGAACCTCAGGTATGGA,3,137166,35175,4381,123690,99621,35423,123344,108376,⋯,123710,47008,134782,136306,137709,123045,33837,32261,136304,135305
4,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGAGAGATAAGTAGTCAG,4,68440,71253,67238,73868,67236,71880,64019,64022,⋯,64612,63786,70436,65552,2853,65551,67168,39930,28538,74114
5,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGGAGTTTTGAGGTAGAA,5,137728,19780,120926,91533,5730,144152,143823,35446,⋯,51952,4980,134914,19964,132236,52185,34199,214925,117952,68538
6,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGTCTCATTCGATATTGA,6,36849,1387,2385,63108,36476,2887,67810,3884,⋯,5792,75285,2730,19767,34789,35981,1882,64621,49619,781


In [34]:
labels = cells$annot

In [35]:
df <- mutate(df,Label=labels[V1],.before=2) %>% select(-V1) 

In [36]:
head(df)

,ID,Label,V2,V3,V4,V5,V6,V7,V8,V9,⋯,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21
,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,1084,117474,33297,2934,2355,854,2437,4221,⋯,116766,48132,20116,115992,34544,31818,50700,143608,571,3626
2,exp1_idx1AGACCATATC_idx2AACGAGGCTA.ACGAGGTTTTTAATTATTG,visceral muscle primordium,51177,19874,69268,44378,67519,212700,189174,131569,⋯,20273,130972,9391,3473,109102,132457,130068,46597,53857,183430
3,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCAGAACCTCAGGTATGGA,brain,137166,35175,4381,123690,99621,35423,123344,108376,⋯,123710,47008,134782,136306,137709,123045,33837,32261,136304,135305
4,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGAGAGATAAGTAGTCAG,epidermis,68440,71253,67238,73868,67236,71880,64019,64022,⋯,64612,63786,70436,65552,2853,65551,67168,39930,28538,74114
5,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGGAGTTTTGAGGTAGAA,sensory nervous system,137728,19780,120926,91533,5730,144152,143823,35446,⋯,51952,4980,134914,19964,132236,52185,34199,214925,117952,68538
6,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGTCTCATTCGATATTGA,sensory nervous system,36849,1387,2385,63108,36476,2887,67810,3884,⋯,5792,75285,2730,19767,34789,35981,1882,64621,49619,781


In [37]:
df <- pivot_longer(df,cols=3:k1,names_to = "NN")

Warning message:
“Using an external vector in selections was deprecated in tidyselect 1.1.0.
ℹ Please use `all_of()` or `any_of()` instead.
  # Was:
  data %>% select(k1)

  # Now:
  data %>% select(all_of(k1))

See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.”


In [38]:
df <- mutate(df,nnLabel=labels[value])

In [39]:
head(df, 6)

ID,Label,V21,NN,value,nnLabel
<chr>,<chr>,<int>,<chr>,<int>,<chr>
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,3626,V2,1084,plasmatocytes anlage
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,3626,V3,117474,plasmatocytes anlage
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,3626,V4,33297,sensory nervous system primordium
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,3626,V5,2934,plasmatocytes anlage
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,3626,V6,2355,plasmatocytes anlage
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,3626,V7,854,plasmatocytes anlage


In [40]:
df <- select(df,-NN,-value)

In [41]:
head(df)

ID,Label,V21,nnLabel
<chr>,<chr>,<int>,<chr>
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,3626,plasmatocytes anlage
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,3626,plasmatocytes anlage
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,3626,sensory nervous system primordium
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,3626,plasmatocytes anlage
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,3626,plasmatocytes anlage
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,3626,plasmatocytes anlage


In [42]:
dim(df)

[1] 4530455       4

In [43]:
df  %>% filter(Label==nnLabel)  %>% count(Label)

Label,n
<chr>,<int>
Malpighian tubule,6405
amnioserosa,117350
anlage in statu nascendi,35
brain,229655
brain primordium,52907
cellular blastoderm / pole cell,7
crystal cell,160
ectoderm anlage in statu nascendi,76
ectoderm primordium,9695


In [44]:
res <- group_by(df,ID) %>% summarise(nk=sum (Label==nnLabel)  )

In [45]:
head(res)

ID,nk
<chr>,<int>
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAACTCCAATCGTCCGTCCT,6
exp10_idx1AATACGATAA_idx2AACTACCGAC.AACTAGTTGTAACGGACCAA,5
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGACGATAACTAGGCGC,5
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGACGATCCTAGCGCCT,15
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGACGATGGTCCAGGAG,1
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGATATGTCTGATTAAGA,0


In [1214]:
dim(res)

[1] 238445      2

In [46]:
res  %>% count(nk)

nk,n
<int>,<int>
0,26947
1,20080
2,16807
3,14894
4,13366
5,11919
6,11043
7,10165
8,9928


In [1221]:
res %>% filter(nk == 1)  %>% head

ID,nk
<chr>,<int>
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGACGATGGTCCAGGAG,1
exp10_idx1AATACGATAA_idx2AACTACCGAC.CAAGCAACTGCTCTCGCCT,1
exp10_idx1AATACGATAA_idx2AACTACCGAC.CAGCAAGTTGCCGCAACGA,1
exp10_idx1AATACGATAA_idx2AACTACCGAC.GACCAGGATCCTAGCGCCT,1
exp10_idx1AATACGATAA_idx2AACTACCGAC.TAATACCAGTATCCATGACT,1
exp10_idx1AATACGATAA_idx2AACTACCGAC.TAGCCAATTTCATCTCTGCA,1


In [1220]:
df  %>% filter(ID == "exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGATATGTCTGATTAAGA")

ID,Label,V21,nnLabel
<chr>,<chr>,<int>,<chr>
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGATATGTCTGATTAAGA,muscle,210714,brain
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGATATGTCTGATTAAGA,muscle,210714,brain
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGATATGTCTGATTAAGA,muscle,210714,brain
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGATATGTCTGATTAAGA,muscle,210714,brain
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGATATGTCTGATTAAGA,muscle,210714,brain
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGATATGTCTGATTAAGA,muscle,210714,brain
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGATATGTCTGATTAAGA,muscle,210714,midgut
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGATATGTCTGATTAAGA,muscle,210714,brain
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGATATGTCTGATTAAGA,muscle,210714,brain


In [47]:
res_label <- group_by(df,ID, Label) %>% summarise(nk=sum (Label==nnLabel)  )

`summarise()` has grouped output by 'ID'. You can override using the `.groups` argument.


In [48]:
head(res_label)

ID,Label,nk
<chr>,<chr>,<int>
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAACTCCAATCGTCCGTCCT,ubiquitous,6
exp10_idx1AATACGATAA_idx2AACTACCGAC.AACTAGTTGTAACGGACCAA,midgut,5
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGACGATAACTAGGCGC,muscle,5
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGACGATCCTAGCGCCT,brain,15
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGACGATGGTCCAGGAG,epidermis,1
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGATATGTCTGATTAAGA,muscle,0


In [49]:
res_label  %>% filter(nk == 0)  %>% ungroup  %>% count(Label)

Label,n
<chr>,<int>
Malpighian tubule,252
amnioserosa,487
anlage in statu nascendi,21
brain,935
brain primordium,904
cellular blastoderm / pole cell,12
crystal cell,110
ectoderm anlage,5
ectoderm anlage in statu nascendi,18


In [52]:
df_sub300  %>% count(annot)  %>% arrange(annot)

annot,n
<chr>,<int>
Malpighian tubule,1090
amnioserosa,7610
anlage in statu nascendi,40
brain,26617
brain primordium,9816
cellular blastoderm / pole cell,16
crystal cell,177
ectoderm anlage,5
ectoderm anlage in statu nascendi,51


In [1359]:
res_label  %>% filter(nk == 1)  %>% ungroup  %>% count(Label)

Label,n
<chr>,<int>
Malpighian tubule,90
amnioserosa,279
anlage in statu nascendi,11
brain,1363
brain primordium,1127
cellular blastoderm / pole cell,2
crystal cell,23
ectoderm anlage in statu nascendi,14
ectoderm primordium,164


In [53]:
singletons = (res %>% filter(nk == 0) )$ID

In [54]:
doubletons = (res %>% filter(nk == 1) )$ID

In [ ]:
# create the dataframe from the cds from 300 PCAs
df_sub300 = make_df(cds_sub)

In [57]:
cell_type = "glia"

In [459]:
options(repr.plot.width=14, repr.plot.height=8)

In [8]:
# cell.type = "visceral muscle"
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% singletons), size = ifelse(df_sub300$cell %in% singletons, 0.2, 0.05), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "singletons") + 
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [9]:
#cell.type = "visceral muscle"
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% singletons & df_sub300$annot == "Malpighian tubule"), 
           size = ifelse( df_sub300$cell %in% singletons & df_sub300$annot == "Malpighian tubule", 2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Malpighian tubule singletons") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [10]:
#cell.type = "visceral muscle"
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% singletons & df_sub300$annot == "Malpighian tubule"), 
           size = ifelse( df_sub300$cell %in% singletons & df_sub300$annot == "Malpighian tubule", 2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Malpighian tubule singletons") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [11]:
#cell.type = "visceral muscle"
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = !(df_sub300$cell %in% singletons) & df_sub300$annot == "Malpighian tubule"), 
           size = ifelse( !(df_sub300$cell %in% singletons) & df_sub300$annot == "Malpighian tubule", 2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Malpighian tubule not singletons") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [12]:
#cell.type = "visceral muscle"
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% doubletons & df_sub300$annot == "Malpighian tubule"), 
           size = ifelse( df_sub300$cell %in% doubletons & df_sub300$annot == "Malpighian tubule", 2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Malpighian tubule doubletons") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [13]:
#cell.type = "visceral muscle"
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = !(df_sub300$cell %in% doubletons) & df_sub300$annot == "Malpighian tubule" ), 
           size = ifelse( !(df_sub300$cell %in% doubletons) & df_sub300$annot == "Malpighian tubule", 2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Malpighian tubule not doubletons") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [14]:
#cell.type = "visceral muscle"
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% doubletons), size = ifelse(df_sub300$cell %in% doubletons, 0.1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "doubletons") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [69]:
nineteentons = (res %>% filter(nk == 19) )$ID

In [15]:
#cell.type = "visceral muscle"
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% nineteentons), size = ifelse(df_sub300$cell %in% nineteentons, 0.1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "nineteentons") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


### Subset on singletons and also on doubletons

In [72]:
ls()

[1] "cds_sub"                           "cell_type"                        
 [3] "cell.type"                         "cellIDs"                          
 [5] "cells"                             "cellTypes"                        
 [7] "df"                                "df_sub300"                        
 [9] "doubletons"                        "expresses.gene"                   
[11] "get.gene.id"                       "get.gene.id2"                     
[13] "get.norm.expr.matrix"              "get.norm.expr.matrix.3"           
[15] "ids"                               "k"                                
[17] "k1"                                "labels"                           
[19] "make_df"                           "nineteentons"                     
[21] "nnMatches"                         "pca"                              
[23] "peedeef"                           "plot.expr"                        
[25] "plot.expr_clus"                    "plot.expr_clus.br"                
[27] "plot.expr_clus.br2"                "plot.expr3"                       
[29] "rann"                              "res"                              
[31] "res_label"                         "singletons"                       
[33] "two.set.differential.gene.test"    "two.set.differential.gene.test.3" 
[35] "two.set.differential.gene.test.3a" "two.set.differential.gene.test.3b"

In [73]:
cds_sub_no_sing = cds_sub[, !(df_sub300$cell %in% singletons)]

In [75]:
df_sub300  %>% filter(!(cell %in% singletons))  %>% dim

[1] 211498     32

In [76]:
dim(cds_sub_no_sing)

[1]  13938 211498

In [77]:
df_sub_no_singletons = make_df(cds_sub_no_sing)

In [78]:
## read in the previously calculated marker files
load("data/markers for fly datasets.RData")

In [79]:
ls()

[1] "cds_sub"                                         
 [2] "cds_sub_no_sing"                                 
 [3] "cell_type"                                       
 [4] "cell.type"                                       
 [5] "cell.type.2.markers"                             
 [6] "cell.type.markers.sub300_no_singletons.cell.type"
 [7] "cell.type.markers.sub300.cell.type"              
 [8] "cellIDs"                                         
 [9] "cells"                                           
[10] "cellTypes"                                       
[11] "cluster.markers"                                 
[12] "cluster.markers.sub300"                          
[13] "cluster.markers.sub300_no_doubletons"            
[14] "cluster.markers.sub300_no_doubletons_all"        
[15] "cluster.markers.sub300_no_singletons"            
[16] "df"                                              
[17] "df_sub_no_singletons"                            
[18] "df_sub300"                                       
[19] "doubletons"                                      
[20] "expresses.gene"                                  
[21] "get.gene.id"                                     
[22] "get.gene.id2"                                    
[23] "get.norm.expr.matrix"                            
[24] "get.norm.expr.matrix.3"                          
[25] "ids"                                             
[26] "k"                                               
[27] "k1"                                              
[28] "labels"                                          
[29] "make_df"                                         
[30] "nineteentons"                                    
[31] "nnMatches"                                       
[32] "pca"                                             
[33] "peedeef"                                         
[34] "plot.expr"                                       
[35] "plot.expr_clus"                                  
[36] "plot.expr_clus.br"                               
[37] "plot.expr_clus.br2"                              
[38] "plot.expr3"                                      
[39] "rann"                                            
[40] "res"                                             
[41] "res_label"                                       
[42] "singletons"                                      
[43] "two.set.differential.gene.test"                  
[44] "two.set.differential.gene.test.3"                
[45] "two.set.differential.gene.test.3a"               
[46] "two.set.differential.gene.test.3b"

### skip steps to find marker genes

In [1255]:
DEG.results.sub300 = lapply(sort((unique(df_sub300$cluster))),
                     function(this.cluster) {
    message("Finding markers for cluster ", this.cluster)
    cbind(
        two.set.differential.gene.test.3a(
            cds_sub,
            df_sub300$cluster == this.cluster,
            df_sub300$cluster != this.cluster,
            sample.n.max = 50000),
        data.frame(cluster = this.cluster))
                         })

Finding markers for cluster 1

# of cells sampled in set 1: 2548

# of cells sampled in set 2: 50000

Finding markers for cluster 2

# of cells sampled in set 1: 2423

# of cells sampled in set 2: 50000

Finding markers for cluster 3

# of cells sampled in set 1: 2409

# of cells sampled in set 2: 50000

Finding markers for cluster 4

# of cells sampled in set 1: 2403

# of cells sampled in set 2: 50000

Finding markers for cluster 5

# of cells sampled in set 1: 2376

# of cells sampled in set 2: 50000

Finding markers for cluster 6

# of cells sampled in set 1: 2357

# of cells sampled in set 2: 50000

Finding markers for cluster 7

# of cells sampled in set 1: 2346

# of cells sampled in set 2: 50000

Finding markers for cluster 8

# of cells sampled in set 1: 2344

# of cells sampled in set 2: 50000

Finding markers for cluster 9

# of cells sampled in set 1: 2336

# of cells sampled in set 2: 50000

Finding markers for cluster 10

# of cells sampled in set 1: 2316

# of cells samp

# of cells sampled in set 2: 50000

Finding markers for cluster 81

# of cells sampled in set 1: 1455

# of cells sampled in set 2: 50000

Finding markers for cluster 82

# of cells sampled in set 1: 1451

# of cells sampled in set 2: 50000

Finding markers for cluster 83

# of cells sampled in set 1: 1445

# of cells sampled in set 2: 50000

Finding markers for cluster 84

# of cells sampled in set 1: 1416

# of cells sampled in set 2: 50000

Finding markers for cluster 85

# of cells sampled in set 1: 1409

# of cells sampled in set 2: 50000

Finding markers for cluster 86

# of cells sampled in set 1: 1406

# of cells sampled in set 2: 50000

Finding markers for cluster 87

# of cells sampled in set 1: 1403

# of cells sampled in set 2: 50000

Finding markers for cluster 88

# of cells sampled in set 1: 1402

# of cells sampled in set 2: 50000

Finding markers for cluster 89

# of cells sampled in set 1: 1378

# of cells sampled in set 2: 50000

Finding markers for cluster 90

# of 

Finding markers for cluster 160

# of cells sampled in set 1: 528

# of cells sampled in set 2: 50000

Finding markers for cluster 161

# of cells sampled in set 1: 526

# of cells sampled in set 2: 50000

Finding markers for cluster 162

# of cells sampled in set 1: 485

# of cells sampled in set 2: 50000

Finding markers for cluster 163

# of cells sampled in set 1: 466

# of cells sampled in set 2: 50000

Finding markers for cluster 164

# of cells sampled in set 1: 453

# of cells sampled in set 2: 50000

Finding markers for cluster 165

# of cells sampled in set 1: 432

# of cells sampled in set 2: 50000

Finding markers for cluster 166

# of cells sampled in set 1: 371

# of cells sampled in set 2: 50000

Finding markers for cluster 167

# of cells sampled in set 1: 370

# of cells sampled in set 2: 50000

Finding markers for cluster 168

# of cells sampled in set 1: 225

# of cells sampled in set 2: 50000

Finding markers for cluster 169

# of cells sampled in set 1: 184

# of c

In [1256]:
cluster.markers.sub300 = do.call(rbind, DEG.results.sub300) %>% filter(higher.expr == "Set 1") %>%
    select(
        cluster = cluster, gene, 
        cluster.n.umi = set.1.n.umi,
        other.n.umi = set.2.n.umi,
        cluster.tpm = set.1.tpm,
        other.tpm = set.2.tpm,
        log2.ratio, heuristic.score) %>%
    arrange(-heuristic.score)

In [1281]:
head(cluster.markers, 20)
head(cluster.markers.sub300, 20)

,cluster,gene,cluster.n.umi,other.n.umi,cluster.tpm,other.tpm,log2.ratio,heuristic.score
,<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG43896158,159,CG43896,6181,103,169782.689,3.9803178,15.057092,261.5920
Muc26B156,157,Muc26B,3922,129,78413.767,6.2073293,13.409355,218.0205
CG7715158,159,CG7715,5655,607,150265.856,22.3719218,12.650453,217.5520
Muc96D156,157,Muc96D,2574,92,49862.918,4.3355259,13.190049,205.8401
CG7017156,157,CG7017,3325,328,77239.149,13.8080423,12.348736,200.5072
CG17826156,157,CG17826,1070,30,21551.482,1.5907303,13.022141,187.4611
Cda9158,159,Cda9,569,11,15473.694,0.4088698,13.422992,186.8161
obst-F158,159,obst-F,466,6,12801.082,0.2458294,13.326872,181.8330
CG13545157,158,CG13545,900,65,12518.634,2.5552825,11.781825,160.3731


,cluster,gene,cluster.n.umi,other.n.umi,cluster.tpm,other.tpm,log2.ratio,heuristic.score
,<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Muc26B159,160,Muc26B,3993,111,10368.322,4.1576480,10.973182,146.3826
Muc11A162,163,Muc11A,4284,73,6913.090,2.4916789,10.951194,139.6860
Alp8162,163,Alp8,4089,31,4923.237,0.9517154,11.300649,138.6102
CG7409171,172,CG7409,600,24,4121.228,0.4739611,11.449160,137.4954
CG11370167,168,CG11370,3708,446,15737.252,17.5042494,9.732111,135.6850
CG7017159,160,CG7017,3759,265,11685.814,10.6958361,9.964547,134.6469
ppk503,170,ppk,456,59,5091.065,1.7152860,10.872648,133.8862
CG11350140,141,CG11350,7260,239,9507.727,8.1099484,10.027442,132.5130
Mur18B162,163,Mur18B,2971,77,5330.350,2.6636099,10.506749,130.0765


In [1283]:
cluster.markers  %>% filter(gene %in% (c("Muc26B", "Muc11A", "Alp8", "CG43896", "CG7715" )))  %>%  head(12)
cluster.markers.sub300  %>% filter(gene %in% (c("Muc26B", "Muc11A", "Alp8", "CG43896", "CG7715" )))  %>%  head(12)

,cluster,gene,cluster.n.umi,other.n.umi,cluster.tpm,other.tpm,log2.ratio,heuristic.score
,<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG43896158,159,CG43896,6181,103,169782.68940,3.980318,15.0570921,261.591955
Muc26B156,157,Muc26B,3922,129,78413.76680,6.207329,13.4093546,218.020520
CG7715158,159,CG7715,5655,607,150265.85566,22.371922,12.6504532,217.551961
Muc11A148,149,Muc11A,4377,28,5517.63414,1.209200,11.2863097,140.289908
Alp8148,149,Alp8,4112,34,3884.16447,1.080193,10.8666714,129.571581
CG7715156,157,CG7715,945,2442,20187.57353,48.503146,8.6717316,124.016613
CG771551,52,CG7715,452,3769,715.97945,70.468913,3.3245304,31.535791
Muc26B45,46,Muc26B,41,427,77.99196,13.425134,2.4347407,15.347714
CG7715132,133,CG7715,74,3239,148.38053,57.158770,1.3512331,9.759752


,cluster,gene,cluster.n.umi,other.n.umi,cluster.tpm,other.tpm,log2.ratio,heuristic.score
,<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Muc26B159,160,Muc26B,3993,111,10368.32230,4.1576480,10.9731815,146.382615
Muc11A162,163,Muc11A,4284,73,6913.08960,2.4916789,10.9511940,139.686023
Alp8162,163,Alp8,4089,31,4923.23702,0.9517154,11.3006488,138.610192
CG438961,2,CG43896,6210,83,2734.47541,2.9518611,9.4345161,107.719304
CG77151,2,CG7715,6105,486,2912.44871,16.6408119,7.3671719,84.785187
CG7715159,160,CG7715,1011,2690,2863.91837,63.7625614,5.4666865,62.780949
CG7715170,171,CG7715,12,2034,310.73237,53.9292147,2.5000270,20.710634
CG43896170,171,CG43896,4,1348,108.84354,34.2476549,1.6266566,11.027603
CG7715168,169,CG7715,13,1553,122.02448,48.4356305,1.3035475,9.050271


In [1246]:
DEG.results.sub300_no_singletons = lapply(sort((unique(df_sub_no_singletons$cluster))),
                     function(this.cluster) {
    message("Finding markers for cluster ", this.cluster)
    cbind(
        two.set.differential.gene.test.3a(
            cds_sub_no_sing,
            df_sub_no_singletons$cluster == this.cluster,
            df_sub_no_singletons$cluster != this.cluster,
            sample.n.max = 50000),
        data.frame(cluster = this.cluster))
                         })

Finding markers for cluster 1

# of cells sampled in set 1: 2463

# of cells sampled in set 2: 50000

Finding markers for cluster 2

# of cells sampled in set 1: 2227

# of cells sampled in set 2: 50000

Finding markers for cluster 3

# of cells sampled in set 1: 1396

# of cells sampled in set 2: 50000

Finding markers for cluster 4

# of cells sampled in set 1: 2093

# of cells sampled in set 2: 50000

Finding markers for cluster 5

# of cells sampled in set 1: 2174

# of cells sampled in set 2: 50000

Finding markers for cluster 6

# of cells sampled in set 1: 2123

# of cells sampled in set 2: 50000

Finding markers for cluster 7

# of cells sampled in set 1: 2023

# of cells sampled in set 2: 50000

Finding markers for cluster 8

# of cells sampled in set 1: 2101

# of cells sampled in set 2: 50000

Finding markers for cluster 9

# of cells sampled in set 1: 2221

# of cells sampled in set 2: 50000

Finding markers for cluster 10

# of cells sampled in set 1: 2100

# of cells samp

# of cells sampled in set 2: 50000

Finding markers for cluster 81

# of cells sampled in set 1: 1354

# of cells sampled in set 2: 50000

Finding markers for cluster 82

# of cells sampled in set 1: 1315

# of cells sampled in set 2: 50000

Finding markers for cluster 83

# of cells sampled in set 1: 1224

# of cells sampled in set 2: 50000

Finding markers for cluster 84

# of cells sampled in set 1: 1246

# of cells sampled in set 2: 50000

Finding markers for cluster 85

# of cells sampled in set 1: 1266

# of cells sampled in set 2: 50000

Finding markers for cluster 86

# of cells sampled in set 1: 1272

# of cells sampled in set 2: 50000

Finding markers for cluster 87

# of cells sampled in set 1: 1162

# of cells sampled in set 2: 50000

Finding markers for cluster 88

# of cells sampled in set 1: 1237

# of cells sampled in set 2: 50000

Finding markers for cluster 89

# of cells sampled in set 1: 1222

# of cells sampled in set 2: 50000

Finding markers for cluster 90

# of 

Finding markers for cluster 160

# of cells sampled in set 1: 438

# of cells sampled in set 2: 50000

Finding markers for cluster 161

# of cells sampled in set 1: 488

# of cells sampled in set 2: 50000

Finding markers for cluster 162

# of cells sampled in set 1: 410

# of cells sampled in set 2: 50000

Finding markers for cluster 163

# of cells sampled in set 1: 435

# of cells sampled in set 2: 50000

Finding markers for cluster 164

# of cells sampled in set 1: 376

# of cells sampled in set 2: 50000

Finding markers for cluster 165

# of cells sampled in set 1: 387

# of cells sampled in set 2: 50000

Finding markers for cluster 166

# of cells sampled in set 1: 328

# of cells sampled in set 2: 50000

Finding markers for cluster 167

# of cells sampled in set 1: 355

# of cells sampled in set 2: 50000

Finding markers for cluster 168

# of cells sampled in set 1: 198

# of cells sampled in set 2: 50000

Finding markers for cluster 169

# of cells sampled in set 1: 177

# of c

In [1249]:
cluster.markers.sub300_no_singletons = do.call(rbind, DEG.results.sub300_no_singletons) %>% filter(higher.expr == "Set 1") %>%
    select(
        cluster = cluster, gene, 
        cluster.n.umi = set.1.n.umi,
        other.n.umi = set.2.n.umi,
        cluster.tpm = set.1.tpm,
        other.tpm = set.2.tpm,
        log2.ratio, heuristic.score) %>%
    arrange(-heuristic.score)

In [1250]:
cluster.markers.sub300_no_singletons  %>%  head

,cluster,gene,cluster.n.umi,other.n.umi,cluster.tpm,other.tpm,log2.ratio,heuristic.score
,<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Muc26B159,160,Muc26B,3048,85,10265.684,2.9018029,11.361401,151.3984
Muc11A162,163,Muc11A,4202,57,7240.091,1.2972972,11.621855,149.0153
CG7017159,160,CG7017,3024,200,12257.985,7.1512453,10.554414,143.3453
CG7409171,172,CG7409,526,14,4108.171,0.2547309,11.676902,140.1769
CG11370167,168,CG11370,3506,434,15867.110,17.1741523,9.769936,136.3281
Mur18B162,163,Mur18B,2928,61,5529.166,1.8673438,10.913131,135.6841


In [1284]:
cluster.markers.sub300  %>% filter(gene %in% (c("Muc26B", "Muc11A", "Alp8", "CG43896", "CG7715" )))  %>%  head(12)
cluster.markers.sub300_no_singletons  %>% filter(gene %in% (c("Muc26B", "Muc11A", "Alp8", "CG43896", "CG7715" )))  %>%  head(12)

,cluster,gene,cluster.n.umi,other.n.umi,cluster.tpm,other.tpm,log2.ratio,heuristic.score
,<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Muc26B159,160,Muc26B,3993,111,10368.32230,4.1576480,10.9731815,146.382615
Muc11A162,163,Muc11A,4284,73,6913.08960,2.4916789,10.9511940,139.686023
Alp8162,163,Alp8,4089,31,4923.23702,0.9517154,11.3006488,138.610192
CG438961,2,CG43896,6210,83,2734.47541,2.9518611,9.4345161,107.719304
CG77151,2,CG7715,6105,486,2912.44871,16.6408119,7.3671719,84.785187
CG7715159,160,CG7715,1011,2690,2863.91837,63.7625614,5.4666865,62.780949
CG7715170,171,CG7715,12,2034,310.73237,53.9292147,2.5000270,20.710634
CG43896170,171,CG43896,4,1348,108.84354,34.2476549,1.6266566,11.027603
CG7715168,169,CG7715,13,1553,122.02448,48.4356305,1.3035475,9.050271


,cluster,gene,cluster.n.umi,other.n.umi,cluster.tpm,other.tpm,log2.ratio,heuristic.score
,<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Muc26B159,160,Muc26B,3048,85,10265.68401,2.901803,11.361401,151.398428
Muc11A162,163,Muc11A,4202,57,7240.09127,1.297297,11.621855,149.015321
Alp8162,163,Alp8,3913,44,5133.35092,1.623859,10.933995,134.772054
CG438961,2,CG43896,6037,112,2810.31908,4.218980,9.072750,103.946788
CG77151,2,CG7715,5938,708,2979.30561,23.065451,6.951869,80.233219
CG7715159,160,CG7715,779,2504,2983.00308,71.100741,5.370608,61.993102
CG7715168,169,CG7715,13,1216,126.85031,41.764080,1.568656,10.977941
CG43896168,169,CG43896,7,646,68.30401,23.090034,1.503534,9.193909
CG771531,32,CG7715,66,1667,118.95351,48.529265,1.264045,8.729912


### compare prior cell type annotations

In [1261]:
DEG.results.sub300.cell.type = lapply(sort((unique(df_sub300$annot))),
                     function(this.cell.type) {
    message("Finding markers for cluster ", this.cell.type)
    cbind(
        two.set.differential.gene.test.3a(
            cds_sub,
            df_sub300$annot == this.cell.type,
            df_sub300$annot != this.cell.type,
            sample.n.max = 50000),
        data.frame(cell.type = this.cell.type))
                         })

Finding markers for cluster amnioserosa

# of cells sampled in set 1: 7610

# of cells sampled in set 2: 50000

Finding markers for cluster anlage in statu nascendi

# of cells sampled in set 1: 40

# of cells sampled in set 2: 50000

Finding markers for cluster brain

# of cells sampled in set 1: 26617

# of cells sampled in set 2: 50000

Finding markers for cluster brain primordium

# of cells sampled in set 1: 9816

# of cells sampled in set 2: 50000

Finding markers for cluster cellular blastoderm / pole cell

# of cells sampled in set 1: 16

# of cells sampled in set 2: 50000

Finding markers for cluster crystal cell

# of cells sampled in set 1: 177

# of cells sampled in set 2: 50000

Finding markers for cluster ectoderm anlage

# of cells sampled in set 1: 5

# of cells sampled in set 2: 50000

Finding markers for cluster ectoderm anlage in statu nascendi

# of cells sampled in set 1: 51

# of cells sampled in set 2: 50000

Finding markers for cluster ectoderm primordium

# of 

In [1263]:
cell.type.markers.sub300.cell.type = do.call(rbind, DEG.results.sub300.cell.type) %>% filter(higher.expr == "Set 1") %>%
    select(
        cell.type = cell.type, gene, 
        cell.type.n.umi = set.1.n.umi,
        other.n.umi = set.2.n.umi,
        cell.type.tpm = set.1.tpm,
        other.tpm = set.2.tpm,
        log2.ratio, heuristic.score) %>%
    arrange(-heuristic.score)

In [1270]:
cell.type.markers.sub300.cell.type  %>% filter(cell.type.n.umi > 100)  %>% head(20)

,cell.type,gene,cell.type.n.umi,other.n.umi,cell.type.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
pip32,salivary gland,pip,38398,9911,29916.415,412.274111,6.177695,91.85427
Ppn28,plasmatocytes,Ppn,73383,14116,35715.779,536.739395,6.053511,91.55520
CG43332,amnioserosa,CG43332,23490,1608,7785.421,56.955284,7.069691,91.38809
CG12011,amnioserosa,CG12011,82838,15148,29343.847,535.995432,5.772004,85.66128
nur32,salivary gland,nur,26904,10233,22058.236,378.677940,5.860397,84.56023
Muc30E,amnioserosa,Muc30E,15253,1442,4852.606,45.768575,6.697076,82.00464
CG1475632,salivary gland,CG14756,15387,5802,12488.709,206.762957,5.909542,80.41972
CG1429219,Malpighian tubule,CG14292,1706,461,1335.417,5.328381,7.721238,80.17853
CG1135041,tracheal primordium,CG11350,4932,540,2300.934,17.603434,6.950507,77.62765


In [1265]:
DEG.results.sub300_no_singletons.cell.type = lapply(sort((unique(df_sub_no_singletons$annot))),
                     function(this.cell.type) {
    message("Finding markers for cluster ", this.cell.type)
    cbind(
        two.set.differential.gene.test.3a(
            cds_sub_no_sing,
            df_sub_no_singletons$annot == this.cell.type,
            df_sub_no_singletons$annot != this.cell.type,
            sample.n.max = 50000),
        data.frame(cell.type = this.cell.type))
                         })

Finding markers for cluster amnioserosa

# of cells sampled in set 1: 7123

# of cells sampled in set 2: 50000

Finding markers for cluster anlage in statu nascendi

# of cells sampled in set 1: 19

# of cells sampled in set 2: 50000

Finding markers for cluster brain

# of cells sampled in set 1: 25682

# of cells sampled in set 2: 50000

Finding markers for cluster brain primordium

# of cells sampled in set 1: 8912

# of cells sampled in set 2: 50000

Finding markers for cluster cellular blastoderm / pole cell

# of cells sampled in set 1: 4

# of cells sampled in set 2: 50000

Finding markers for cluster crystal cell

# of cells sampled in set 1: 67

# of cells sampled in set 2: 50000

Finding markers for cluster ectoderm anlage in statu nascendi

# of cells sampled in set 1: 33

# of cells sampled in set 2: 50000

Finding markers for cluster ectoderm primordium

# of cells sampled in set 1: 1452

# of cells sampled in set 2: 50000

Finding markers for cluster endoderm primordium



In [1266]:
cell.type.markers.sub300_no_singletons.cell.type = do.call(rbind, DEG.results.sub300_no_singletons.cell.type) %>% filter(higher.expr == "Set 1") %>%
    select(
        cell.type = cell.type, gene, 
        cell.type.n.umi = set.1.n.umi,
        other.n.umi = set.2.n.umi,
        cell.type.tpm = set.1.tpm,
        other.tpm = set.2.tpm,
        log2.ratio, heuristic.score) %>%
    arrange(-heuristic.score)

In [1271]:
cell.type.markers.sub300_no_singletons.cell.type %>% filter(cell.type.n.umi > 100)  %>% head(20)

,cell.type,gene,cell.type.n.umi,other.n.umi,cell.type.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Ppn27,plasmatocytes,Ppn,71774,14120,39945.582,536.442522,6.215782,95.01310
pip31,salivary gland,pip,37863,9930,31750.702,397.472602,6.316164,94.45537
Pde1c5,crystal cell,Pde1c,582,5906,23364.987,270.162694,6.429046,93.29910
CG43332,amnioserosa,CG43332,23047,1551,8171.342,57.964400,7.114585,92.46495
Mur18B18,Malpighian tubule,Mur18B,1677,313,1762.054,5.027292,8.191532,88.33634
CG1429218,Malpighian tubule,CG14292,1694,274,1716.660,5.258797,8.099503,87.03912
CG12011,amnioserosa,CG12011,81662,15596,30913.826,541.220982,5.833227,87.00849
nur31,salivary gland,nur,26383,10064,23290.655,368.527442,5.977926,86.72491
CG1135040,tracheal primordium,CG11350,4883,828,2882.860,16.759489,7.342767,84.39619


In [80]:
cell.type.markers.sub300.cell.type %>% filter(cell.type.n.umi > 100 & 
                                                            cell.type == "plasmatocytes")  %>% head(20)

,cell.type,gene,cell.type.n.umi,other.n.umi,cell.type.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Ppn28,plasmatocytes,Ppn,73383,14116,35715.7788,536.739395,6.053511,91.55520
Pxn28,plasmatocytes,Pxn,12668,2411,6173.7776,93.268241,6.033238,75.97157
Col4a128,plasmatocytes,Col4a1,35872,11438,17492.9629,449.982027,5.277561,74.38495
vkg28,plasmatocytes,vkg,22952,7448,11136.7554,296.224930,5.227630,70.27592
Ance-528,plasmatocytes,Ance-5,1061,196,521.5562,7.292113,5.974939,53.95037
NtR28,plasmatocytes,NtR,1803,486,900.4277,19.621909,5.448361,53.48148
Plod28,plasmatocytes,Plod,15887,11368,7821.5455,486.544675,4.003847,51.78345
CG3105128,plasmatocytes,CG31051,1142,279,567.3650,10.717179,5.597580,51.22163
CG109229,plasmatocytes,CG1092,1278,335,637.4403,13.417458,5.466402,50.93816


In [1272]:
cell.type.markers.sub300_no_singletons.cell.type %>% filter(cell.type.n.umi > 100 & 
                                                            cell.type == "plasmatocytes")  %>% head(20)

,cell.type,gene,cell.type.n.umi,other.n.umi,cell.type.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Ppn27,plasmatocytes,Ppn,71774,14120,39945.5823,536.442522,6.215782,95.01310
Pxn27,plasmatocytes,Pxn,12200,2357,6795.4350,90.346121,6.217079,79.14691
Col4a127,plasmatocytes,Col4a1,34974,11468,19506.1940,449.208424,5.437195,77.48938
vkg27,plasmatocytes,vkg,22254,7348,12342.0464,294.444644,5.384550,73.18363
NtR27,plasmatocytes,NtR,1781,488,1016.9215,18.334900,5.716857,57.11947
Ance-527,plasmatocytes,Ance-5,1000,232,563.7018,7.820584,5.997914,54.82900
Plod27,plasmatocytes,Plod,15175,11106,8535.6323,472.477755,4.172129,54.48572
CG3105127,plasmatocytes,CG31051,1108,268,630.5397,10.444006,5.783923,53.80627
CG109228,plasmatocytes,CG1092,1260,360,719.0644,14.628470,5.523872,52.43250


In [1279]:
df_sub_no_singletons  %>% count(cluster, annot)  %>% 
filter(annot == "Malpighian tubule")  %>% arrange(-n)  %>% head(20)

,cluster,annot,n
,<fct>,<chr>,<int>
1,103,Malpighian tubule,456
2,163,Malpighian tubule,200
3,144,Malpighian tubule,20
4,55,Malpighian tubule,12
5,8,Malpighian tubule,10
6,17,Malpighian tubule,10
7,143,Malpighian tubule,9
8,54,Malpighian tubule,8
9,106,Malpighian tubule,8


In [1280]:
df_sub300  %>% count(cluster, annot)  %>% 
filter(annot == "Malpighian tubule")  %>% arrange(-n)  %>% head(20)

,cluster,annot,n
,<fct>,<chr>,<int>
1,103,Malpighian tubule,469
2,163,Malpighian tubule,206
3,144,Malpighian tubule,23
4,12,Malpighian tubule,20
5,55,Malpighian tubule,20
6,49,Malpighian tubule,17
7,5,Malpighian tubule,16
8,17,Malpighian tubule,16
9,8,Malpighian tubule,15


In [16]:
#  looking at specific cluster
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = !(df_sub300$cell %in% doubletons) & df_sub300$annot == "Malpighian tubule"  & df_sub300$cluster == 163), 
           size = ifelse( !(df_sub300$cell %in% doubletons) & df_sub300$annot == "Malpighian tubule", 2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Malpighian tubule not doubletons cluster 163") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [17]:
#cell.type = "visceral muscle"
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = !(df_sub300$cell %in% doubletons) & df_sub300$annot == "Malpighian tubule" ), 
           size = ifelse( !(df_sub300$cell %in% doubletons) & df_sub300$annot == "Malpighian tubule", 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Malpighian tubule not doubletons") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [18]:
"visceral muscle"
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = !(df_sub300$cell %in% singletons) & df_sub300$annot == "Malpighian tubule" ), 
           size = ifelse( !(df_sub300$cell %in% singletons) & df_sub300$annot == "Malpighian tubule", 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Malpighian tubule not singletons") +
theme_classic()
print(peedeef)

[1] "visceral muscle"

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [19]:
plot_cells(cds_sub_no_sing, genes = "Muc11A")

ERROR: Error in plot_cells(cds_sub_no_sing, genes = "Muc11A"): could not find function "plot_cells"


In [20]:
plot.expr_clus.br2(cds_sub, c("Muc11A"), size_pos = 3, size_neg = 0.06, stroke = 0.02, limit = 0.01)

ERROR: Error in plot.expr_clus.br2(cds_sub, c("Muc11A"), size_pos = 3, size_neg = 0.06, : could not find function "plot.expr_clus.br2"


In [21]:
plot.expr_clus.br2(cds_sub_no_sing, c("Muc11A"), size_pos = 3, size_neg = 0.06, stroke = 0.02, limit = 0.01)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_sing, c("Muc11A"), size_pos = 3, : could not find function "plot.expr_clus.br2"


### Remove both singletons and doubletons

In [85]:
cds_sub_no_doubletons = cds_sub_no_sing[, !(df_sub_no_singletons$cell %in% doubletons)]

In [22]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Muc11A"), size_pos = 3, size_neg = 0.2, stroke = 0.02, limit = 0.01)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Muc11A"), size_pos = 3, : could not find function "plot.expr_clus.br2"


In [23]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = 
               df_sub300$annot == "Malpighian tubule" ), 
           size = ifelse( 
                         df_sub300$annot == "Malpighian tubule", 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


### try muscle as another example

In [24]:
#cell.type = "visceral muscle"
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = !(df_sub300$cell %in% doubletons) & df_sub300$annot == "muscle"  & df_sub300$cluster == 163), 
           size = ifelse( !(df_sub300$cell %in% doubletons) & df_sub300$annot == "muscle", 2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [25]:
#cell.type = "visceral muscle"
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = !(df_sub300$cell %in% doubletons) & df_sub300$annot == "muscle" ), 
           size = ifelse( !(df_sub300$cell %in% doubletons) & df_sub300$annot == "muscle", 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [26]:
"visceral muscle"
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = !(df_sub300$cell %in% singletons) & df_sub300$annot == "muscle" ), 
           size = ifelse( !(df_sub300$cell %in% singletons) & df_sub300$annot == "muscle", 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

[1] "visceral muscle"

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [27]:

peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = !(df_sub300$cell %in% singletons) & !(df_sub300$cell %in% doubletons) &
               df_sub300$annot == "muscle" ), 
           size = ifelse( !(df_sub300$cell %in% singletons) & !(df_sub300$cell %in% doubletons) &
                         df_sub300$annot == "muscle", 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [87]:
df_sub_no_doubletons = make_df(cds_sub_no_doubletons)

In [1341]:
DEG.results.sub300_no_doubletons = lapply(sort((unique(df_sub_no_doubletons$cluster))),
                     function(this.cluster) {
    message("Finding markers for cluster ", this.cluster)
    cbind(
        two.set.differential.gene.test.3a(
            cds_sub_no_doubletons,
            df_sub_no_doubletons$cluster == this.cluster,
            df_sub_no_doubletons$cluster != this.cluster,
            sample.n.max = 50000),
        data.frame(cluster = this.cluster))
                         })

Finding markers for cluster 1

# of cells sampled in set 1: 2401

# of cells sampled in set 2: 50000

Finding markers for cluster 2

# of cells sampled in set 1: 2045

# of cells sampled in set 2: 50000

Finding markers for cluster 3

# of cells sampled in set 1: 1067

# of cells sampled in set 2: 50000

Finding markers for cluster 4

# of cells sampled in set 1: 1806

# of cells sampled in set 2: 50000

Finding markers for cluster 5

# of cells sampled in set 1: 2079

# of cells sampled in set 2: 50000

Finding markers for cluster 6

# of cells sampled in set 1: 1871

# of cells sampled in set 2: 50000

Finding markers for cluster 7

# of cells sampled in set 1: 1770

# of cells sampled in set 2: 50000

Finding markers for cluster 8

# of cells sampled in set 1: 1895

# of cells sampled in set 2: 50000

Finding markers for cluster 9

# of cells sampled in set 1: 2148

# of cells sampled in set 2: 50000

Finding markers for cluster 10

# of cells sampled in set 1: 1960

# of cells samp

Finding markers for cluster 81

# of cells sampled in set 1: 1277

# of cells sampled in set 2: 50000

Finding markers for cluster 82

# of cells sampled in set 1: 1180

# of cells sampled in set 2: 50000

Finding markers for cluster 83

# of cells sampled in set 1: 987

# of cells sampled in set 2: 50000

Finding markers for cluster 84

# of cells sampled in set 1: 1137

# of cells sampled in set 2: 50000

Finding markers for cluster 85

# of cells sampled in set 1: 1188

# of cells sampled in set 2: 50000

Finding markers for cluster 86

# of cells sampled in set 1: 1131

# of cells sampled in set 2: 50000

Finding markers for cluster 87

# of cells sampled in set 1: 1017

# of cells sampled in set 2: 50000

Finding markers for cluster 88

# of cells sampled in set 1: 1100

# of cells sampled in set 2: 50000

Finding markers for cluster 89

# of cells sampled in set 1: 1102

# of cells sampled in set 2: 50000

Finding markers for cluster 90

# of cells sampled in set 1: 952

# of cel

# of cells sampled in set 2: 50000

Finding markers for cluster 161

# of cells sampled in set 1: 457

# of cells sampled in set 2: 50000

Finding markers for cluster 162

# of cells sampled in set 1: 361

# of cells sampled in set 2: 50000

Finding markers for cluster 163

# of cells sampled in set 1: 382

# of cells sampled in set 2: 50000

Finding markers for cluster 164

# of cells sampled in set 1: 304

# of cells sampled in set 2: 50000

Finding markers for cluster 165

# of cells sampled in set 1: 356

# of cells sampled in set 2: 50000

Finding markers for cluster 166

# of cells sampled in set 1: 285

# of cells sampled in set 2: 50000

Finding markers for cluster 167

# of cells sampled in set 1: 318

# of cells sampled in set 2: 50000

Finding markers for cluster 168

# of cells sampled in set 1: 170

# of cells sampled in set 2: 50000

Finding markers for cluster 169

# of cells sampled in set 1: 171

# of cells sampled in set 2: 50000

Finding markers for cluster 170

# of

In [1342]:
cluster.markers.sub300_no_doubletons = do.call(rbind, DEG.results.sub300_no_doubletons) %>% filter(higher.expr == "Set 1") %>%
    select(
        cluster = cluster, gene, 
        cluster.n.umi = set.1.n.umi,
        other.n.umi = set.2.n.umi,
        cluster.tpm = set.1.tpm,
        other.tpm = set.2.tpm,
        log2.ratio, heuristic.score) %>%
    arrange(-heuristic.score)

In [1343]:
cluster.markers.sub300_no_doubletons  %>%  head

,cluster,gene,cluster.n.umi,other.n.umi,cluster.tpm,other.tpm,log2.ratio,heuristic.score
,<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Muc26B159,160,Muc26B,2947,71,11399.564,2.5494796,11.64908,156.9926
Muc11A162,163,Muc11A,3780,45,7151.767,0.9844656,11.81533,151.2869
CG7409171,172,CG7409,453,20,4089.077,0.2765578,11.64530,139.7193
CG14421150,151,CG14421,3521,192,8224.650,4.0385755,10.67272,138.8085
Alp8162,163,Alp8,3709,27,5261.148,1.1925089,11.22858,138.8014
CG7017159,160,CG7017,2812,241,12742.012,10.3560468,10.13192,138.1732


In [88]:
dim(df_sub_no_doubletons)  # using pca 300 cds

[1] 191418     32

In [89]:
dim(cds_sub_no_doubletons)  # 

[1]  13938 191418

In [1346]:
dim(cds_sub_no_sing)

[1]  13938 211498

In [90]:
length(doubletons)
length(singletons)

[1] 20080

[1] 26947

In [1348]:
head(res)

ID,nk
<chr>,<int>
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAACTCCAATCGTCCGTCCT,6
exp10_idx1AATACGATAA_idx2AACTACCGAC.AACTAGTTGTAACGGACCAA,5
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGACGATAACTAGGCGC,5
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGACGATCCTAGCGCCT,15
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGACGATGGTCCAGGAG,1
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGATATGTCTGATTAAGA,0


In [1349]:
head(df)

ID,Label,V21,nnLabel
<chr>,<chr>,<int>,<chr>
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,3626,plasmatocytes anlage
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,3626,plasmatocytes anlage
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,3626,sensory nervous system primordium
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,3626,plasmatocytes anlage
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,3626,plasmatocytes anlage
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,3626,plasmatocytes anlage


In [1369]:
cell.type.markers.sub300.cell.type  %>% filter(cell.type == "muscle")  %>%  head(20)

,cell.type,gene,cell.type.n.umi,other.n.umi,cell.type.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
tey25,muscle,tey,1964,1638,762.34753,70.041469,3.423716,32.78617
Mp2025,muscle,Mp20,4137,1896,504.73719,54.475322,3.185614,28.61397
CG4507825,muscle,CG45078,1824,714,244.08650,19.842287,3.549807,28.17534
Mf25,muscle,Mf,7493,5314,919.35534,178.201118,2.359043,23.22725
beat-IIb25,muscle,beat-IIb,1474,2182,579.46721,100.363721,2.515186,23.09210
Gapdh125,muscle,Gapdh1,4423,3080,514.39818,92.681725,2.457046,22.13686
Zasp6625,muscle,Zasp66,1662,907,222.79963,30.772827,2.809881,21.93411
beat-IIa25,muscle,beat-IIa,904,1603,371.61340,69.495814,2.398193,20.48425
Act57B25,muscle,Act57B,32620,47559,4906.25710,1577.983536,1.635627,20.05393


In [1370]:
cell.type.markers.sub300_no_singletons.cell.type  %>% filter(cell.type == "muscle")  %>%  head(20)

,cell.type,gene,cell.type.n.umi,other.n.umi,cell.type.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
tey24,muscle,tey,1597,1643,845.04318,69.559791,3.582107,34.83451
Mp2024,muscle,Mp20,4063,1966,649.10988,56.567685,3.495132,32.66040
CG4507824,muscle,CG45078,1788,868,312.90491,24.917110,3.593748,29.80720
Mf24,muscle,Mf,7225,5557,1143.41562,177.853242,2.676502,27.19432
Zasp6624,muscle,Zasp66,1619,952,279.69292,28.290679,3.255330,26.47511
Gapdh124,muscle,Gapdh1,4270,3104,636.48976,90.331424,2.800954,26.09441
Mpcp124,muscle,Mpcp1,525,186,79.67202,5.247212,3.672789,23.26343
beat-IIb24,muscle,beat-IIb,1105,2228,587.96653,101.074582,2.526110,23.24537
Act57B24,muscle,Act57B,29143,48429,5529.94972,1551.159977,1.832989,22.79012


In [1371]:
cell.type.markers.sub300.cell.type  %>% filter(cell.type == "somatic muscle")  %>%  head(20)

,cell.type,gene,cell.type.n.umi,other.n.umi,cell.type.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
bt37,somatic muscle,bt,94422,28224,14502.3675,855.56655,4.081579,56.42416
up37,somatic muscle,up,47990,21558,8765.8965,779.15485,3.490069,45.71240
wupA37,somatic muscle,wupA,32291,16004,5760.6562,550.86013,3.383859,42.27207
Mhc37,somatic muscle,Mhc,66939,39510,10760.8221,1235.01410,3.122021,41.81521
sls37,somatic muscle,sls,47282,25980,6855.9405,803.48356,3.091220,39.39249
Sh37,somatic muscle,Sh,16272,7146,3050.0225,299.93213,3.341310,38.67592
l(2)0128937,somatic muscle,l(2)01289,6133,2246,1000.3516,70.29999,3.810462,37.98166
CG3352137,somatic muscle,CG33521,4961,1947,910.0490,71.69011,3.646113,35.84634
Mf37,somatic muscle,Mf,9959,5112,1348.4166,122.70925,3.446241,35.83443


In [1373]:
cell.type.markers.sub300_no_singletons.cell.type  %>% filter(cell.type == "somatic muscle")  %>%  head(20)

,cell.type,gene,cell.type.n.umi,other.n.umi,cell.type.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
bt36,somatic muscle,bt,92826,27379,15359.2344,815.02633,4.234347,58.88669
up36,somatic muscle,up,46828,20136,9239.4016,734.08962,3.651807,48.10796
wupA36,somatic muscle,wupA,31409,15383,6056.9358,527.61971,3.518286,44.20589
Mhc36,somatic muscle,Mhc,65044,37995,11311.6783,1179.65882,3.260149,43.90003
sls36,somatic muscle,sls,46054,24047,7188.2227,742.16007,3.273890,41.94384
Sh36,somatic muscle,Sh,15798,7190,3199.7939,297.32053,3.423044,39.85865
l(2)0128936,somatic muscle,l(2)01289,5946,2208,1047.2543,69.87395,3.885212,38.98334
CG3352136,somatic muscle,CG33521,4859,1795,963.0629,67.77183,3.807740,37.74607
Mf36,somatic muscle,Mf,9606,5191,1407.2562,120.62542,3.532368,36.94749


In [28]:
plot.expr_clus.br2(cds_sub_no_sing, c("bt"), size_pos = 0.5, size_neg = 0.06, stroke = 0.02, limit = 3)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_sing, c("bt"), size_pos = 0.5, : could not find function "plot.expr_clus.br2"


In [29]:
plot.expr_clus.br2(cds_sub, c("bt"), size_pos = 0.5, size_neg = 0.06, stroke = 0.02, limit = 3)

ERROR: Error in plot.expr_clus.br2(cds_sub, c("bt"), size_pos = 0.5, size_neg = 0.06, : could not find function "plot.expr_clus.br2"


In [91]:
ls(pattern = "markers")

[1] "cell.type.2.markers"                             
[2] "cell.type.markers.sub300_no_singletons.cell.type"
[3] "cell.type.markers.sub300.cell.type"              
[4] "cluster.markers"                                 
[5] "cluster.markers.sub300"                          
[6] "cluster.markers.sub300_no_doubletons"            
[7] "cluster.markers.sub300_no_doubletons_all"        
[8] "cluster.markers.sub300_no_singletons"

## make heatmaps of cluster with no singletons


In [1375]:
ls(pattern = "clust")

[1] "cluster.markers"                     
[2] "cluster.markers.sub300"              
[3] "cluster.markers.sub300_no_doubletons"
[4] "cluster.markers.sub300_no_singletons"

### get all tpm's for clusters

In [1402]:
#  _all suffix means both set 1 and set 2 so results for all genes
cluster.markers.sub300_no_doubletons_all = do.call(rbind, DEG.results.sub300_no_doubletons) %>%
    select(
        cluster = cluster, gene, 
        cluster.n.umi = set.1.n.umi,
        other.n.umi = set.2.n.umi,
        cluster.tpm = set.1.tpm,
        other.tpm = set.2.tpm,
        log2.ratio, heuristic.score) %>%
    arrange(-heuristic.score)

In [92]:
dim(cluster.markers.sub300_no_doubletons_all)

[1] 2425212       8

In [93]:
exp_matrix_sub300_no_doubletons = cluster.markers.sub300_no_doubletons_all %>% 
select(cluster, gene, cluster.tpm )

In [94]:
exp_matrix_sub300_no_doubletons = exp_matrix_sub300_no_doubletons %>%  
pivot_wider(names_from = "cluster", values_from = "cluster.tpm")

In [95]:
exp_matrix_sub300_no_doubletons  %>%  head

gene,160,163,172,151,168,170,141,146,165,⋯,8,29,97,71,132,19,118,21,33,3
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Muc26B,11399.564008,19.107997,2.086528,0.000000,0.000000,0.00000,8.585809,0.000000,4.304964,⋯,0.000000,7.550007,7.848214,5.677609,8.641855,6.529676,4.74873,3.8060725,9.798139,11.985219
Muc11A,0.000000,7151.766859,0.000000,0.000000,7.590133,0.00000,0.000000,6.073342,0.000000,⋯,0.000000,0.000000,0.652234,2.025432,1.459941,1.399975,2.39835,0.0000000,11.207518,2.809862
CG7409,0.000000,0.000000,4089.076906,2.177805,0.000000,0.00000,41.150358,0.000000,0.000000,⋯,0.000000,0.000000,0.000000,0.928308,0.000000,0.000000,0.00000,0.7635414,0.000000,0.000000
CG14421,1.517765,0.000000,408.001412,8224.649810,0.000000,23.41646,526.892422,5.365068,0.000000,⋯,1.623706,4.522774,1.953751,1.548365,12.916647,2.293638,0.00000,16.8938476,0.000000,2.643439
Alp8,0.000000,5261.148065,0.000000,0.000000,0.000000,0.00000,7.485179,4.319542,0.000000,⋯,1.398730,0.000000,0.000000,2.317245,3.139688,1.225833,0.00000,1.0344950,2.556276,3.374912
CG7017,12742.011542,6.600423,0.000000,0.000000,0.000000,0.00000,4.109510,6.166064,8.669248,⋯,2.713134,10.096972,7.971661,16.008114,15.927200,14.990986,11.31910,15.2404849,33.954258,30.834494


In [98]:
#  need to get fly_peaks
load("data/fly_TF_target_match.RData")

In [99]:
ls()

[1] "cds_sub"                                         
 [2] "cds_sub_no_doubletons"                           
 [3] "cds_sub_no_sing"                                 
 [4] "cell_type"                                       
 [5] "cell.type"                                       
 [6] "cell.type.2.markers"                             
 [7] "cell.type.markers.sub300_no_singletons.cell.type"
 [8] "cell.type.markers.sub300.cell.type"              
 [9] "cellIDs"                                         
[10] "cells"                                           
[11] "cellTypes"                                       
[12] "cluster.markers"                                 
[13] "cluster.markers.sub300"                          
[14] "cluster.markers.sub300_no_doubletons"            
[15] "cluster.markers.sub300_no_doubletons_all"        
[16] "cluster.markers.sub300_no_singletons"            
[17] "df"                                              
[18] "df_sub_no_doubletons"                            
[19] "df_sub_no_singletons"                            
[20] "df_sub300"                                       
[21] "doubletons"                                      
[22] "exp_matrix"                                      
[23] "exp_matrix_mean"                                 
[24] "exp_matrix_sub300_no_doubletons"                 
[25] "expresses.gene"                                  
[26] "fly_peaks_TF"                                    
[27] "gene_max_mean_entropy"                           
[28] "get.gene.id"                                     
[29] "get.gene.id2"                                    
[30] "get.norm.expr.matrix"                            
[31] "get.norm.expr.matrix.3"                          
[32] "ids"                                             
[33] "k"                                               
[34] "k1"                                              
[35] "labels"                                          
[36] "make_df"                                         
[37] "missing_genes"                                   
[38] "missing_genes_aliases"                           
[39] "nineteentons"                                    
[40] "nnMatches"                                       
[41] "pca"                                             
[42] "peedeef"                                         
[43] "plot.expr"                                       
[44] "plot.expr_clus"                                  
[45] "plot.expr_clus.br"                               
[46] "plot.expr_clus.br2"                              
[47] "plot.expr3"                                      
[48] "rann"                                            
[49] "res"                                             
[50] "res_label"                                       
[51] "singletons"                                      
[52] "two.set.differential.gene.test"                  
[53] "two.set.differential.gene.test.3"                
[54] "two.set.differential.gene.test.3a"               
[55] "two.set.differential.gene.test.3b"               
[56] "vector_angle_emb_targets"

In [100]:
fly_TFs = c(unique(fly_peaks_TF$TF))

In [101]:
TF_exp_matrix_sub300_no_doubletons = exp_matrix_sub300_no_doubletons %>%  
filter(gene %in% fly_TFs)  %>% 
column_to_rownames("gene")

In [102]:
TF_exp_matrix_sub300_no_doubletons_log = log2(TF_exp_matrix_sub300_no_doubletons + 1)

In [1416]:
options(repr.plot.width=20, repr.plot.height=60)

In [30]:
pheatmap(TF_exp_matrix_sub300_no_doubletons_log)

ERROR: Error in pheatmap(TF_exp_matrix_sub300_no_doubletons_log): could not find function "pheatmap"


## Try removing anything with 6 neighbors of less

In [1421]:
options(repr.plot.width=16, repr.plot.height=12)

In [103]:
multitons = (res %>% filter(nk <= 4) )$ID

In [1426]:
length(multitons)

[1] 92094

In [31]:
#cell.type = "visceral muscle"
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% multitons), size = ifelse(df_sub300$cell %in% multitons, 0.1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "cells with <= 4 matches") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [107]:
cds_sub_no_multitons = cds_sub[, !(df_sub300$cell %in% multitons)]

In [108]:
df_sub_no_multitons = make_df(cds_sub_no_multitons)

In [1430]:
df_sub_no_multitons  %>% count(cluster, annot)

cluster,annot,n
<fct>,<chr>,<int>
1,amnioserosa,2284
1,brain,1
1,brain primordium,1
1,ectoderm primordium,8
1,epidermis primordium,9
1,midgut primordium,2
1,peripheral nervous system,1
1,plasmatocytes anlage,1
1,procrystal cell,5


In [32]:
plot.expr_clus.br2(cds_sub_no_multitons, c("bt"), size_pos = 0.5, size_neg = 0.06, stroke = 0.02, limit = 3)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_multitons, c("bt"), size_pos = 0.5, : could not find function "plot.expr_clus.br2"


In [1434]:
df_sub_no_multitons$tmp.expr = counts(cds_sub_no_multitons)["bt", ]/df_sub_no_multitons$Size_Factor

In [1436]:
df_sub_no_multitons  %>% head

,orig.ident,nCount_RNA,nFeature_RNA,rt_bc,cell,time,percent.mt,percent.ribo,barn_dro,barn_hu,barn_mus,RNA_snn_res.0.8,doublet_score,predicted_doublet,doublet_cluster,seurat_clusters,doublet_subcluster,RNA_snn_res.0.08,lasso_age,NNv1_age,NNv2_age,lasso_shift,NNv1_shift,NNv2_shift,annot,ident,inferred_time_window,cell.type_time.bin,Size_Factor,cluster,umap_1,umap_2,tmp.expr
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<int>,<int>,<int>,<chr>,<dbl>,<chr>,<lgl>,<fct>,<lgl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<fct>,<chr>,<chr>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGAGAGATAAGTAGTCAG,exp1,292,202,AAGTAGTCAG,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGAGAGATAAGTAGTCAG,hrs_10_14,0.3424658,0.6849315,320,3,0,23,0.06585398,Singlet,FALSE,12,FALSE,12,12.544636,12.211529,13.83098,0.00000000,0.00000000,0.0000000,epidermis,12,12-14,epidermis:12-14,0.6967951,165,-1.3580737,0.3582040,0.000000
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGTCTCATTCGATATTGA,exp1,354,272,TCGATATTGA,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGTCTCATTCGATATTGA,hrs_14_18,0.5649718,1.6949153,380,0,1,8,0.08257165,Singlet,FALSE,11,FALSE,11,13.458162,15.126904,14.55855,-0.54183787,0.00000000,0.0000000,sensory nervous system,11,14-16,sensory nervous system:14-16,0.8306988,148,-0.6711318,-2.9462104,0.000000
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCTCCTGATTGCGCCAGAA,exp1,287,221,TGCGCCAGAA,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCTCCTGATTGCGCCAGAA,hrs_08_12,0.0000000,3.4843206,307,0,0,6,0.06023494,Singlet,FALSE,9,FALSE,9,9.715162,9.000631,11.35599,0.00000000,0.00000000,0.0000000,midline primordium,9,8-10,midline primordium:8-10,0.6893560,120,-7.5437593,-0.6611074,0.000000
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CGAGATGCTACGCCGTTCA,exp1,509,376,ACGCCGTTCA,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CGAGATGCTACGCCGTTCA,hrs_08_12,0.5893910,5.5009823,552,2,1,0,0.08149588,Singlet,FALSE,21,FALSE,21,12.520638,11.694070,13.23313,0.52063800,0.00000000,1.2331295,plasmatocytes,21,10-12,plasmatocytes:10-12,1.2076129,89,-1.6608665,1.2526858,0.000000
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CGAGATGCTGTTGAAGGAT,exp1,438,282,GTTGAAGGAT,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CGAGATGCTGTTGAAGGAT,hrs_06_10,0.6849315,0.4566210,492,1,1,13,0.04439834,Singlet,FALSE,9,FALSE,9,10.045072,10.022837,10.51352,0.04507167,0.02283669,0.5135212,brain,9,10-12,brain:10-12,1.0017980,88,-2.4943113,-5.3918748,0.000000
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CGAGATGCTTGATAGAACG,exp1,313,227,TGATAGAACG,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CGAGATGCTTGATAGAACG,hrs_14_18,0.6389776,13.4185304,344,2,0,12,0.07449210,Singlet,FALSE,11,FALSE,11,10.338754,15.219664,14.57722,-3.66124596,0.00000000,0.0000000,ubiquitous,11,14-16,ubiquitous:14-16,0.7637470,44,1.8231004,3.9306800,1.309334


In [1438]:
df_sub_no_multitons  %>% filter(tmp.expr > 0)  %>% dim

[1] 31783    33

In [1439]:
df_sub_no_multitons  %>% filter(tmp.expr > 0)  %>% count(annot)

annot,n
<chr>,<int>
Malpighian tubule,81
amnioserosa,292
brain,7045
brain primordium,84
crystal cell,1
ectoderm primordium,20
endoderm primordium,1
epidermis,2935
epidermis primordium,26


In [1441]:
df_sub_no_multitons  %>% filter(tmp.expr > 0 & tmp.expr < 2)  %>% count(annot)

annot,n
<chr>,<int>
Malpighian tubule,56
amnioserosa,237
brain,6182
brain primordium,78
crystal cell,1
ectoderm primordium,18
endoderm primordium,1
epidermis,2255
epidermis primordium,24


In [1442]:
df_sub_no_multitons  %>% filter(tmp.expr > 0 & tmp.expr < 2)  %>% count(cluster)  %>% arrange(-n)

cluster,n
<fct>,<int>
50,1126
62,975
71,925
19,844
3,571
46,341
132,311
44,310
11,309


In [1443]:
bt_lt2  = (df_sub_no_multitons  %>% filter(tmp.expr > 0 & tmp.expr < 2))$cell

In [33]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% bt_lt2), size = ifelse(df_sub300$cell %in% bt_lt2, 0.1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [34]:
bt_gte2  = (df_sub_no_multitons  %>% filter( tmp.expr >= 2))$cell

ERROR: Error in df_sub_no_multitons %>% filter(tmp.expr >= 2): could not find function "%>%"


In [35]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% bt_gte2), size = ifelse(df_sub300$cell %in% bt_gte2, 0.1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1448]:
df_sub_no_multitons  %>% filter(tmp.expr > 0 & tmp.expr < 2 &
                               cluster %in% c(34, 40, 39, 9, 18, 106, 46, 18))  %>% count(annot)

annot,n
<chr>,<int>
brain,35
epidermis,9
fat body,6
glia,1
hindgut / anal pad,1
midgut,7
muscle,4
plasmatocytes,6
salivary gland,128


In [1449]:
df_sub_no_multitons  %>% filter(tmp.expr > 0 & tmp.expr < 2 &
                               !(cluster %in% c(34, 40, 39, 9, 18, 106, 46, 18)))  %>% count(annot)

annot,n
<chr>,<int>
Malpighian tubule,56
amnioserosa,237
brain,6147
brain primordium,78
crystal cell,1
ectoderm primordium,18
endoderm primordium,1
epidermis,2246
epidermis primordium,24


In [1454]:
df_sub300$tmp.expr = counts(cds_sub)["bt", ]/df_sub300$Size_Factor

In [1455]:
df_sub300  %>% filter(tmp.expr > 0 & tmp.expr < 2 &
                               !(cluster %in% c(34, 40, 39, 9, 18, 106, 46, 18)))  %>% count(annot)

annot,n
<chr>,<int>
Malpighian tubule,115
amnioserosa,290
brain,6993
brain primordium,157
cellular blastoderm / pole cell,2
crystal cell,14
ectoderm primordium,25
endoderm primordium,5
epidermis,4941


### try Muc11A

In [1456]:
df_sub_no_multitons$tmp.expr.Muc11A = counts(cds_sub_no_multitons)["Muc11A", ]/df_sub_no_multitons$Size_Factor

In [1457]:
df_sub_no_multitons  %>% head

,orig.ident,nCount_RNA,nFeature_RNA,rt_bc,cell,time,percent.mt,percent.ribo,barn_dro,barn_hu,barn_mus,RNA_snn_res.0.8,doublet_score,predicted_doublet,doublet_cluster,seurat_clusters,doublet_subcluster,RNA_snn_res.0.08,lasso_age,NNv1_age,NNv2_age,lasso_shift,NNv1_shift,NNv2_shift,annot,ident,inferred_time_window,cell.type_time.bin,Size_Factor,cluster,umap_1,umap_2,tmp.expr,tmp.expr.Muc11A
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<int>,<int>,<int>,<chr>,<dbl>,<chr>,<lgl>,<fct>,<lgl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<fct>,<chr>,<chr>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGAGAGATAAGTAGTCAG,exp1,292,202,AAGTAGTCAG,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGAGAGATAAGTAGTCAG,hrs_10_14,0.3424658,0.6849315,320,3,0,23,0.06585398,Singlet,FALSE,12,FALSE,12,12.544636,12.211529,13.83098,0.00000000,0.00000000,0.0000000,epidermis,12,12-14,epidermis:12-14,0.6967951,165,-1.3580737,0.3582040,0.000000,0
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGTCTCATTCGATATTGA,exp1,354,272,TCGATATTGA,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGTCTCATTCGATATTGA,hrs_14_18,0.5649718,1.6949153,380,0,1,8,0.08257165,Singlet,FALSE,11,FALSE,11,13.458162,15.126904,14.55855,-0.54183787,0.00000000,0.0000000,sensory nervous system,11,14-16,sensory nervous system:14-16,0.8306988,148,-0.6711318,-2.9462104,0.000000,0
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCTCCTGATTGCGCCAGAA,exp1,287,221,TGCGCCAGAA,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCTCCTGATTGCGCCAGAA,hrs_08_12,0.0000000,3.4843206,307,0,0,6,0.06023494,Singlet,FALSE,9,FALSE,9,9.715162,9.000631,11.35599,0.00000000,0.00000000,0.0000000,midline primordium,9,8-10,midline primordium:8-10,0.6893560,120,-7.5437593,-0.6611074,0.000000,0
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CGAGATGCTACGCCGTTCA,exp1,509,376,ACGCCGTTCA,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CGAGATGCTACGCCGTTCA,hrs_08_12,0.5893910,5.5009823,552,2,1,0,0.08149588,Singlet,FALSE,21,FALSE,21,12.520638,11.694070,13.23313,0.52063800,0.00000000,1.2331295,plasmatocytes,21,10-12,plasmatocytes:10-12,1.2076129,89,-1.6608665,1.2526858,0.000000,0
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CGAGATGCTGTTGAAGGAT,exp1,438,282,GTTGAAGGAT,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CGAGATGCTGTTGAAGGAT,hrs_06_10,0.6849315,0.4566210,492,1,1,13,0.04439834,Singlet,FALSE,9,FALSE,9,10.045072,10.022837,10.51352,0.04507167,0.02283669,0.5135212,brain,9,10-12,brain:10-12,1.0017980,88,-2.4943113,-5.3918748,0.000000,0
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CGAGATGCTTGATAGAACG,exp1,313,227,TGATAGAACG,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CGAGATGCTTGATAGAACG,hrs_14_18,0.6389776,13.4185304,344,2,0,12,0.07449210,Singlet,FALSE,11,FALSE,11,10.338754,15.219664,14.57722,-3.66124596,0.00000000,0.0000000,ubiquitous,11,14-16,ubiquitous:14-16,0.7637470,44,1.8231004,3.9306800,1.309334,0


In [1458]:
df_sub_no_multitons  %>% filter(tmp.expr.Muc11A > 0)  %>% dim

[1] 186  34

In [1459]:
df_sub_no_multitons  %>% filter(tmp.expr.Muc11A > 0)  %>% count(annot)

annot,n
<chr>,<int>
Malpighian tubule,75
amnioserosa,1
brain,25
brain primordium,1
ectoderm primordium,1
epidermis,1
foregut / hindgut / anal pad,1
hindgut / anal pad,1
midgut,41


In [1461]:
df_sub_no_multitons  %>% filter(tmp.expr.Muc11A > 0 & tmp.expr.Muc11A < 1)  %>% count(annot)

annot,n
<chr>,<int>
Malpighian tubule,14
brain,22
ectoderm primordium,1
foregut / hindgut / anal pad,1
hindgut / anal pad,1
midgut,23
muscle,3
procrystal cell,1
salivary gland,1


In [1462]:
df_sub_no_multitons  %>% filter(tmp.expr.Muc11A > 0 & tmp.expr.Muc11A < 2)  %>% count(cluster)  %>% arrange(-n)

cluster,n
<fct>,<int>
163,56
103,9
50,8
62,5
71,5
32,3
87,3
3,2
7,2


In [1466]:
Muc11A_lt1  = (df_sub_no_multitons  %>% filter(tmp.expr.Muc11A > 0 & tmp.expr.Muc11A < 1))$cell

In [36]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% Muc11A_lt1), size = ifelse(df_sub300$cell %in% Muc11A_lt2, 2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1468]:
Muc11A_gte1  = (df_sub_no_multitons  %>% filter( tmp.expr.Muc11A >= 1))$cell

In [37]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% Muc11A_gte1), size = ifelse(df_sub300$cell %in% Muc11A_gte1, 2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1478]:
df_sub_no_multitons  %>% filter(tmp.expr.Muc11A > 0 & tmp.expr.Muc11A < 1 &
                               cluster %in% c(103, 163))  %>% count(annot)

annot,n
<chr>,<int>
Malpighian tubule,14
midgut,16
tracheal primordium,1


In [1480]:
df_sub_no_multitons  %>% filter(tmp.expr.Muc11A > 0 & tmp.expr.Muc11A < 1 &
                               !(cluster %in% c(103, 163)))  %>% count(annot)

annot,n
<chr>,<int>
brain,22
ectoderm primordium,1
foregut / hindgut / anal pad,1
hindgut / anal pad,1
midgut,7
muscle,3
procrystal cell,1
salivary gland,1
somatic muscle,1


In [1481]:
df_sub300$tmp.expr.Muc11A = counts(cds_sub)["Muc11A", ]/df_sub300$Size_Factor

In [1482]:
df_sub300  %>% filter(tmp.expr.Muc11A > 0 & tmp.expr.Muc11A < 1 &
                               !(cluster %in% c(103, 163)))  %>% count(annot)

annot,n
<chr>,<int>
brain,23
brain primordium,1
ectoderm primordium,2
epidermis,5
epidermis primordium,1
foregut / hindgut / anal pad,1
hindgut / anal pad,1
midgut,10
muscle,7


In [1483]:
df_sub300  %>% filter(tmp.expr.Muc11A > 0 & tmp.expr.Muc11A < 1 &
                               (cluster %in% c(103, 163)))  %>% count(annot)

annot,n
<chr>,<int>
Malpighian tubule,19
midgut,22
sensory nervous system,1
tracheal primordium,5
tracheal system,6
ubiquitous,1
unknown,2


# Reannotate fly cells in cds_sub (300 PCAs)

In [110]:
cluster.markers.sub300_no_singletons %>% filter(!(cluster %in% c(160, 163, 103, 172, 168, 
                                                              170, 14, 8,
                                                              141, 151, 146, 165,
                                                              85, 41, 99, 110, 31, 60, 111, 169,
                                                              51, 95, 73, 1, 174, 2,
                                                              171, 20, 98, 45, 96, 92, 130, 88,
                                                              13, 55, 167, 104, 123,  25, 129,
                                                              157, 122, 58, 28, 89, 148, 159,
                                                              142, 154, 124, 149, 126, 113,
                                                              81, 139, 34, 147, 166, 107,
                                                              155, 173, 40, 80, 161, 120, 162, 
                                                               134, 156, 115, 56, 30, 128, 137, 158,
                                                              121, 109, 94, 145, 68, 86, 114, 17, 63,
                                                              47, 164, 166, 5, 66, 39, 9, 106,
                                                              10, 152,  36, 127, 12, 75, 46, 133, 136, 119, 48,
                                                              49, 64, 72, 140) ))  %>%  
head(10)

,cluster,gene,cluster.n.umi,other.n.umi,cluster.tpm,other.tpm,log2.ratio,heuristic.score
,<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG918453,54,CG9184,7147,35616,14831.3446,1522.1598978,3.283510,45.49782
CG1059153,54,CG10591,5091,23244,10405.6717,991.7346141,3.389818,45.23787
CG5715149,150,CG5715,32,17,138.1496,0.9084849,6.177659,43.98798
sr143,144,sr,591,3816,2365.5387,165.4169404,3.829295,42.92089
bbg14,15,bbg,4838,15567,7143.5268,700.3065817,3.348519,42.86983
scro56,57,scro,598,1464,1232.0042,67.3759216,4.171375,42.83152
lmd99,100,lmd,691,3111,1997.0907,132.5558449,3.902385,42.78733
Desi53,54,Desi,89,86,182.2837,2.5356873,5.688051,42.76239
ImpE253,54,ImpE2,3061,14790,6225.0365,603.3959499,3.364516,42.40670


In [109]:
cluster_expr_sub300_no_singletons  %>% filter(!(cluster %in% c(160, 163, 103, 172, 168, 
                                                              170, 14, 8,
                                                              141, 151, 146, 165,
                                                              85, 41, 99, 110, 31, 60, 111, 169,
                                                              51, 95, 73, 1, 174, 2,
                                                              171, 20, 98, 45, 96, 92, 130, 88,
                                                              13, 55, 167, 104, 123,  25, 129,
                                                              157, 122, 58, 28, 89, 148, 159,
                                                              142, 154, 124, 149, 126, 113,
                                                              81, 139, 34, 147, 166, 107,
                                                              155, 173, 40, 80, 161, 120, 162, 
                                                               134, 156, 115, 56, 30, 128, 137, 158,
                                                              121, 109, 94, 145, 68, 86, 114, 17, 63,
                                                              47, 164, 166, 5, 66, 39, 9, 106,
                                                              10, 152,  36, 127, 12, 75, 46, 133, 136, 119, 48,
                                                              49, 64, 72, 140) ))  %>%  
head(10)

ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function 'head': object 'cluster_expr_sub300_no_singletons' not found


In [38]:
plot.expr_clus.br2(cds_sub_no_multitons, c("sr"), 
                   size_pos = 0.1, size_neg = 0.2, stroke = 0.02, limit = 1) 

ERROR: Error in plot.expr_clus.br2(cds_sub_no_multitons, c("sr"), size_pos = 0.1, : could not find function "plot.expr_clus.br2"


In [111]:
a = c(160)
b = c(163, 103)
c = c(172)
d = c(168)
e = c(170, 14, 8, 143)  # 8 could be broken down further
f = c(141, 151)
g = c(146)
h = c(165)
i = c(85, 41, 99, 110, 31, 60, 111)
j = c(169)
k = c(174)
l = c(51)
m = c(95, 73)
n = c(1)
o = c(2) # CG43896 is only at one tip so could be broken down further
p = c(171)
q = c(20, 98, 45, 96, 92, 130, 88) # 20 should be split with 13, 55
r = c(13, 55)
s = c(167, 104, 123,  25) # has considerable scatter
t = c(129) # possibly two branches
u = c(157)
v = c(122, 58, 28, 89)
w = c(148)
x = c(159)
y = c(142, 154) # has spur on 154 that could be removed
z = c(124)
aa = c(149, 126) # 121, 109 are spurs, tributaries
ab = c(113) # could be extension of y at least in part, but looks like much is separate
ac = c(81, 139, 34)
ad = c(147)
ae = c(166, 107)  # markers shar, but distant on map; and only a small cluster of 107, which could be broken 3 ways
af = c(173)
ag = c(155)
ah = c(40, 134 ) # 40 has spur that could be broken off
ai = c(80)
aj = c(161, 120, 162)
ak = c(156)
al = c(115, 56) # coalescent for 124, 128, 137
am = c(30)
an = c(128, 137)  # see what the markers are here
ao = c(158) # closely related to 170, 14
ap = c(121, 109) # Tsp74F lights up main trajectory of these two, but both could be broken further
aq = c(94)
ar = c(145, 68, 86)
as = c(114)  # CG13545 is only in a single part of this larger cluster, part of brain and so might be neuron
at = c(17)  # 49 is base and with part of 107 feeds into 12
au = c(63) # CG7300 is only in left spur of 63, so could be broken down further; other spur feeds 128
av = c(47, 164)
aw = c(166) # markers also light up small part of 107, but not the main cluster.  Look for better markers of bulk of 107
ax = c(5, 66) # Ptx1 marker also lights up 159 which is a spur of 5
ay = c(39, 9, 106)
az = c(10, 152, 36, 127) # basal clusters
ba = c(12) # Lim3 also in adjacent 2, 49 is base
bb = c(46) # CG44422 not specific but "CG44422", "disco-r" pair very specific
bc = c(75) # separate cluster of 75 so could be broken down further tey marker also shared with 156
bd = c(133) # Desi in small cluster of 133, part of brain, so could neuronal; shared with 121 or 54
be = c(136, 119, 48, 100) # basal cluster
bf = c(49) # with part of 107 base of 17, 12
bg = c(64) # Ccp84Ac covers it but shared with 113
bh = c(72, 57) # may be base of 124, 128, both could be broken
bi = c(140) # very poor cluster, may be with 123
## below done only on umap picture
bj = c(23, 116, 105, 153)
bk = c(52, 26, 102, 27)
bl = c(87, 70, 35, 37, 22, 59)
bm = c(132, 125, 108)
bn = c(44)
bo = c(97, 118)
bp = c(91, 144, 54, 15, 84)
bq = c(6, 117, 38, 43, 4)
br = c(138, 67, 65)
bs = c(32, 82, 131)
bt = c(11, 76, 3, 62, 50)
bu = c(71, 19)
bv = c(33, 61)
bw = c(77, 53)
bx = c(29, 135)
by = c(21)
bz = c(18) # scattered cells
ca = c(59, 74, 101)
cb = c(112, 79, 7)
cc = c(16, 93, 42, 78, 24)
cd = c(69, 83)
ce = c(90)
cf = c(150) # very scattered cells near 12

In [39]:
clus = c(150)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [ ]:
clus = c(a, b, c, d, e, f, g, h, i , j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z,
        aa,ab, ac, ad, ae, af, ag, ah, ai, aj, ak, al, am, an, ao, ap, aq, ar, as, at, au, av, aw,
        ax, ay, az, ba, bb, bc, bd, be, bf, bh, bi, bj, bk, bl, bm, bn, bo, bp, bq, br, bs, bt, bu, bv, 
         bw, bx, by, bz, ca, cb, cc, cd, ce)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

In [113]:
df_sub300$cell.type.2 = rep(NA, ncol(cds_sub))


In [1747]:
df_sub300  %>%  head

,orig.ident,nCount_RNA,nFeature_RNA,rt_bc,cell,time,percent.mt,percent.ribo,barn_dro,barn_hu,barn_mus,RNA_snn_res.0.8,doublet_score,predicted_doublet,doublet_cluster,seurat_clusters,doublet_subcluster,RNA_snn_res.0.08,lasso_age,NNv1_age,NNv2_age,lasso_shift,NNv1_shift,NNv2_shift,annot,ident,inferred_time_window,cell.type_time.bin,Size_Factor,cluster,umap_1,umap_2,tmp.expr,tmp.expr.Muc11A,cell.type.2
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<int>,<int>,<int>,<chr>,<dbl>,<chr>,<lgl>,<fct>,<lgl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<fct>,<chr>,<chr>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,exp1,346,247,AAGTTGCCAT,exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,hrs_06_10,1.4450867,4.3352601,371,0,0,16,0.05276064,Singlet,FALSE,21,FALSE,21,8.402794,7.885203,8.119637,0.0000000,0.0000000,0.000000,sensory nervous system primordium,21,6-8,sensory nervous system primordium:6-8,0.8108612,89,-2.0067012,1.218446,0,0,NA
exp1_idx1AGACCATATC_idx2AACGAGGCTA.ACGAGGTTTTTAATTATTG,exp1,279,225,TTAATTATTG,exp1_idx1AGACCATATC_idx2AACGAGGCTA.ACGAGGTTTTTAATTATTG,hrs_08_12,0.7168459,5.0179211,331,4,5,7,0.07219402,Singlet,FALSE,11,FALSE,11,11.597093,9.940381,9.959537,0.0000000,0.0000000,0.000000,visceral muscle primordium,11,8-10,visceral muscle primordium:8-10,0.6571200,112,-4.6204820,1.096802,0,0,NA
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCAGAACCTCAGGTATGGA,exp1,586,425,CAGGTATGGA,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCAGAACCTCAGGTATGGA,hrs_08_12,1.0238908,6.4846416,630,0,0,4,0.09008910,Singlet,FALSE,11,FALSE,11,11.266422,10.062633,11.156040,0.0000000,0.0000000,0.000000,brain,11,10-12,brain:10-12,1.4059888,22,-3.6831634,2.801265,0,0,NA
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGAGAGATAAGTAGTCAG,exp1,292,202,AAGTAGTCAG,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGAGAGATAAGTAGTCAG,hrs_10_14,0.3424658,0.6849315,320,3,0,23,0.06585398,Singlet,FALSE,12,FALSE,12,12.544636,12.211529,13.830982,0.0000000,0.0000000,0.000000,epidermis,12,12-14,epidermis:12-14,0.6967951,165,-1.3580737,0.358204,0,0,NA
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGGAGTTTTGAGGTAGAA,exp1,335,243,TGAGGTAGAA,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGGAGTTTTGAGGTAGAA,hrs_14_18,4.4776119,5.6716418,386,0,0,8,0.08478501,Singlet,FALSE,11,FALSE,11,12.542527,13.739003,12.519031,-1.4574728,-0.2609968,-1.480969,sensory nervous system,11,12-14,sensory nervous system:12-14,0.7811049,102,-3.8563070,3.315827,0,0,NA
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGTCTCATTCGATATTGA,exp1,354,272,TCGATATTGA,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGTCTCATTCGATATTGA,hrs_14_18,0.5649718,1.6949153,380,0,1,8,0.08257165,Singlet,FALSE,11,FALSE,11,13.458162,15.126904,14.558554,-0.5418379,0.0000000,0.000000,sensory nervous system,11,14-16,sensory nervous system:14-16,0.8306988,148,-0.6711318,-2.946210,0,0,NA


In [117]:
set.cell.type = function(data_df, cell.type, x) {
       message("Assigning cells for cell type ", cell.type)
       data_df$cell.type.2 = ifelse(x, cell.type, data_df$cell.type.2)
       return(data_df)
    }


In [114]:
df_sub300_copy = df_sub300

In [1786]:
df_sub300 = df_sub300_copy

In [118]:
df_sub300 = set.cell.type(df_sub300, "a", df_sub300$cluster %in% a)  

Assigning cells for cell type a



In [119]:
df_sub300 = set.cell.type(df_sub300, "b", df_sub300$cluster %in% b)

Assigning cells for cell type b



In [120]:
df_sub300 = set.cell.type(df_sub300, "c", df_sub300$cluster %in% c)

Assigning cells for cell type c



In [121]:
df_sub300 = set.cell.type(df_sub300, "d", df_sub300$cluster %in% d)

Assigning cells for cell type d



In [122]:
df_sub300 = set.cell.type(df_sub300, "e", df_sub300$cluster %in% e)

Assigning cells for cell type e



In [123]:
df_sub300 = set.cell.type(df_sub300, "f", df_sub300$cluster %in% f)

Assigning cells for cell type f



In [124]:
df_sub300 = set.cell.type(df_sub300, "g", df_sub300$cluster %in% g)

Assigning cells for cell type g



In [125]:
df_sub300 = set.cell.type(df_sub300, "h", df_sub300$cluster %in% h)

Assigning cells for cell type h



In [126]:
df_sub300 = set.cell.type(df_sub300, "i", df_sub300$cluster %in% i)

Assigning cells for cell type i



In [127]:
df_sub300 = set.cell.type(df_sub300, "j", df_sub300$cluster %in% j)

Assigning cells for cell type j



In [128]:
df_sub300 = set.cell.type(df_sub300, "k", df_sub300$cluster %in% k)

Assigning cells for cell type k



In [129]:
df_sub300 = set.cell.type(df_sub300, "l", df_sub300$cluster %in% l)

Assigning cells for cell type l



In [130]:
df_sub300 = set.cell.type(df_sub300, "m", df_sub300$cluster %in% m)

Assigning cells for cell type m



In [131]:
df_sub300 = set.cell.type(df_sub300, "n", df_sub300$cluster %in% n)

Assigning cells for cell type n



In [132]:
df_sub300 = set.cell.type(df_sub300, "o", df_sub300$cluster %in% o)

Assigning cells for cell type o



In [133]:
copy cells and continue

ERROR: Error in parse(text = x, srcfile = src): <text>:1:6: unexpected symbol
1: copy cells
         ^


In [134]:
df_sub300 = set.cell.type(df_sub300, "p", df_sub300$cluster %in% p)  

Assigning cells for cell type p



In [135]:
df_sub300 = set.cell.type(df_sub300, "q", df_sub300$cluster %in% q)

Assigning cells for cell type q



In [136]:
df_sub300 = set.cell.type(df_sub300, "r", df_sub300$cluster %in% r)

Assigning cells for cell type r



In [137]:
df_sub300 = set.cell.type(df_sub300, "s", df_sub300$cluster %in% s)

Assigning cells for cell type s



In [138]:
df_sub300 = set.cell.type(df_sub300, "t", df_sub300$cluster %in% t)

Assigning cells for cell type t



In [139]:
df_sub300 = set.cell.type(df_sub300, "u", df_sub300$cluster %in% u)

Assigning cells for cell type u



In [140]:
df_sub300 = set.cell.type(df_sub300, "v", df_sub300$cluster %in% v)

Assigning cells for cell type v



In [141]:
df_sub300 = set.cell.type(df_sub300, "w", df_sub300$cluster %in% w)

Assigning cells for cell type w



In [142]:
df_sub300 = set.cell.type(df_sub300, "x", df_sub300$cluster %in% x)

Assigning cells for cell type x



In [143]:
df_sub300 = set.cell.type(df_sub300, "y", df_sub300$cluster %in% y)

Assigning cells for cell type y



In [144]:
df_sub300 = set.cell.type(df_sub300, "z", df_sub300$cluster %in% z)

Assigning cells for cell type z



In [145]:
df_sub300 = set.cell.type(df_sub300, "aa", df_sub300$cluster %in% aa)

Assigning cells for cell type aa



In [146]:
df_sub300 = set.cell.type(df_sub300, "ab", df_sub300$cluster %in% ab)

Assigning cells for cell type ab



In [147]:
df_sub300 = set.cell.type(df_sub300, "ac", df_sub300$cluster %in% ac)

Assigning cells for cell type ac



In [148]:
df_sub300 = set.cell.type(df_sub300, "ad", df_sub300$cluster %in% ad)

Assigning cells for cell type ad



In [ ]:
## and again

In [149]:
df_sub300 = set.cell.type(df_sub300, "ae", df_sub300$cluster %in% ae)  

Assigning cells for cell type ae



In [150]:
df_sub300 = set.cell.type(df_sub300, "af", df_sub300$cluster %in% af)

Assigning cells for cell type af



In [151]:
df_sub300 = set.cell.type(df_sub300, "ag", df_sub300$cluster %in% ag)

Assigning cells for cell type ag



In [152]:
df_sub300 = set.cell.type(df_sub300, "ah", df_sub300$cluster %in% ah)

Assigning cells for cell type ah



In [153]:
df_sub300 = set.cell.type(df_sub300, "ai", df_sub300$cluster %in% ai)

Assigning cells for cell type ai



In [154]:
df_sub300 = set.cell.type(df_sub300, "aj", df_sub300$cluster %in% aj)

Assigning cells for cell type aj



In [155]:
df_sub300 = set.cell.type(df_sub300, "ak", df_sub300$cluster %in% ak)

Assigning cells for cell type ak



In [156]:
df_sub300 = set.cell.type(df_sub300, "al", df_sub300$cluster %in% al)

Assigning cells for cell type al



In [157]:
df_sub300 = set.cell.type(df_sub300, "am", df_sub300$cluster %in% am)

Assigning cells for cell type am



In [158]:
df_sub300 = set.cell.type(df_sub300, "an", df_sub300$cluster %in% an)

Assigning cells for cell type an



In [159]:
df_sub300 = set.cell.type(df_sub300, "ao", df_sub300$cluster %in% ao)

Assigning cells for cell type ao



In [160]:
df_sub300 = set.cell.type(df_sub300, "ap", df_sub300$cluster %in% ap)

Assigning cells for cell type ap



In [161]:
df_sub300 = set.cell.type(df_sub300, "aq", df_sub300$cluster %in% aq)

Assigning cells for cell type aq



In [162]:
df_sub300 = set.cell.type(df_sub300, "ar", df_sub300$cluster %in% ar)

Assigning cells for cell type ar



In [163]:
df_sub300 = set.cell.type(df_sub300, "as", df_sub300$cluster %in% as)

Assigning cells for cell type as



In [164]:
df_sub300 = set.cell.type(df_sub300, "at", df_sub300$cluster %in% at)  

Assigning cells for cell type at



In [165]:
df_sub300 = set.cell.type(df_sub300, "au", df_sub300$cluster %in% au)

Assigning cells for cell type au



In [166]:
df_sub300 = set.cell.type(df_sub300, "av", df_sub300$cluster %in% av)

Assigning cells for cell type av



In [167]:
df_sub300 = set.cell.type(df_sub300, "aw", df_sub300$cluster %in% aw)

Assigning cells for cell type aw



In [168]:
df_sub300 = set.cell.type(df_sub300, "ax", df_sub300$cluster %in% ax)

Assigning cells for cell type ax



In [169]:
df_sub300 = set.cell.type(df_sub300, "ay", df_sub300$cluster %in% ay)

Assigning cells for cell type ay



In [170]:
df_sub300 = set.cell.type(df_sub300, "az", df_sub300$cluster %in% az)

Assigning cells for cell type az



In [171]:
df_sub300 = set.cell.type(df_sub300, "ba", df_sub300$cluster %in% ba)

Assigning cells for cell type ba



In [172]:
df_sub300 = set.cell.type(df_sub300, "bb", df_sub300$cluster %in% bb)

Assigning cells for cell type bb



In [173]:
df_sub300 = set.cell.type(df_sub300, "bc", df_sub300$cluster %in% bc)

Assigning cells for cell type bc



In [174]:
df_sub300 = set.cell.type(df_sub300, "bd", df_sub300$cluster %in% bd)

Assigning cells for cell type bd



In [175]:
df_sub300 = set.cell.type(df_sub300, "be", df_sub300$cluster %in% be)

Assigning cells for cell type be



In [176]:
df_sub300 = set.cell.type(df_sub300, "bf", df_sub300$cluster %in% bf)

Assigning cells for cell type bf



In [177]:
df_sub300 = set.cell.type(df_sub300, "bg", df_sub300$cluster %in% bg)

Assigning cells for cell type bg



In [178]:
df_sub300 = set.cell.type(df_sub300, "bh", df_sub300$cluster %in% bh)

Assigning cells for cell type bh



In [179]:
df_sub300 = set.cell.type(df_sub300, "bi", df_sub300$cluster %in% bi)  

Assigning cells for cell type bi



In [180]:
df_sub300 = set.cell.type(df_sub300, "bj", df_sub300$cluster %in% bj)

Assigning cells for cell type bj



In [181]:
df_sub300 = set.cell.type(df_sub300, "bk", df_sub300$cluster %in% bk)

Assigning cells for cell type bk



In [182]:
df_sub300 = set.cell.type(df_sub300, "bl", df_sub300$cluster %in% bl)

Assigning cells for cell type bl



In [183]:
df_sub300 = set.cell.type(df_sub300, "bm", df_sub300$cluster %in% bm)

Assigning cells for cell type bm



In [184]:
df_sub300 = set.cell.type(df_sub300, "bn", df_sub300$cluster %in% bn)

Assigning cells for cell type bn



In [185]:
df_sub300 = set.cell.type(df_sub300, "bo", df_sub300$cluster %in% bo)

Assigning cells for cell type bo



In [186]:
df_sub300 = set.cell.type(df_sub300, "bp", df_sub300$cluster %in% bp)

Assigning cells for cell type bp



In [187]:
df_sub300 = set.cell.type(df_sub300, "bq", df_sub300$cluster %in% bq)

Assigning cells for cell type bq



In [188]:
df_sub300 = set.cell.type(df_sub300, "br", df_sub300$cluster %in% br)

Assigning cells for cell type br



In [189]:
df_sub300 = set.cell.type(df_sub300, "bs", df_sub300$cluster %in% bs)

Assigning cells for cell type bs



In [190]:
df_sub300 = set.cell.type(df_sub300, "bt", df_sub300$cluster %in% bt)

Assigning cells for cell type bt



In [191]:
df_sub300 = set.cell.type(df_sub300, "bu", df_sub300$cluster %in% bu)

Assigning cells for cell type bu



In [192]:
df_sub300 = set.cell.type(df_sub300, "bv", df_sub300$cluster %in% bv)

Assigning cells for cell type bv



In [193]:
df_sub300 = set.cell.type(df_sub300, "bw", df_sub300$cluster %in% bw)

Assigning cells for cell type bw



In [194]:
df_sub300 = set.cell.type(df_sub300, "bx", df_sub300$cluster %in% bx)

Assigning cells for cell type bx



In [195]:
df_sub300 = set.cell.type(df_sub300, "by", df_sub300$cluster %in% by)

Assigning cells for cell type by



In [196]:
df_sub300 = set.cell.type(df_sub300, "bz", df_sub300$cluster %in% bz)

Assigning cells for cell type bz



In [197]:
df_sub300 = set.cell.type(df_sub300, "ca", df_sub300$cluster %in% ca)

Assigning cells for cell type ca



In [198]:
df_sub300 = set.cell.type(df_sub300, "cb", df_sub300$cluster %in% cb)

Assigning cells for cell type cb



In [199]:
df_sub300 = set.cell.type(df_sub300, "cc", df_sub300$cluster %in% cc)

Assigning cells for cell type cc



In [200]:
df_sub300 = set.cell.type(df_sub300, "cd", df_sub300$cluster %in% cd)

Assigning cells for cell type cd



In [201]:
df_sub300 = set.cell.type(df_sub300, "ce", df_sub300$cluster %in% ce)

Assigning cells for cell type ce



In [202]:
df_sub300 = set.cell.type(df_sub300, "cf", df_sub300$cluster %in% cf)

Assigning cells for cell type cf



In [203]:
df_sub300  %>% count(cell.type.2, cluster)

cell.type.2,cluster,n
<chr>,<fct>,<int>
a,160,528
aa,126,1081
aa,149,717
ab,113,1189
ac,34,1812
ac,81,1455
ac,139,970
ad,147,850
ae,107,1241


In [204]:
df_sub300  %>% count(cell.type.2)

cell.type.2,n
<chr>,<int>
a,528
aa,1798
ab,1189
ac,4237
ad,850
ae,1241
af,115
ag,636
ah,2766


## DE genes on new annotation

In [205]:
DEG.results = lapply(sort((unique(df_sub300$cell.type.2))),
                     function(this.cell.type) {
    message("Finding markers for cell.type ", this.cell.type)
    cbind(
        two.set.differential.gene.test.3a(
            cds_sub,
            df_sub300$cell.type.2 == this.cell.type,
            df_sub300$cell.type.2 != this.cell.type,
            sample.n.max = 50000),
        data.frame(cell.type.2 = this.cell.type))
                         })

Finding markers for cell.type a

# of cells sampled in set 1: 528

# of cells sampled in set 2: 50000

Finding markers for cell.type aa

# of cells sampled in set 1: 1798

# of cells sampled in set 2: 50000

Finding markers for cell.type ab

# of cells sampled in set 1: 1189

# of cells sampled in set 2: 50000

Finding markers for cell.type ac

# of cells sampled in set 1: 4237

# of cells sampled in set 2: 50000

Finding markers for cell.type ad

# of cells sampled in set 1: 850

# of cells sampled in set 2: 50000

Finding markers for cell.type ae

# of cells sampled in set 1: 1241

# of cells sampled in set 2: 50000

Finding markers for cell.type af

# of cells sampled in set 1: 115

# of cells sampled in set 2: 50000

Finding markers for cell.type ag

# of cells sampled in set 1: 636

# of cells sampled in set 2: 50000

Finding markers for cell.type ah

# of cells sampled in set 1: 2766

# of cells sampled in set 2: 50000

Finding markers for cell.type ai

# of cells sampled in set 

# of cells sampled in set 2: 50000

Finding markers for cell.type v

# of cells sampled in set 1: 6021

# of cells sampled in set 2: 50000

Finding markers for cell.type w

# of cells sampled in set 1: 748

# of cells sampled in set 2: 50000

Finding markers for cell.type x

# of cells sampled in set 1: 581

# of cells sampled in set 2: 50000

Finding markers for cell.type y

# of cells sampled in set 1: 1571

# of cells sampled in set 2: 50000

Finding markers for cell.type z

# of cells sampled in set 1: 1101

# of cells sampled in set 2: 50000



In [206]:
cell.type.2.markers = do.call(rbind, DEG.results)  %>%
    select(
        cell.type.2 = cell.type.2, gene, 
        cell.type.2.n.umi = set.1.n.umi,
        other.n.umi = set.2.n.umi,
        cell.type.2.tpm = set.1.tpm,
        other.tpm = set.2.tpm,
        log2.ratio, heuristic.score) %>%
    arrange(-heuristic.score)

In [207]:
head(cell.type.2.markers, 20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Muc26B,a,Muc26B,3993,111,10368.322,4.1576480,10.973182,146.3826
CG740954,c,CG7409,600,24,4121.228,0.4739611,11.449160,137.4954
CG1137061,d,CG11370,3708,446,15737.252,17.5042494,9.732111,135.6850
CG746563,f,CG7465,23068,761,18549.567,23.9239377,9.539638,135.2642
CG7017,a,CG7017,3759,265,11685.814,10.6958361,9.964547,134.6469
CG1135063,f,CG11350,8044,159,6300.220,4.2137333,10.238870,129.2290
Muc96D,a,Muc96D,2590,118,6427.809,5.1071969,10.039601,127.0043
CG1313354,c,CG13133,405,14,2679.090,0.1829864,11.145094,126.9211
CG1456661,d,CG14566,518,60,3985.539,1.6730948,10.542048,126.0926


In [208]:
cell.type.2.markers  %>% filter(heuristic.score > 60)  %>% dim

[1] 399   8

In [280]:
cell.type.2.markers  %>% filter(heuristic.score > 60 & gene %in% fly_TFs)  %>% dim

[1] 10  8

In [284]:
cell.type.2.markers  %>% filter(heuristic.score > 40 & gene %in% fly_TFs)  %>% dim

[1] 96  8

In [209]:
unique((cell.type.2.markers  %>% filter(heuristic.score > 60))$cell.type.2)

[1] "a"  "c"  "d"  "f"  "h"  "g"  "j"  "k"  "o"  "l"  "b"  "ac" "p"  "t"  "u" 
[16] "x"  "z"  "w"  "v"  "m"  "aw" "ad" "ag" "n"  "af" "y"  "al" "ai" "ak" "aa"
[31] "as" "ab" "ao" "aq" "bc" "ce" "am" "aj" "ah" "s"  "cd" "ar" "au" "at" "bj"
[46] "cb" "av" "ca" "ae"

In [210]:
length(unique((cell.type.2.markers  %>% filter(heuristic.score > 60))$cell.type.2))

[1] 49

In [211]:
colData(cds_sub)$cell.type.2 = rep(NA, ncol(cds_sub))

In [212]:
colData(cds_sub)$cell.type.2 = df_sub300$cell.type.2

In [40]:
plot_cells(cds_sub, color_cells_by = "cell.type.2", cell_size = 0.1,group_label_size = 5)

ERROR: Error in plot_cells(cds_sub, color_cells_by = "cell.type.2", cell_size = 0.1, : could not find function "plot_cells"


In [41]:
plot_cells(cds_sub, color_cells_by = "cell.type.2", cell_size = 0.1,group_label_size = 0)

ERROR: Error in plot_cells(cds_sub, color_cells_by = "cell.type.2", cell_size = 0.1, : could not find function "plot_cells"


In [215]:
saveRDS(cds_sub, "data/cds_fly_sub_PCA300_older_cells_reannotated.rds")

In [216]:
saveRDS(cell.type.2.markers, "data/fly_sub300_cell.type.2.markers")

In [217]:
cell.type.2.markers  %>% filter(cell.type.2 == "ay")   %>% head

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
up25,ay,up,24733,25245,13495.945,986.13271,3.773138,51.76876
bt25,ay,bt,31445,40455,16680.929,1367.51442,3.607517,50.59903
Sh25,ay,Sh,9814,8339,5337.145,356.68246,3.899317,48.28182
wupA25,ay,wupA,15322,18677,8226.581,697.40159,3.558164,46.27838
Mhc25,ay,Mhc,25471,46980,13678.470,1561.99176,3.129525,42.99881
CG3352125,ay,CG33521,2727,2254,1475.741,92.42132,3.981544,41.91849


In [42]:
plot.expr_clus.br2(cds_sub, gene = "up", size_pos = 1)

ERROR: Error in plot.expr_clus.br2(cds_sub, gene = "up", size_pos = 1): could not find function "plot.expr_clus.br2"


In [218]:
ls()

[1] "a"                                               
  [2] "aa"                                              
  [3] "ab"                                              
  [4] "ac"                                              
  [5] "ad"                                              
  [6] "ae"                                              
  [7] "af"                                              
  [8] "ag"                                              
  [9] "ah"                                              
 [10] "ai"                                              
 [11] "aj"                                              
 [12] "ak"                                              
 [13] "al"                                              
 [14] "am"                                              
 [15] "an"                                              
 [16] "ao"                                              
 [17] "ap"                                              
 [18] "aq"                                              
 [19] "ar"                                              
 [20] "as"                                              
 [21] "at"                                              
 [22] "au"                                              
 [23] "av"                                              
 [24] "aw"                                              
 [25] "ax"                                              
 [26] "ay"                                              
 [27] "az"                                              
 [28] "b"                                               
 [29] "ba"                                              
 [30] "bb"                                              
 [31] "bc"                                              
 [32] "bd"                                              
 [33] "be"                                              
 [34] "bf"                                              
 [35] "bg"                                              
 [36] "bh"                                              
 [37] "bi"                                              
 [38] "bj"                                              
 [39] "bk"                                              
 [40] "bl"                                              
 [41] "bm"                                              
 [42] "bn"                                              
 [43] "bo"                                              
 [44] "bp"                                              
 [45] "bq"                                              
 [46] "br"                                              
 [47] "bs"                                              
 [48] "bt"                                              
 [49] "bu"                                              
 [50] "bv"                                              
 [51] "bw"                                              
 [52] "bx"                                              
 [53] "by"                                              
 [54] "bz"                                              
 [55] "c"                                               
 [56] "ca"                                              
 [57] "cb"                                              
 [58] "cc"                                              
 [59] "cd"                                              
 [60] "cds_sub"                                         
 [61] "cds_sub_no_doubletons"                           
 [62] "cds_sub_no_multitons"                            
 [63] "cds_sub_no_sing"                                 
 [64] "ce"                                              
 [65] "cell_type"                                       
 [66] "cell.type"                                       
 [67] "cell.type.2.markers"                             
 [68] "cell.type.markers.sub300_no_singletons.cell.type"
 [69] "cell.type.markers.sub300.cell.type"              
 [70] "cellIDs"                                         
 [71] "cells

In [1907]:
ls(pattern = "markers")

[1] "cell.type.2.markers"                             
[2] "cell.type.markers.sub300_no_singletons.cell.type"
[3] "cell.type.markers.sub300.cell.type"              
[4] "cluster.markers"                                 
[5] "cluster.markers.sub300"                          
[6] "cluster.markers.sub300_no_doubletons"            
[7] "cluster.markers.sub300_no_doubletons_all"        
[8] "cluster.markers.sub300_no_singletons"

In [1909]:
save(cell.type.2.markers, cell.type.markers.sub300_no_singletons.cell.type, 
     cell.type.markers.sub300.cell.type, cluster.markers, cluster.markers.sub300, 
     cluster.markers.sub300_no_doubletons, cluster.markers.sub300_no_doubletons_all, 
     cluster.markers.sub300_no_singletons, file = "data/markers for fly datasets.RData")

In [1912]:
ls(pattern = "cds")

[1] "cds_pcg"               "cds_sub"               "cds_sub_no_doubletons"
[4] "cds_sub_no_multitons"  "cds_sub_no_sing"       "cds_worm"

In [1913]:
saveRDS(cds_sub_no_doubletons, "data/cds_fly_pca300_no_doubletons,rds")

In [1914]:
saveRDS(cds_sub_no_sing, "data/cds_fly_pca300_no_doubletons,rds")

In [220]:
# remake with new cell.type labels
cds_sub_no_sing = cds_sub[, !(df_sub300$cell %in% doubletons)]

In [221]:
# remake with new cell.type labels
cds_sub_no_doubletons = cds_sub_no_sing[, !(df_sub_no_singletons$cell %in% doubletons)]

In [224]:
saveRDS(cds_sub_no_doubletons, "data/cds_fly_pca300_no_doubletons,rds")

In [222]:
dim(cds_sub_no_doubletons)

[1]  13938 197447

## DE genes on new annotation no doubletons

In [223]:
# remake with new cell.type labels
df_sub_no_doubletons  = make_df(cds_sub_no_doubletons)

In [225]:
DEG.results_no_doubletons = lapply(sort((unique(df_sub_no_doubletons$cell.type.2))),
                     function(this.cell.type) {
    message("Finding markers for cell.type ", this.cell.type)
    cbind(
        two.set.differential.gene.test.3a(
            cds_sub_no_doubletons,
            df_sub_no_doubletons$cell.type.2 == this.cell.type,
            df_sub_no_doubletons$cell.type.2 != this.cell.type,
            sample.n.max = 50000),
        data.frame(cell.type.2 = this.cell.type))
                         })

Finding markers for cell.type a

# of cells sampled in set 1: 432

# of cells sampled in set 2: 50000

Finding markers for cell.type aa

# of cells sampled in set 1: 1540

# of cells sampled in set 2: 50000

Finding markers for cell.type ab

# of cells sampled in set 1: 1048

# of cells sampled in set 2: 50000

Finding markers for cell.type ac

# of cells sampled in set 1: 3765

# of cells sampled in set 2: 50000

Finding markers for cell.type ad

# of cells sampled in set 1: 719

# of cells sampled in set 2: 50000

Finding markers for cell.type ae

# of cells sampled in set 1: 1019

# of cells sampled in set 2: 50000

Finding markers for cell.type af

# of cells sampled in set 1: 94

# of cells sampled in set 2: 50000

Finding markers for cell.type ag

# of cells sampled in set 1: 495

# of cells sampled in set 2: 50000

Finding markers for cell.type ah

# of cells sampled in set 1: 2347

# of cells sampled in set 2: 50000

Finding markers for cell.type ai

# of cells sampled in set 1

# of cells sampled in set 2: 50000

Finding markers for cell.type v

# of cells sampled in set 1: 5308

# of cells sampled in set 2: 50000

Finding markers for cell.type w

# of cells sampled in set 1: 628

# of cells sampled in set 2: 50000

Finding markers for cell.type x

# of cells sampled in set 1: 512

# of cells sampled in set 2: 50000

Finding markers for cell.type y

# of cells sampled in set 1: 1366

# of cells sampled in set 2: 50000

Finding markers for cell.type z

# of cells sampled in set 1: 963

# of cells sampled in set 2: 50000



In [226]:
cell.type.2.markers_no_doubletons = do.call(rbind, DEG.results_no_doubletons)  %>%
    select(
        cell.type.2 = cell.type.2, gene, 
        cell.type.2.n.umi = set.1.n.umi,
        other.n.umi = set.2.n.umi,
        cell.type.2.tpm = set.1.tpm,
        other.tpm = set.2.tpm,
        log2.ratio, heuristic.score) %>%
    arrange(-heuristic.score)

In [274]:
cell.type.2.markers_no_doubletons  %>% head

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Muc26B,a,Muc26B,3651,121,11236.035,3.9689778,11.142896,149.9385
CG7017,a,CG7017,3415,236,12317.772,9.9390129,10.137043,137.7479
CG740954,c,CG7409,458,16,3866.913,0.3382439,11.496625,137.0092
CG1137061,d,CG11370,2890,826,16311.339,18.7725763,9.688159,135.5730
CG746563,f,CG7465,20683,922,18938.232,27.8101498,9.360509,133.0043
CG3260363,f,CG32603,9367,299,9354.678,8.3255405,9.970285,131.5243


In [275]:
cell.type.2.markers_no_doubletons  %>% filter(heuristic.score > 60)   %>% dim

[1] 416   8

In [283]:
cell.type.2.markers_no_doubletons  %>% filter(heuristic.score > 40)   %>% dim

[1] 2123    8

In [285]:
cell.type.2.markers_no_doubletons  %>% filter(heuristic.score > 60 & gene %in% fly_TFs)  %>% dim

[1] 10  8

In [286]:
cell.type.2.markers_no_doubletons  %>% filter(heuristic.score > 40 & gene %in% fly_TFs)  %>% dim

[1] 113   8

In [288]:
length(unique((cell.type.2.markers_no_doubletons  %>% filter(heuristic.score > 60))$cell.type.2))

[1] 51

In [455]:
unique((cell.type.2.markers_no_doubletons  %>% filter(heuristic.score > 60))$cell.type.2)  %>% sort

[1] "a"  "aa" "ab" "ac" "ad" "ae" "af" "ag" "ah" "ai" "aj" "ak" "al" "am" "ao"
[16] "ap" "aq" "ar" "as" "at" "au" "av" "aw" "b"  "bc" "bj" "c"  "ca" "cb" "cc"
[31] "cd" "ce" "d"  "f"  "g"  "h"  "j"  "k"  "l"  "m"  "n"  "o"  "p"  "s"  "t" 
[46] "u"  "v"  "w"  "x"  "y"  "z"

In [456]:
unique((cell.type.2.markers_no_doubletons  %>% filter(heuristic.score > 40))$cell.type.2)  %>% sort

[1] "a"  "aa" "ab" "ac" "ad" "ae" "af" "ag" "ah" "ai" "aj" "ak" "al" "am" "an"
[16] "ao" "ap" "aq" "ar" "as" "at" "au" "av" "aw" "ax" "ay" "az" "b"  "ba" "bb"
[31] "bc" "bd" "be" "bf" "bg" "bh" "bi" "bj" "bk" "bl" "bq" "bt" "c"  "ca" "cb"
[46] "cc" "cd" "ce" "cf" "d"  "e"  "f"  "g"  "h"  "i"  "j"  "k"  "l"  "m"  "n" 
[61] "o"  "p"  "q"  "r"  "s"  "t"  "u"  "v"  "w"  "x"  "y"  "z"

In [277]:
cell.type.markers.sub300.cell.type %>% filter(heuristic.score > 60)   %>% dim

[1] 83  8

In [287]:
cell.type.markers.sub300.cell.type %>% filter(heuristic.score > 40)   %>% dim

[1] 710   8

In [281]:
cell.type.markers.sub300.cell.type %>% filter(heuristic.score > 60 & gene %in% fly_TFs)  %>% dim

[1] 1 8

In [282]:
cell.type.markers.sub300.cell.type %>% filter(heuristic.score > 40 & gene %in% fly_TFs)  %>% dim

[1] 48  8

In [292]:
length(unique((cell.type.markers.sub300.cell.type  %>% filter(heuristic.score > 60))$cell.type))

[1] 19

### Make expression matrix

In [227]:
exp_matrix_sub300_cell.type.2_no_doubletons = cell.type.2.markers_no_doubletons %>% 
select(cell.type.2, gene, cell.type.2.tpm )

In [228]:
exp_matrix_sub300_cell.type.2_no_doubletons  = exp_matrix_sub300_cell.type.2_no_doubletons  %>%  
pivot_wider(names_from = "cell.type.2", values_from = "cell.type.2.tpm")

In [229]:
TF_exp_matrix_sub300_cell.type.2_no_doubletons = exp_matrix_sub300_cell.type.2_no_doubletons %>%  
filter(gene %in% fly_TFs)  %>% 
column_to_rownames("gene")

In [230]:
TF_exp_matrix_sub300_cell.type.2_no_doubletons  %>% head

,a,c,d,f,g,h,j,l,ac,p,⋯,bp,bs,bn,bx,bv,bu,bw,by,bm,bo
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
lz,0.000000,4.865328,7.602135,3.111284,0.000000,2653.35239,0.000000,10.24233,4.069243,0.000000,⋯,6.509293,4.428309,1.515547,1.931052,3.473598,2.475727,6.841887,2.273606,5.057638,4.616199
lab,43.303446,0.000000,7.664362,18.228002,314.728276,16.78037,0.000000,42.32914,28.974029,6.755926,⋯,52.511672,25.319108,35.563781,31.744393,30.515664,50.839772,24.654562,32.160768,47.711803,19.879523
pdm3,350.913897,119.632237,0.000000,16.772789,23.640521,68.55093,5.234615,25.67455,53.504036,144.508367,⋯,38.312221,29.345579,39.167407,42.290454,67.766081,66.020395,62.138596,42.208948,36.458530,32.495187
sim,9.173147,1.934386,3.086801,9.529305,8.387431,0.00000,24.539877,15.26463,11.038655,0.000000,⋯,12.099036,7.643226,21.890647,13.064890,11.063674,11.654416,12.610305,13.027923,14.041077,16.115924
cas,118.762001,74.381757,148.769656,48.258788,45.094130,122.25675,164.447710,209.53466,131.687173,0.000000,⋯,96.548925,122.984584,288.878177,113.398364,74.569828,85.751831,110.892854,210.868084,216.678396,233.821730
seq,105.169692,78.842652,74.145589,113.632272,324.659152,302.86602,169.059906,225.38024,164.109962,521.302251,⋯,102.994574,59.210993,80.184069,100.459174,202.421599,181.147833,132.326009,65.566818,124.914603,91.692358


In [231]:
TF_exp_matrix_sub300_cell.type.2_no_doubletons_log = log2(TF_exp_matrix_sub300_cell.type.2_no_doubletons + 1)

In [461]:
options(repr.plot.width=20, repr.plot.height = 60)

In [234]:
library(pheatmap)

In [43]:
pheatmap(TF_exp_matrix_sub300_cell.type.2_no_doubletons_log)

ERROR: Error in pheatmap(TF_exp_matrix_sub300_cell.type.2_no_doubletons_log): could not find function "pheatmap"


In [238]:
(head(exp_matrix_sub300_cell.type.2_no_doubletons))

,a,c,d,f,g,h,j,l,ac,p,⋯,bp,bs,bn,bx,bv,bu,bw,by,bm,bo
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Muc26B,11236.03548,1.934386,0.00000,6.085184,0.000000,4.304964,22.555034,5.7737813,0.0000000,0,⋯,1.9768935,3.235536,0.5368438,8.244071,12.032071,5.5083899,8.627551,3.7406520,6.780661,5.8641627
CG7017,12317.77183,0.000000,0.00000,6.260625,5.722330,8.669248,0.000000,0.0000000,0.0000000,0,⋯,0.7254060,30.514094,6.9524386,8.661267,25.653128,15.0861709,26.892909,17.2739601,12.820472,10.4034864
CG7409,0.00000,3866.912562,0.00000,23.616071,0.000000,0.000000,5.776807,0.0000000,0.0000000,0,⋯,0.0000000,1.804183,0.0000000,0.000000,0.000000,0.3636798,0.000000,0.7504173,0.000000,0.0000000
CG11370,44.00799,2.179219,16311.33886,494.561686,9.038455,0.000000,3.188653,0.4046702,4.2444311,0,⋯,0.8970721,7.340034,1.7951774,9.806871,3.573583,9.3569042,4.377091,3.5630590,10.449361,0.3818778
CG7465,29.85168,1325.683259,151.37943,18938.231752,32.455947,5.144668,204.935374,17.6205273,5.6911549,0,⋯,10.4081381,49.504509,38.9990375,46.161684,34.820299,29.3456451,33.734044,39.3271711,40.643456,17.6810878
CG32603,0.00000,662.552574,33.41797,9354.678300,6.814617,0.000000,0.000000,12.3753799,0.9483542,0,⋯,4.4435928,25.183830,8.5456464,21.146668,19.237389,18.0183938,13.889679,22.4665064,23.153994,9.0116771


In [239]:
exp_matrix_sub300_cell.type.2_no_doubletons_mean <-t(apply(exp_matrix_sub300_cell.type.2_no_doubletons,1, function(x) x - mean(x)))

In [240]:
head(exp_matrix_sub300_cell.type.2_no_doubletons_mean)

,a,c,d,f,g,h,j,l,ac,p,⋯,bp,bs,bn,bx,bv,bu,bw,by,bm,bo
Muc26B,11098.28232,-135.8188,-137.75316,-131.6680,-137.75316,-133.44820,-115.19813,-131.97938,-137.75316,-137.75316,⋯,-135.77627,-134.51762,-137.21632,-129.50909,-125.72109,-132.24477,-129.12561,-134.01251,-130.97250,-131.88900
CG7017,12161.74551,-156.0263,-156.02632,-149.7657,-150.30399,-147.35708,-156.02632,-156.02632,-156.02632,-156.02632,⋯,-155.30092,-125.51223,-149.07388,-147.36506,-130.37319,-140.94015,-129.13341,-138.75236,-143.20585,-145.62284
CG7409,-46.53667,3820.3759,-46.53667,-22.9206,-46.53667,-46.53667,-40.75986,-46.53667,-46.53667,-46.53667,⋯,-46.53667,-44.73248,-46.53667,-46.53667,-46.53667,-46.17299,-46.53667,-45.78625,-46.53667,-46.53667
CG11370,-170.55460,-212.3834,16096.77626,279.9991,-205.52414,-214.56260,-211.37394,-214.15792,-210.31816,-214.56260,⋯,-213.66552,-207.22256,-212.76742,-204.75572,-210.98901,-205.20569,-210.18550,-210.99954,-204.11323,-214.18072
CG7465,-235.90670,1059.9249,-114.37894,18672.4734,-233.30243,-260.61371,-60.82300,-248.13785,-260.06722,-265.75838,⋯,-255.35024,-216.25387,-226.75934,-219.59669,-230.93808,-236.41273,-232.02433,-226.43120,-225.11492,-248.07729
CG32603,-128.71377,533.8388,-95.29580,9225.9645,-121.89916,-128.71377,-128.71377,-116.33839,-127.76542,-128.71377,⋯,-124.27018,-103.52994,-120.16813,-107.56710,-109.47638,-110.69538,-114.82409,-106.24727,-105.55978,-119.70210


In [241]:
saveRDS(exp_matrix_sub300_cell.type.2_no_doubletons_mean, 
       "data/exp_matrix_PCA300_cell.type.2_no_doubletons_mean.rds")

In [694]:
saveRDS(exp_matrix_sub300_cell.type.2_no_doubletons, 
       "data/exp_matrix_PCA300_cell.type.2_no_doubletons.rds")

In [1964]:
ls(pattern = "matrix")

[1] "exp_matrix"                                        
 [2] "exp_matrix_mean"                                   
 [3] "exp_matrix_sub300_cell.type.2_no_doubletons"       
 [4] "exp_matrix_sub300_cell.type.2_no_doubletons_mean"  
 [5] "exp_matrix_sub300_no_doubletons"                   
 [6] "exp_matrix_sub300_no_singletons"                   
 [7] "get.norm.expr.matrix"                              
 [8] "get.norm.expr.matrix.3"                            
 [9] "TF_exp_matrix"                                     
[10] "TF_exp_matrix_emb"                                 
[11] "TF_exp_matrix_emb_log"                             
[12] "TF_exp_matrix_log"                                 
[13] "TF_exp_matrix_max"                                 
[14] "TF_exp_matrix_norm"                                
[15] "TF_exp_matrix_sub300_cell.type.2_no_doubletons"    
[16] "TF_exp_matrix_sub300_cell.type.2_no_doubletons_log"
[17] "TF_exp_matrix_sub300_no_doubletons"                
[18] "TF_exp_matrix_sub300_no_doubletons_log"

In [242]:
exp_matrix_mean_original = exp_matrix_mean

In [243]:
exp_matrix_mean = exp_matrix_sub300_cell.type.2_no_doubletons_mean

In [244]:
gen = "bs"

In [463]:
options(repr.plot.width=12, repr.plot.height=10)

In [246]:
head(fly_emb_TFs, 20)

ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function 'head': object 'fly_emb_TFs' not found


In [44]:

tf = gen
message(tf)
    
    file_name = paste(tf, "aggreg_vs_TF_profile","jpg", sep = ".")
    file_name = paste("worm_agg_emb_graphs_emb_targets", file_name, sep = "/")

            
          
target = "tx"
    
target_values = exp_matrix_mean  %>% as.data.frame  %>% filter(rownames(exp_matrix_mean) == target)
t_target_values = as.data.frame(t(target_values))
t_target_values_row = rownames_to_column(t_target_values, var = "cell.type")
 colnames(t_target_values_row) <- c("cell.type", "expr")

 
    
tf_values = exp_matrix_mean  %>% as.data.frame  %>% filter(rownames(exp_matrix_mean) == tf)
t_tf_values = as.data.frame(t(tf_values))
t_tf_values_row = rownames_to_column(t_tf_values, var = "cell.type")
 colnames(t_tf_values_row) <- c("cell.type", "expr")

    
title = paste(target, "_from_embryo", sep = "_")  

#agg_subtitle = paste("max", target_max, "target_mean", target_mean, "target_entropy", target_entropy,  sep = "_" )    
gg_agg = ggplot(t_target_values_row, aes(x = cell.type, y = expr)) + 
ggtitle(title) +
theme_classic() +
theme(axis.text.x= element_text(size = 0.1,angle = -90, hjust = 0)) +
 geom_bar(stat="identity")

    tf_title = paste(tf, "alone", sep = " ")

#    tf_subtitle = paste ("max", tf_max, "tf_mean", tf_mean, "tf_entropy", tf_entropy,  sep = "_")
gg_tf = ggplot(t_tf_values_row, aes(x = cell.type, y = expr)) +
    theme_classic() +
theme(axis.text.x= element_text(size = 9,angle = -90, hjust = 0)) +
  geom_bar(stat="identity") + ggtitle(tf_title)

#jpeg("test.jpg", width =1100, height = 1100)
    gg_both = plot_grid(gg_agg, gg_tf,              # Draw grid of plots
          ncol = 1,
          rel_heights = c(1,1.8),
          align = "v")
    print(gg_both)


ERROR: Error in eval(expr, envir, enclos): object 'gen' not found


In [468]:
df_sub_no_doubletons  %>% count(cell.type.2, annot)  %>% 
       arrange(cell.type.2, -n)  %>% group_by(cell.type.2) %>% 
    mutate(rank = rank(n))  %>%  slice_max(rank, n = 4)  %>% head(60)  %>% tail(60)

cell.type.2,annot,n,rank
<chr>,<chr>,<int>,<dbl>
a,midgut,78,27.0
a,ubiquitous,58,26.0
a,epidermis,49,25.0
a,salivary gland,42,24.0
aa,foregut / hindgut / anal pad,640,21.0
aa,epidermis,354,20.0
aa,hindgut / anal pad,224,19.0
aa,somatic muscle,99,18.0
ab,epidermis,516,21.0


In [272]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "ac")  %>% head(10)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
pip3,ac,pip,47027,6925,32792.0852,264.990019,6.945832,104.19514
nur3,ac,nur,26613,9706,19927.8068,355.439960,5.804980,82.91002
CG147563,ac,CG14756,14481,5763,10818.1132,201.054867,5.742558,76.95771
CG131593,ac,CG13159,6748,2596,4972.8202,104.269209,5.561909,68.30101
CG341903,ac,CG34190,914,134,618.1632,4.290539,6.868429,63.69902
PH4alphaSG13,ac,PH4alphaSG1,1719,504,1247.2033,17.378873,6.084505,62.58301
PH4alphaSG23,ac,PH4alphaSG2,1097,293,789.6215,9.326634,6.256719,60.23246
CrzR3,ac,CrzR,952,222,676.5130,9.290377,6.038750,56.78904
GlcAT-P3,ac,GlcAT-P,5727,4643,4109.0061,205.663831,4.313432,51.78242


In [276]:
cell.type.markers.sub300.cell.type  %>% head

,cell.type,gene,cell.type.n.umi,other.n.umi,cell.type.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG92694,cellular blastoderm / pole cell,CG9269,19,157,4414.498,5.530986,9.400733,113.82746
CG50704,cellular blastoderm / pole cell,CG5070,17,184,3949.814,6.730373,8.997031,107.49593
Jon65Aiv4,cellular blastoderm / pole cell,Jon65Aiv,16,819,4000.000,16.867280,7.806536,93.41414
pip32,salivary gland,pip,38398,9911,29916.415,412.274111,6.177695,91.85427
Ppn28,plasmatocytes,Ppn,73383,14116,35715.779,536.739395,6.053511,91.55520
CG43332,amnioserosa,CG43332,23490,1608,7785.421,56.955284,7.069691,91.38809


In [279]:
unique((cell.type.markers.sub300.cell.type %>% filter(heuristic.score > 60))$cell.type)
unique((cell.type.markers.sub300.cell.type %>% filter(heuristic.score > 60))$cell.type)  %>% length

[1] "cellular blastoderm / pole cell"   "salivary gland"                   
 [3] "plasmatocytes"                     "amnioserosa"                      
 [5] "maternal"                          "ectoderm anlage"                  
 [7] "Malpighian tubule"                 "tracheal primordium"              
 [9] "salivary gland body primordium"    "glia"                             
[11] "crystal cell"                      "spiracles (tracheal system)"      
[13] "ectoderm anlage in statu nascendi" "proventriculus"                   
[15] "spiracle (tracheal system)"        "unknown"                          
[17] "visceral muscle"                   "tracheal system"                  
[19] "fat body"

[1] 19

In [294]:
unique((cell.type.2.markers_no_doubletons  %>% filter(heuristic.score > 40 & gene %in% fly_TFs))$gene)

[1] "lz"      "lab"     "pdm3"    "sim"     "cas"     "seq"     "sv"     
 [8] "Eip74EF" "otp"     "Doc2"    "Ptx1"    "fru"     "Sp1"     "svp"    
[15] "inv"     "C15"     "CG42741" "Lim3"    "Crg-1"   "HHEX"    "fd3F"   
[22] "disco-r" "Clamp"   "lmd"     "slbo"    "CG34224" "SoxN"    "ken"    
[29] "Atf6"    "Drgx"    "HLH54F"  "sens"    "tup"     "ham"     "ss"     
[36] "pros"    "dimm"    "CG4328"  "Oaz"     "ara"     "kni"     "dysf"   
[43] "ftz-f1"  "CrebA"   "Mes2"    "CG9932"  "Kah"     "Sox102F" "Rfx"    
[50] "ush"     "pdm2"    "H2.0"    "en"      "jigr1"   "Eip93F"  "caup"   
[57] "pho"     "sna"     "sens-2"  "tx"      "dve"     "CG43347" "klu"    
[64] "Dref"

In [45]:
tf = "lz"
message(tf)
    
    file_name = paste(tf, "aggreg_vs_TF_profile","jpg", sep = ".")
    file_name = paste("worm_agg_emb_graphs_emb_targets", file_name, sep = "/")

            
          
target = "lab"
    
target_values = exp_matrix_mean  %>% as.data.frame  %>% filter(rownames(exp_matrix_mean) == target)
t_target_values = as.data.frame(t(target_values))
t_target_values_row = rownames_to_column(t_target_values, var = "cell.type")
 colnames(t_target_values_row) <- c("cell.type", "expr")

 
    
tf_values = exp_matrix_mean  %>% as.data.frame  %>% filter(rownames(exp_matrix_mean) == tf)
t_tf_values = as.data.frame(t(tf_values))
t_tf_values_row = rownames_to_column(t_tf_values, var = "cell.type")
 colnames(t_tf_values_row) <- c("cell.type", "expr")

    
title = paste(target, "_from_embryo", sep = "_")  

#agg_subtitle = paste("max", target_max, "target_mean", target_mean, "target_entropy", target_entropy,  sep = "_" )    
gg_agg = ggplot(t_target_values_row, aes(x = cell.type, y = expr)) + 
ggtitle(title) +
theme_classic() +
theme(axis.text.x= element_text(size = 0.1,angle = -90, hjust = 0)) +
 geom_bar(stat="identity")

    tf_title = paste(tf, "alone", sep = " ")

#    tf_subtitle = paste ("max", tf_max, "tf_mean", tf_mean, "tf_entropy", tf_entropy,  sep = "_")
gg_tf = ggplot(t_tf_values_row, aes(x = cell.type, y = expr)) +
    theme_classic() +
theme(axis.text.x= element_text(size = 9,angle = -90, hjust = 0)) +
  geom_bar(stat="identity") + ggtitle(tf_title)

#jpeg("test.jpg", width =1100, height = 1100)
    gg_both = plot_grid(gg_agg, gg_tf,              # Draw grid of plots
          ncol = 1,
          rel_heights = c(1,1.8),
          align = "v")
    print(gg_both)


lz



ERROR: Error in exp_matrix_mean %>% as.data.frame %>% filter(rownames(exp_matrix_mean) == : could not find function "%>%"


In [46]:
plot.expr_clus.br2(cds_sub, gene = "lz", size_pos = 0.1, size_neg = 0.02, stroke = 0.01)

ERROR: Error in plot.expr_clus.br2(cds_sub, gene = "lz", size_pos = 0.1, size_neg = 0.02, : could not find function "plot.expr_clus.br2"


In [47]:
plot.expr.fly(cds_sub, gene = "Muc11A", size_pos = 0.3, size_neg = 0.3, stroke = 0.01)

ERROR: Error in plot.expr.fly(cds_sub, gene = "Muc11A", size_pos = 0.3, size_neg = 0.3, : could not find function "plot.expr.fly"


In [308]:
plot.expr_clus.br2

function (cds, gene_short_name, coexpr = F, ncol = NULL, x = 1, 
    y = 2, size_pos = NULL, stroke = NULL, non.expr.alpha = 1/3, 
    color.pal = c("grey85", "red", "orange", "yellow"), sample.n = F, 
    label_size = NULL, limit = NULL, size_neg = NULL) 
{
    if (sample.n != F) {
        set.seed(42)
        cds = cds[, sample(1:ncol(cds), sample.n)]
    }
    S_matrix <- reducedDims(cds)[["UMAP"]]
    data_df <- data.frame(S_matrix)
    colnames(data_df) <- c("umap_1", "umap_2")
    data_df$cluster = as.vector(clusters(cds))
    data_df$sample_name <- row.names(data_df)
    data_df$Size_Factor <- colData(cds)$Size_Factor
    data_df$lineage <- colData(cds)$lineage
    data_df$species <- colData(cds)$species
    if (length(nchar(gene_short_name)) > 1) {
        label_df <- data_df %>% group_by(cluster) %>% summarize(umap_1 = median(umap_1), 
            umap_2 = median(umap_2))
        a <- as.data.frame(as.matrix(t(counts(cds)[gene_short_name, 
            ])))
        a <- a/data_df$Size_Factor
        a$min <- apply(a, 1, min)
        data_df$min <- a$min
        if (is.null(size_pos)) {
            size_pos = 0.1
        }
        if (is.null(stroke)) {
            stroke = 0.4
        }
        if (is.null(label_size)) {
            label_size = 4
        }
        if (is.null(limit)) {
            limit = 1
        }
        if (is.null(size_neg)) {
            size_neg = 0.1
        }
        plot = ggplot(data_df, alpha = min > 0) + geom_point(aes(x = umap_1, 
            y = umap_2, color = log2(min + 1)), size = ifelse(data_df$min > 
            limit, size_pos, size_neg), stroke = stroke) + scale_color_gradientn(colors = color.pal) + 
            scale_alpha_manual(values = c(non.expr.alpha, 1)) + 
            guides(alpha = F, color = guide_colorbar(title = "Minimum\nnormalized\nlog2 expression\nfor specified genes")) + 
            geom_text(data = label_df, aes(label = cluster, x = umap_1 + 
                0.25, y = umap_2 + 0.25), size = label_size) + 
            monocle3:::monocle_theme_opts()
        return(plot)
    }
    if (length(nchar(gene_short_name)) == 1) {
        label_df <- data_df %>% group_by(cluster) %>% summarize(umap_1 = median(umap_1), 
            umap_2 = median(umap_2))
        data_df$tmp.expr = counts(cds)[gene_short_name, ]/data_df$Size_Factor
        if (is.null(size_pos)) {
            size_pos = 0
        }
        if (is.null(stroke)) {
            stroke = 0.4
        }
        if (is.null(label_size)) {
            label_size = 4
        }
        if (is.null(limit)) {
            limit = 1
        }
        if (is.null(size_neg)) {
            size_neg = 0.1
        }
        plot = ggplot(data_df) + geom_point(aes(x = umap_1, y = umap_2, 
            color = log2(tmp.expr/Size_Factor + 1), alpha = tmp.expr > 
                0), size = ifelse(data_df$tmp.exp > limit, size_pos, 
            size_neg), stroke = stroke) + scale_color_gradientn(colors = color.pal) + 
            scale_alpha_manual(values = c(non.expr.alpha, 1)) + 
            guides(alpha = F, color = guide_colorbar(title = paste(gene_short_name, 
                "\nnormalized\nlog2 expression", sep = ""))) + 
            geom_text(data = label_df, aes(label = cluster, x = umap_1 + 
                0.25, y = umap_2 + 0.25), size = label_size) + 
            monocle3:::monocle_theme_opts()
        return(plot)
    }
}
<bytecode: 0x7fa45f38d7b0>

In [313]:
plot.expr.fly = function (cds, gene_short_name, coexpr = F, ncol = NULL, x = 1, 
    y = 2, size_pos = NULL, stroke = NULL, non.expr.alpha = 1/3, 
    color.pal = c("grey85", "red", "orange", "yellow"), sample.n = F, 
    label_size = NULL, limit = NULL, size_neg = NULL) 
{
    if (sample.n != F) {
        set.seed(42)
        cds = cds[, sample(1:ncol(cds), sample.n)]
    }
    S_matrix <- reducedDims(cds)[["UMAP"]]
    data_df <- data.frame(S_matrix)
    colnames(data_df) <- c("umap_1", "umap_2")
    data_df$cell.type.2 = colData(cds)$cell.type.2
    data_df$sample_name <- row.names(data_df)
    data_df$Size_Factor <- colData(cds)$Size_Factor
    data_df$lineage <- colData(cds)$lineage
    data_df$species <- colData(cds)$species
    if (length(nchar(gene_short_name)) > 1) {
        label_df <- data_df %>% group_by(cell.type.2) %>% summarize(umap_1 = median(umap_1), 
            umap_2 = median(umap_2))
        a <- as.data.frame(as.matrix(t(counts(cds)[gene_short_name, 
            ])))
        a <- a/data_df$Size_Factor
        a$min <- apply(a, 1, min)
        data_df$min <- a$min
        if (is.null(size_pos)) {
            size_pos = 0.1
        }
        if (is.null(stroke)) {
            stroke = 0.4
        }
        if (is.null(label_size)) {
            label_size = 4
        }
        if (is.null(limit)) {
            limit = 1
        }
        if (is.null(size_neg)) {
            size_neg = 0.1
        }
        plot = ggplot(data_df, alpha = min > 0) + geom_point(aes(x = umap_1, 
            y = umap_2, color = log2(min + 1)), size = ifelse(data_df$min > 
            limit, size_pos, size_neg), stroke = stroke) + scale_color_gradientn(colors = color.pal) + 
            scale_alpha_manual(values = c(non.expr.alpha, 1)) + 
            guides(alpha = F, color = guide_colorbar(title = "Minimum\nnormalized\nlog2 expression\nfor specified genes")) + 
            geom_text(data = label_df, aes(label = cell.type.2, x = umap_1 + 
                0.25, y = umap_2 + 0.25), size = label_size) + 
            monocle3:::monocle_theme_opts()
        return(plot)
    }
    if (length(nchar(gene_short_name)) == 1) {
        label_df <- data_df %>% group_by(cell.type.2) %>% summarize(umap_1 = median(umap_1), 
            umap_2 = median(umap_2))
        data_df$tmp.expr = counts(cds)[gene_short_name, ]/data_df$Size_Factor
        if (is.null(size_pos)) {
            size_pos = 0
        }
        if (is.null(stroke)) {
            stroke = 0.4
        }
        if (is.null(label_size)) {
            label_size = 4
        }
        if (is.null(limit)) {
            limit = 1
        }
        if (is.null(size_neg)) {
            size_neg = 0.1
        }
        plot = ggplot(data_df) + geom_point(aes(x = umap_1, y = umap_2, 
            color = log2(tmp.expr/Size_Factor + 1), alpha = tmp.expr > 
                0), size = ifelse(data_df$tmp.exp > limit, size_pos, 
            size_neg), stroke = stroke) + scale_color_gradientn(colors = color.pal) + 
            scale_alpha_manual(values = c(non.expr.alpha, 1)) + 
            guides(alpha = F, color = guide_colorbar(title = paste(gene_short_name, 
                "\nnormalized\nlog2 expression", sep = ""))) + 
            geom_text(data = label_df, aes(label = cell.type.2, x = umap_1 + 
                0.25, y = umap_2 + 0.25), size = label_size) + 
            monocle3:::monocle_theme_opts()
        return(plot)
    }
}

### distribution of annot cell types in cds_sub

In [345]:
unique(df_sub_no_doubletons$annot)  %>% sort

[1] "amnioserosa"                         
 [2] "anlage in statu nascendi"            
 [3] "brain"                               
 [4] "brain primordium"                    
 [5] "cellular blastoderm / pole cell"     
 [6] "crystal cell"                        
 [7] "ectoderm anlage"                     
 [8] "ectoderm anlage in statu nascendi"   
 [9] "ectoderm primordium"                 
[10] "endoderm primordium"                 
[11] "epidermis"                           
[12] "epidermis primordium"                
[13] "fat body"                            
[14] "foregut / hindgut / anal pad"        
[15] "foregut / hindgut primordium"        
[16] "germ cell"                           
[17] "glia"                                
[18] "hindgut / anal pad"                  
[19] "hindgut primordium"                  
[20] "Malpighian tubule"                   
[21] "maternal"                            
[22] "mesoderm primordium"                 
[23] "midgut"                              
[24] "midgut primordium"                   
[25] "midline primordium"                  
[26] "muscle"                              
[27] "peripheral nervous system"           
[28] "peripheral nervous system primordium"
[29] "plasmatocytes"                       
[30] "plasmatocytes anlage"                
[31] "procrystal cell"                     
[32] "proventriculus"                      
[33] "salivary gland"                      
[34] "salivary gland body primordium"      
[35] "salivary gland primordium"           
[36] "sensory nervous system"              
[37] "sensory nervous system primordium"   
[38] "somatic muscle"                      
[39] "somatic muscle primordium"           
[40] "spiracle (tracheal system)"          
[41] "spiracles (tracheal system)"         
[42] "tracheal primordium"                 
[43] "tracheal system"                     
[44] "trunk mesoderm primordium"           
[45] "ubiquitous"                          
[46] "unknown"                             
[47] "ventral nerve cord"                  
[48] "ventral nerve cord primordium"       
[49] "visceral muscle"                     
[50] "visceral muscle primordium"          
[51] "yolk nuclei"

In [48]:
annot_type = "yolk nuclei"
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$annot %in% annot_type), size =  ifelse(df_sub300$annot %in% annot_type, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [444]:
df_sub_no_doubletons  %>% count(cell.type.2, annot)  %>% 
       arrange(cell.type.2, -n)  %>% group_by(cell.type.2) %>% 
    mutate(rank = rank(n))  %>%  filter(annot == annot_type)  %>% arrange(-n)

cell.type.2,annot,n,rank
<chr>,<chr>,<int>,<dbl>
ca,yolk nuclei,36,22.0
bl,yolk nuclei,33,26.0
bt,yolk nuclei,33,6.0
s,yolk nuclei,32,24.0
bq,yolk nuclei,31,3.0
i,yolk nuclei,26,13.0
bk,yolk nuclei,21,21.5
bs,yolk nuclei,21,5.0
bx,yolk nuclei,21,6.0


In [453]:
df_sub_no_doubletons  %>% count(cell.type.2, annot)  %>% 
       arrange(cell.type.2, -n)  %>% group_by(cell.type.2) %>% 
    mutate(rank = rank(n))  %>%  filter(cell.type.2 == "x") %>% head(120)  %>% tail(60)

cell.type.2,annot,n,rank
<chr>,<chr>,<int>,<dbl>
x,midgut,381,24.0
x,epidermis,38,23.0
x,somatic muscle,17,22.0
x,proventriculus,14,21.0
x,brain,7,20.0
x,fat body,6,19.0
x,tracheal system,5,17.5
x,ventral nerve cord,5,17.5
x,foregut / hindgut / anal pad,4,15.0


In [470]:
ls(pattern = "marker")

[1] "cell.type.2.markers"                             
[2] "cell.type.2.markers_no_doubletons"               
[3] "cell.type.markers.sub300_no_singletons.cell.type"
[4] "cell.type.markers.sub300.cell.type"              
[5] "cluster.markers"                                 
[6] "cluster.markers.sub300"                          
[7] "cluster.markers.sub300_no_doubletons"            
[8] "cluster.markers.sub300_no_doubletons_all"        
[9] "cluster.markers.sub300_no_singletons"

## get markers for Sue.

In [473]:
cell.type.2.markers_no_doubletons  %>% arrange(-heuristic.score)  %>% group_by(cell.type.2) %>% 
    slice_max(heuristic.score, n = 10)  %>% head(60)

cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
a,Muc26B,3651,121,11236.035477,3.9689778,11.142896,149.93852
a,CG7017,3415,236,12317.771831,9.9390129,10.137043,137.74793
a,Muc96D,2335,122,6924.704567,5.8653887,9.978195,127.29927
a,CG17826,970,33,2991.398860,1.5259765,10.209763,117.89302
a,CG11300,477,17,1102.911259,0.6713437,9.366093,94.67629
a,EAChm,257,117,1599.503427,3.7442647,8.397224,89.38265
a,CG18649,97,6,530.483940,0.2496856,8.729600,79.03677
a,CG32198,211,11,540.493251,0.3955226,8.597327,78.07061
a,CG32453,83,16,549.659426,0.4831467,8.533733,77.69978


In [640]:
fly_top_ten_markers.20 = cell.type.2.markers_no_doubletons  %>% filter(cell.type.2.tpm > other.tpm)  %>% arrange(-heuristic.score)  %>% group_by(cell.type.2) %>% 
    slice_max(heuristic.score, n = 20) 

In [641]:
write.table(fly_top_ten_markers.20, "data/fly_top_ten_markers_new_annotation.20.tsv", quote = FALSE, sep = "\t")

In [642]:
df_sub_no_doubletons  %>% count(cell.type.2)  

cell.type.2,n
<chr>,<int>
a,432
aa,1540
ab,1048
ac,3765
ad,719
ae,1019
af,94
ag,495
ah,2347


In [644]:
fly_top_ten_markers.20   %>% filter(gene == "ad")

cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


In [488]:
colnames(fly_top_ten_markers)

[1] "cell.type.2"       "gene"              "cell.type.2.n.umi"
[4] "other.n.umi"       "cell.type.2.tpm"   "other.tpm"        
[7] "log2.ratio"        "heuristic.score"

## Get TF targets for Sue

In [645]:
ls()

[1] "a"                                                 
  [2] "aa"                                                
  [3] "ab"                                                
  [4] "ac"                                                
  [5] "ad"                                                
  [6] "ae"                                                
  [7] "af"                                                
  [8] "ag"                                                
  [9] "ah"                                                
 [10] "ai"                                                
 [11] "aj"                                                
 [12] "ak"                                                
 [13] "al"                                                
 [14] "am"                                                
 [15] "an"                                                
 [16] "annot_type"                                        
 [17] "ao"                                                
 [18] "ap"                                                
 [19] "aq"                                                
 [20] "ar"                                                
 [21] "as"                                                
 [22] "at"                                                
 [23] "au"                                                
 [24] "av"                                                
 [25] "aw"                                                
 [26] "ax"                                                
 [27] "ay"                                                
 [28] "az"                                                
 [29] "b"                                                 
 [30] "ba"                                                
 [31] "bb"                                                
 [32] "bc"                                                
 [33] "bd"                                                
 [34] "be"                                                
 [35] "bf"                                                
 [36] "bg"                                                
 [37] "bh"                                                
 [38] "bi"                                                
 [39] "bj"                                                
 [40] "bk"                                                
 [41] "bl"                                                
 [42] "bm"                                                
 [43] "bn"                                                
 [44] "bo"                                                
 [45] "bp"                                                
 [46] "bq"                                                
 [47] "br"                                                
 [48] "bs"                                                
 [49] "bt"                                                
 [50] "bu"                                                
 [51] "bv"                                                
 [52] "bw"                                                
 [53] "bx"                                                
 [54] "by"                                                
 [55] "bz"                                                
 [56] "c"                                                 
 [57] "ca"                                                
 [58] "cb"                                                
 [59] "cc"                                                
 [60] "cd"                                                
 [61] "cds_sub"                                           
 [62] "cds_sub_no_doubletons"                             
 [63] "cds_sub_no_multitons"                              
 [64] "cds_sub_no_sing"                                   
 [65] "ce"                                                
 [66] "cell_type"                                         
 [67] "cell.type"                                         
 [68] "cell.type.2.markers"                      

In [647]:
fly_jsd_tf_target = readRDS("data/Fly_JSD_ChiSquare_ttest.rds")

In [648]:
fly_jsd_tf_target  %>% head

TF,nTargets,JSD,wilcoxP,chiP,t_test_p,targetMean,randomMean
<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ab,135,0.2587306,3.316526e-01,1.081180e-04,1.457897e-01,0.007761771,-0.034110661
abd-A,2877,0.1841270,1.619032e-77,1.289780e-65,1.048386e-75,0.046724300,-0.054141999
Abd-B,1522,0.1162042,1.142477e-11,1.274070e-10,2.915755e-14,0.002285970,-0.038557669
ac,799,0.2061522,7.235478e-15,1.076058e-13,2.997855e-17,0.107429744,0.004356283
Adf1,1546,0.1626163,9.429591e-18,8.331135e-27,1.093153e-12,0.018633070,0.099185161
Aef1,1083,0.1370532,1.211261e-08,3.250265e-13,6.120103e-04,0.053187933,0.098447248


In [655]:
fly_jsd_tf_target  %>% filter(wilcoxP < 0.00001 & JSD > 0.18)  %>% count

n
<int>
66


In [658]:
options(repr.matrix.max.rows = 100)

In [659]:
fly_jsd_tf_target_trim = fly_jsd_tf_target  %>% filter(wilcoxP < 0.00001 & JSD > 0.18)

In [660]:
fly_jsd_tf_target_trim

TF,nTargets,JSD,wilcoxP,chiP,t_test_p,targetMean,randomMean
<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
abd-A,2877,0.1841270,1.619032e-77,1.289780e-65,1.048386e-75,0.046724300,-0.054141999
ac,799,0.2061522,7.235478e-15,1.076058e-13,2.997855e-17,0.107429744,0.004356283
Asciz,1015,0.2666737,1.381913e-58,1.012269e-47,6.875121e-57,-0.059485335,0.139650653
ATbp,1094,0.2460150,1.277078e-52,7.181448e-44,1.770003e-49,-0.069432889,0.120991008
bcd,966,0.1976081,2.120398e-25,4.919029e-25,2.333386e-16,0.016560839,0.135763915
BtbVII,620,0.2092590,6.663234e-19,4.253111e-12,3.976728e-18,-0.047787715,0.124351316
CG10209,1056,0.1938530,1.188169e-33,8.973868e-25,1.125802e-29,-0.014933450,0.131503296
CG10274,298,0.1940550,1.103532e-07,1.652799e-03,1.172761e-07,0.006879823,0.136545159
CG10949,2101,0.1910954,1.981784e-37,2.235072e-55,2.206041e-41,0.282186925,0.138537611


In [661]:
fly_targets_new_annot = read_tsv("fly_TF_TargetAngles_2_277_no_doubletons.tsv")

Rows: 699829 Columns: 7
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (4): peakID, TF, Gene, lifeStage
dbl (3): CosineAngle, nPeaks, targetDist

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [662]:
fly_random_new_annot = read_tsv("fly_TF_RandomAngles_2_277_no_doubletons.tsv")

Rows: 671260 Columns: 3
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): TF, Gene
dbl (1): CosineAngle

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [663]:
fly_targets_new_annot  %>%  head

CosineAngle,peakID,TF,Gene,nPeaks,targetDist,lifeStage
<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
0.14595045,ab_ab-GFP_embryonic_1chrX17176552:17176952,ab,baz,251,16746,embryonic
-0.38162215,ab_ab-GFP_embryonic_1chrX13994830:13995230,ab,ben,168,-1315,embryonic
0.08301382,ab_ab-GFP_embryonic_1chr2R10227892:10228292,ab,Hr3,212,-135,embryonic
0.10863988,ab_ab-GFP_embryonic_1chr3L17949012:17949412,ab,Eip75B,175,12602,embryonic
0.15421326,ab_ab-GFP_embryonic_1chr3L12468026:12468426,ab,eyg,262,-205,embryonic
0.38344353,ab_ab-GFP_embryonic_1chr2L18327114:18327514,ab,Fas3,220,7212,embryonic


In [670]:
fly_top_TFs = unique(fly_jsd_tf_target_trim$TF)

In [671]:
fly_top_TFs   %>% length

[1] 66

In [672]:
fly_targets_new_annot %>% arrange(-CosineAngle)  %>% group_by(TF) %>% 
    slice_max(CosineAngle, n = 10)  %>% filter(TF %in% fly_top_TFs) 

CosineAngle,peakID,TF,Gene,nPeaks,targetDist,lifeStage
<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
0.6189138,ATbp_ATbp-GFP_embryonic_1chr2L6028600:6029220,ATbp,CG9107,63,-4371,embryonic
0.6096739,ATbp_ATbp-GFP_embryonic_1chrX18839131:18839751,ATbp,htk,105,-4222,embryonic
0.5729203,ATbp_ATbp-GFP_embryonic_1chr3L14780103:14780723,ATbp,mop,143,-4284,embryonic
0.5657947,ATbp_ATbp-GFP_embryonic_1chr2L19126804:19127424,ATbp,l(2)37Cb,230,-138,embryonic
0.5621825,ATbp_ATbp-GFP_embryonic_1chrX14242925:14243545,ATbp,mRNA-cap,256,-413,embryonic
0.5530836,ATbp_ATbp-GFP_embryonic_1chr3R5746732:5746967,ATbp,Madm,48,4324,embryonic
0.5304849,ATbp_ATbp-GFP_embryonic_1chr3R22530151:22530771,ATbp,CG33099,215,111,embryonic
0.5222559,ATbp_ATbp-GFP_embryonic_1chr3R21263270:21263491,ATbp,Rab1,184,5689,embryonic
0.5117363,ATbp_ATbp-GFP_embryonic_1chr3L21464358:21464622,ATbp,croc,246,11983,embryonic


In [675]:
fly_targets_new_annot %>% arrange(CosineAngle)  %>% group_by(TF) %>% 
    slice_min(CosineAngle, n = 10)  %>% filter(TF %in% fly_top_TFs) 

CosineAngle,peakID,TF,Gene,nPeaks,targetDist,lifeStage
<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
-0.4859642,ATbp_ATbp-GFP_embryonic_1chr3L440263:440570,ATbp,klar,202,666,embryonic
-0.4460530,ATbp_ATbp-GFP_embryonic_1chrX9349102:9349722,ATbp,Zdhhc8,247,6815,embryonic
-0.4363592,ATbp_ATbp-GFP_embryonic_1chr2L12462886:12463506,ATbp,bun,139,-1940,embryonic
-0.4319920,ATbp_ATbp-GFP_embryonic_1chr2L17387950:17388570,ATbp,CLIP-190,72,-1081,embryonic
-0.4313847,ATbp_ATbp-GFP_embryonic_1chr2L2904931:2905551,ATbp,lilli,241,15070,embryonic
-0.4272610,ATbp_ATbp-GFP_embryonic_1chr2L8097305:8097925,ATbp,Bsg,234,8512,embryonic
-0.4179265,ATbp_ATbp-GFP_embryonic_1chr3R21330422:21331042,ATbp,aus,237,18869,embryonic
-0.4082246,ATbp_ATbp-GFP_embryonic_1chr3R21594255:21594875,ATbp,InR,178,-2444,embryonic
-0.4081062,ATbp_ATbp-GFP_embryonic_1chrX3704290:3704910,ATbp,Rala,221,15035,embryonic


In [673]:
fly_targets_new_annot_top10 = fly_targets_new_annot %>% arrange(-CosineAngle)  %>% group_by(TF) %>% 
    slice_max(CosineAngle, n = 10)  %>% filter(TF %in% fly_top_TFs)

In [676]:
fly_targets_new_annot_bot10 = fly_targets_new_annot %>% arrange(CosineAngle)  %>% group_by(TF) %>% 
    slice_min(CosineAngle, n = 10)  %>% filter(TF %in% fly_top_TFs)

In [677]:
write_tsv(fly_targets_new_annot_top10, "data/fly_targets_new_annot_top10.tsv")

In [678]:
write_tsv(fly_targets_new_annot_bot10, "data/fly_targets_new_annot_bot10.tsv")

# Redo nearest neighbor to get all 20 neighbors

In [18]:
cells <- colData(cds_sub)

In [19]:
cellTypes <- cells$annot

In [20]:
cellIDs <- cells$cell

In [21]:
pca <- cds_sub@int_colData$reducedDims$PCA

In [22]:
library(RANN)

In [23]:
nnMatches <- function(ids,labels,df,k=10){
   k1 <- k + 1  # need one more since always finds itself
   rann <- RANN::nn2(df,query=df,k1)
   df <- as.data.frame(rann$nn.idx)
   df <- mutate(df,ID=ids,.before=1)  # add the ids to the df
   df <- mutate(df,Label=labels[V1],.before=2) %>% select(-V1)  # put the labels into the df
   df <- pivot_longer(df,cols=3:k1,names_to = "NN")
   df <- mutate(df,nnLabel=labels[value])  # label the nearest neighbors
   df <- select(df,-NN,-value)
   res <- group_by(df,ID) %>% summarise(nk=sum (Label==nnLabel)  )  # count the matches in the nearest neighors
   return(res)
}

### Do individual steps of Lou's function above

In [24]:
k = 20

In [25]:
k1 <- k + 1

In [26]:
df = pca

In [27]:
rann <- RANN::nn2(df,query=df,k1)

In [28]:
saveRDS(rann, "data/fly_rann_PCA300.rds")

In [492]:
ids = cellIDs

In [493]:
df <- as.data.frame(rann$nn.idx)

In [494]:
head(df)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,1084,117474,33297,2934,2355,854,2437,4221,63518,⋯,116766,48132,20116,115992,34544,31818,50700,143608,571,3626
2,2,51177,19874,69268,44378,67519,212700,189174,131569,24279,⋯,20273,130972,9391,3473,109102,132457,130068,46597,53857,183430
3,3,137166,35175,4381,123690,99621,35423,123344,108376,126736,⋯,123710,47008,134782,136306,137709,123045,33837,32261,136304,135305
4,4,68440,71253,67238,73868,67236,71880,64019,64022,66768,⋯,64612,63786,70436,65552,2853,65551,67168,39930,28538,74114
5,5,137728,19780,120926,91533,5730,144152,143823,35446,54161,⋯,51952,4980,134914,19964,132236,52185,34199,214925,117952,68538
6,6,36849,1387,2385,63108,36476,2887,67810,3884,35089,⋯,5792,75285,2730,19767,34789,35981,1882,64621,49619,781


In [495]:
df <- mutate(df,ID=ids,.before=1)

In [496]:
head(df)

,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,⋯,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,1,1084,117474,33297,2934,2355,854,2437,4221,⋯,116766,48132,20116,115992,34544,31818,50700,143608,571,3626
2,exp1_idx1AGACCATATC_idx2AACGAGGCTA.ACGAGGTTTTTAATTATTG,2,51177,19874,69268,44378,67519,212700,189174,131569,⋯,20273,130972,9391,3473,109102,132457,130068,46597,53857,183430
3,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCAGAACCTCAGGTATGGA,3,137166,35175,4381,123690,99621,35423,123344,108376,⋯,123710,47008,134782,136306,137709,123045,33837,32261,136304,135305
4,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGAGAGATAAGTAGTCAG,4,68440,71253,67238,73868,67236,71880,64019,64022,⋯,64612,63786,70436,65552,2853,65551,67168,39930,28538,74114
5,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGGAGTTTTGAGGTAGAA,5,137728,19780,120926,91533,5730,144152,143823,35446,⋯,51952,4980,134914,19964,132236,52185,34199,214925,117952,68538
6,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGTCTCATTCGATATTGA,6,36849,1387,2385,63108,36476,2887,67810,3884,⋯,5792,75285,2730,19767,34789,35981,1882,64621,49619,781


In [497]:
labels = cells$annot

In [498]:
df <- mutate(df,Label=labels[V1],.before=2) %>% select(-V1) 

In [499]:
head(df)

,ID,Label,V2,V3,V4,V5,V6,V7,V8,V9,⋯,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21
,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,1084,117474,33297,2934,2355,854,2437,4221,⋯,116766,48132,20116,115992,34544,31818,50700,143608,571,3626
2,exp1_idx1AGACCATATC_idx2AACGAGGCTA.ACGAGGTTTTTAATTATTG,visceral muscle primordium,51177,19874,69268,44378,67519,212700,189174,131569,⋯,20273,130972,9391,3473,109102,132457,130068,46597,53857,183430
3,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCAGAACCTCAGGTATGGA,brain,137166,35175,4381,123690,99621,35423,123344,108376,⋯,123710,47008,134782,136306,137709,123045,33837,32261,136304,135305
4,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGAGAGATAAGTAGTCAG,epidermis,68440,71253,67238,73868,67236,71880,64019,64022,⋯,64612,63786,70436,65552,2853,65551,67168,39930,28538,74114
5,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGGAGTTTTGAGGTAGAA,sensory nervous system,137728,19780,120926,91533,5730,144152,143823,35446,⋯,51952,4980,134914,19964,132236,52185,34199,214925,117952,68538
6,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGTCTCATTCGATATTGA,sensory nervous system,36849,1387,2385,63108,36476,2887,67810,3884,⋯,5792,75285,2730,19767,34789,35981,1882,64621,49619,781


In [500]:
k2 = k1 + 1

In [501]:
df <- pivot_longer(df,cols=3:k2,names_to = "NN")

Warning message:
“Using an external vector in selections was deprecated in tidyselect 1.1.0.
ℹ Please use `all_of()` or `any_of()` instead.
  # Was:
  data %>% select(k2)

  # Now:
  data %>% select(all_of(k2))

See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.”


In [502]:
df <- mutate(df,nnLabel=labels[value])

In [503]:
head(df, 6)

ID,Label,NN,value,nnLabel
<chr>,<chr>,<chr>,<int>,<chr>
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,V2,1084,plasmatocytes anlage
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,V3,117474,plasmatocytes anlage
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,V4,33297,sensory nervous system primordium
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,V5,2934,plasmatocytes anlage
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,V6,2355,plasmatocytes anlage
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,V7,854,plasmatocytes anlage


In [504]:
df <- select(df,-NN,-value)

In [505]:
head(df)

ID,Label,nnLabel
<chr>,<chr>,<chr>
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,plasmatocytes anlage
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,plasmatocytes anlage
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,sensory nervous system primordium
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,plasmatocytes anlage
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,plasmatocytes anlage
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,sensory nervous system primordium,plasmatocytes anlage


In [506]:
dim(df)

[1] 4768900       3

In [507]:
df  %>% filter(Label==nnLabel)  %>% count(Label)

Label,n
<chr>,<int>
Malpighian tubule,6759
amnioserosa,123495
anlage in statu nascendi,37
brain,241543
brain primordium,55562
cellular blastoderm / pole cell,7
crystal cell,165
ectoderm anlage in statu nascendi,77
ectoderm primordium,10200


In [508]:
res <- group_by(df,ID) %>% summarise(nk=sum (Label==nnLabel)  )

In [509]:
head(res)

ID,nk
<chr>,<int>
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAACTCCAATCGTCCGTCCT,6
exp10_idx1AATACGATAA_idx2AACTACCGAC.AACTAGTTGTAACGGACCAA,5
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGACGATAACTAGGCGC,5
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGACGATCCTAGCGCCT,15
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGACGATGGTCCAGGAG,1
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGATATGTCTGATTAAGA,0


In [510]:
dim(res)

[1] 238445      2

In [511]:
res  %>% count(nk)

nk,n
<int>,<int>
0,25996
1,19488
2,16298
3,14433
4,13027
5,11691
6,10684
7,10048
8,9463


In [512]:
res %>% filter(nk == 1)  %>% head

ID,nk
<chr>,<int>
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGACGATGGTCCAGGAG,1
exp10_idx1AATACGATAA_idx2AACTACCGAC.CAAGCAACTGCTCTCGCCT,1
exp10_idx1AATACGATAA_idx2AACTACCGAC.CAGCAAGTTGCCGCAACGA,1
exp10_idx1AATACGATAA_idx2AACTACCGAC.CTGAGCATTACTTAACCTT,1
exp10_idx1AATACGATAA_idx2AACTACCGAC.CTGAGCATTTATATACTAA,1
exp10_idx1AATACGATAA_idx2AACTACCGAC.GACCAGGATCCTAGCGCCT,1


In [513]:
df  %>% filter(ID == "exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGATATGTCTGATTAAGA")

ID,Label,nnLabel
<chr>,<chr>,<chr>
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGATATGTCTGATTAAGA,muscle,brain
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGATATGTCTGATTAAGA,muscle,brain
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGATATGTCTGATTAAGA,muscle,brain
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGATATGTCTGATTAAGA,muscle,brain
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGATATGTCTGATTAAGA,muscle,brain
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGATATGTCTGATTAAGA,muscle,brain
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGATATGTCTGATTAAGA,muscle,midgut
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGATATGTCTGATTAAGA,muscle,brain
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGATATGTCTGATTAAGA,muscle,brain


In [514]:
res_label <- group_by(df,ID, Label) %>% summarise(nk=sum (Label==nnLabel)  )

`summarise()` has grouped output by 'ID'. You can override using the `.groups` argument.


In [515]:
head(res_label)

ID,Label,nk
<chr>,<chr>,<int>
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAACTCCAATCGTCCGTCCT,ubiquitous,6
exp10_idx1AATACGATAA_idx2AACTACCGAC.AACTAGTTGTAACGGACCAA,midgut,5
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGACGATAACTAGGCGC,muscle,5
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGACGATCCTAGCGCCT,brain,15
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGACGATGGTCCAGGAG,epidermis,1
exp10_idx1AATACGATAA_idx2AACTACCGAC.AAGGATATGTCTGATTAAGA,muscle,0


In [516]:
res_label  %>% filter(nk == 0)  %>% ungroup  %>% count(Label)

Label,n
<chr>,<int>
Malpighian tubule,248
amnioserosa,473
anlage in statu nascendi,20
brain,864
brain primordium,841
cellular blastoderm / pole cell,12
crystal cell,109
ectoderm anlage,5
ectoderm anlage in statu nascendi,18


In [52]:
df_sub300  %>% count(annot)  %>% arrange(annot)

annot,n
<chr>,<int>
Malpighian tubule,1090
amnioserosa,7610
anlage in statu nascendi,40
brain,26617
brain primordium,9816
cellular blastoderm / pole cell,16
crystal cell,177
ectoderm anlage,5
ectoderm anlage in statu nascendi,51


In [517]:
res_label  %>% filter(nk == 1)  %>% ungroup  %>% count(Label)

Label,n
<chr>,<int>
Malpighian tubule,87
amnioserosa,278
anlage in statu nascendi,12
brain,1277
brain primordium,1096
cellular blastoderm / pole cell,2
crystal cell,23
ectoderm anlage in statu nascendi,14
ectoderm primordium,158


In [518]:
singletons = (res %>% filter(nk == 0) )$ID

In [519]:
doubletons = (res %>% filter(nk == 1) )$ID

In [ ]:
# create the dataframe from the cds from 300 PCAs
df_sub300 = make_df(cds_sub)

In [520]:
options(repr.plot.width=14, repr.plot.height=8)

In [49]:
# cell.type = "visceral muscle"
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% singletons), size = ifelse(df_sub300$cell %in% singletons, 0.2, 0.05), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "singletons") + 
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [50]:
#cell.type = "visceral muscle"
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% singletons & df_sub300$annot == "Malpighian tubule"), 
           size = ifelse( df_sub300$cell %in% singletons & df_sub300$annot == "Malpighian tubule", 2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Malpighian tubule singletons") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [51]:
#cell.type = "visceral muscle"
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% singletons & df_sub300$annot == "Malpighian tubule"), 
           size = ifelse( df_sub300$cell %in% singletons & df_sub300$annot == "Malpighian tubule", 2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Malpighian tubule singletons") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [52]:
#cell.type = "visceral muscle"
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = !(df_sub300$cell %in% singletons) & df_sub300$annot == "Malpighian tubule"), 
           size = ifelse( !(df_sub300$cell %in% singletons) & df_sub300$annot == "Malpighian tubule", 2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Malpighian tubule not singletons") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [53]:
#cell.type = "visceral muscle"
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% doubletons & df_sub300$annot == "Malpighian tubule"), 
           size = ifelse( df_sub300$cell %in% doubletons & df_sub300$annot == "Malpighian tubule", 2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Malpighian tubule doubletons") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [54]:
#cell.type = "visceral muscle"
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = !(df_sub300$cell %in% doubletons) & df_sub300$annot == "Malpighian tubule" ), 
           size = ifelse( !(df_sub300$cell %in% doubletons) & df_sub300$annot == "Malpighian tubule", 2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Malpighian tubule not doubletons") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [55]:
#cell.type = "visceral muscle"
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% doubletons), size = ifelse(df_sub300$cell %in% doubletons, 0.1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "doubletons") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [528]:
nineteentons = (res %>% filter(nk == 19) )$ID

In [56]:
#cell.type = "visceral muscle"
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% nineteentons), size = ifelse(df_sub300$cell %in% nineteentons, 0.1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "nineteentons") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [540]:
twentytons = (res %>% filter(nk == 20) )$ID

In [57]:
#cell.type = "visceral muscle"
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% twentytons), 
           size = ifelse(df_sub300$cell %in% twentytons, 0.1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "nineteentons") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [682]:
df_sub300  %>%  head

,orig.ident,nCount_RNA,nFeature_RNA,rt_bc,cell,time,percent.mt,percent.ribo,barn_dro,barn_hu,⋯,NNv2_shift,annot,ident,inferred_time_window,cell.type_time.bin,Size_Factor,cluster,umap_1,umap_2,cell.type.2
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<int>,<int>,⋯,<dbl>,<chr>,<fct>,<chr>,<chr>,<dbl>,<fct>,<dbl>,<dbl>,<chr>
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,exp1,346,247,AAGTTGCCAT,exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,hrs_06_10,1.4450867,4.3352601,371,0,⋯,0.000000,sensory nervous system primordium,21,6-8,sensory nervous system primordium:6-8,0.8108612,89,-2.0067012,1.218446,v
exp1_idx1AGACCATATC_idx2AACGAGGCTA.ACGAGGTTTTTAATTATTG,exp1,279,225,TTAATTATTG,exp1_idx1AGACCATATC_idx2AACGAGGCTA.ACGAGGTTTTTAATTATTG,hrs_08_12,0.7168459,5.0179211,331,4,⋯,0.000000,visceral muscle primordium,11,8-10,visceral muscle primordium:8-10,0.6571200,112,-4.6204820,1.096802,cb
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCAGAACCTCAGGTATGGA,exp1,586,425,CAGGTATGGA,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCAGAACCTCAGGTATGGA,hrs_08_12,1.0238908,6.4846416,630,0,⋯,0.000000,brain,11,10-12,brain:10-12,1.4059888,22,-3.6831634,2.801265,bl
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGAGAGATAAGTAGTCAG,exp1,292,202,AAGTAGTCAG,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGAGAGATAAGTAGTCAG,hrs_10_14,0.3424658,0.6849315,320,3,⋯,0.000000,epidermis,12,12-14,epidermis:12-14,0.6967951,165,-1.3580737,0.358204,h
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGGAGTTTTGAGGTAGAA,exp1,335,243,TGAGGTAGAA,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGGAGTTTTGAGGTAGAA,hrs_14_18,4.4776119,5.6716418,386,0,⋯,-1.480969,sensory nervous system,11,12-14,sensory nervous system:12-14,0.7811049,102,-3.8563070,3.315827,bk
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGTCTCATTCGATATTGA,exp1,354,272,TCGATATTGA,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGTCTCATTCGATATTGA,hrs_14_18,0.5649718,1.6949153,380,0,⋯,0.000000,sensory nervous system,11,14-16,sensory nervous system:14-16,0.8306988,148,-0.6711318,-2.946210,w


In [58]:
# plot of GATAe positive clusters
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell.type.2 %in% c("at", "ax", "b", "ba", "bf", "ac")), 
               size = ifelse(df_sub300$cell.type.2 %in% c("at", "ax", "b", "ba", "bf", "ac"), 0.1, 0.02), 
               stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "nineteentons") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [59]:
# plot of common negative clusters in high angle positives andhigh negative angles
# and 
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell.type.2 %in% c("af", "aw", "as", "bj", "bk", "bl", "bm", "bn", "bo",
                                                   "bq", "br", "bs", "bt", "bv", "bw", "bx",'by' )), 
               size = ifelse(df_sub300$cell.type.2 %in% c("bj", "bk", "bl", "bm", "bn", "bo",
                                                   "bq", "br", "bs", "bt", "bv", "bx", "bw", 'by'), 0.1, 0.02), 
               stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "nineteentons") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


### Subset on singletons and also on doubletons using the nn21 result for Lou


In [72]:
ls()

[1] "cds_sub"                           "cell_type"                        
 [3] "cell.type"                         "cellIDs"                          
 [5] "cells"                             "cellTypes"                        
 [7] "df"                                "df_sub300"                        
 [9] "doubletons"                        "expresses.gene"                   
[11] "get.gene.id"                       "get.gene.id2"                     
[13] "get.norm.expr.matrix"              "get.norm.expr.matrix.3"           
[15] "ids"                               "k"                                
[17] "k1"                                "labels"                           
[19] "make_df"                           "nineteentons"                     
[21] "nnMatches"                         "pca"                              
[23] "peedeef"                           "plot.expr"                        
[25] "plot.expr_clus"                    "plot.expr_clus.br"                
[27] "plot.expr_clus.br2"                "plot.expr3"                       
[29] "rann"                              "res"                              
[31] "res_label"                         "singletons"                       
[33] "two.set.differential.gene.test"    "two.set.differential.gene.test.3" 
[35] "two.set.differential.gene.test.3a" "two.set.differential.gene.test.3b"

In [530]:
cds_sub_no_sing = cds_sub[, !(df_sub300$cell %in% singletons)]

In [531]:
df_sub300  %>% filter(!(cell %in% singletons))  %>% dim

[1] 212449     33

In [532]:
dim(cds_sub_no_sing)

[1]  13938 212449

In [533]:
df_sub_no_singletons = make_df(cds_sub_no_sing)

In [535]:
## continue to remove doubletons
cds_sub_no_doubletons = cds_sub[, !(df_sub_no_singletons$cell %in% doubletons)]

In [536]:
dim(cds_sub_no_doubletons)

[1]  13938 216570

In [538]:
saveRDS(cds_sub_no_doubletons, "data/cds_fly_pca300_no_doubletons.nn21.rds") 

###  Check neuronal subsubcluster -- Lou fixed  names of cell column in this subcluster so they match the old

In [595]:
neuronal_subsubcluster = read_tsv("neuronal_subsubcluster_annotations.modified.txt")

Rows: 6703 Columns: 3
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): cell_name, annotation
dbl (1): seurat_clusters

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [596]:
neuronal_subsubcluster  %>%  head

cell_name,annotation,seurat_clusters
<chr>,<chr>,<dbl>
exp1_idx1GTAGGCTGCA_idx2AACGGACCAA.GTCCTGCATTTAGGATCTA,Chordotonal glia,9
exp1_idx1GTAGGCTGCA_idx2TGAGGTAGAA.CCGTCTCATTGATGGCCTT,Chordotonal glia,9
exp1_idx1TAGGTAACTT_idx2CCGTCGGAGG.TCCGGTCATGGCTATGACT,Chordotonal glia,9
exp1_idx1TAGTAAGCCG_idx2CCAAGCTGAA.GCAACGTTTCAAGTACTGG,Chordotonal glia,9
exp10_idx1TCTTGAGGTT_idx2CGACTACCAA.TGAATCTGATGCTCTCGCCT,Chordotonal glia,9
exp1_idx1CCTCAGCTCC_idx2TTGAGTATCA.TCCGTATGTTGGAACTAGA,Chordotonal sensory,6


In [ ]:
str_split(fruits, " and ", n = 2)

In [559]:
neuronal_subsubcluster$cell_name  %>% str_split("_", n=3)   %>% head

[[1]]
[1] "16"                                                    
[2] "18"                                                    
[3] "exp1_idx1GTAGGCTGCA_idx2AACGGACCAA.GTCCTGCATTTAGGATCTA"

[[2]]
[1] "16"                                                    
[2] "18"                                                    
[3] "exp1_idx1GTAGGCTGCA_idx2TGAGGTAGAA.CCGTCTCATTGATGGCCTT"

[[3]]
[1] "16"                                                    
[2] "18"                                                    
[3] "exp1_idx1TAGGTAACTT_idx2CCGTCGGAGG.TCCGGTCATGGCTATGACT"

[[4]]
[1] "16"                                                    
[2] "18"                                                    
[3] "exp1_idx1TAGTAAGCCG_idx2CCAAGCTGAA.GCAACGTTTCAAGTACTGG"

[[5]]
[1] "16"                                                      
[2] "18"                                                      
[3] "exp10_idx1TCTTGAGGTT_idx2CGACTACCAA.TGAATCTGATGCTCTCGCCT"

[[6]]
[1] "16"                                                    
[2] "18"                                                    
[3] "exp1_idx1CCTCAGCTCC_idx2TTGAGTATCA.TCCGTATGTTGGAACTAGA"

In [594]:
names_expanded = neuronal_subsubcluster  %>%  separate_wider_delim(cell_name, delim = "_", 
                                                 names = c("t1", "t2", "cell", 
                                                           too_many = "merge", too_few = "align_start")) 

ERROR: [1m[33mError[39m in `separate_wider_delim()`:[22m
[1m[22m[33m![39m Expected 5 pieces in each element of `cell_name`.
[33m![39m 4 values were too short.
[36mℹ[39m Use `too_few = "debug"` to diagnose the problem.
[36mℹ[39m Use `too_few = "align_start"/"align_end"` to silence this message.


In [590]:
neuronal_subsubcluster2 = names_expanded  %>% mutate(cell_name = paste(cell, merge, align_start, sep = "_"), .before=-0)  %>%  
select(cell_name, annotation, seurat_clusters)

In [598]:
Cholinergic_1 = (neuronal_subsubcluster  %>% filter(annotation == "Cholinergic 1"))$cell_name

In [599]:
length(Cholinergic_1)

[1] 1562

In [60]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == "Cholinergic 1"))$cell_name), size = ifelse(df_sub300$cell %in% Cholinergic_1, 0.1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Cholinergic_1") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [61]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == "Cholinergic 2"))$cell_name), 
           size = ifelse(df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == "Cholinergic 2"))$cell_name, 0.1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Cholinergic_2") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [62]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == "Chordotonal glia"))$cell_name), 
           size = ifelse(df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == "Chordotonal glia"))$cell_name, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Chordotonal glia") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [63]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == "Chordotonal sensory"))$cell_name),
           size = ifelse(df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == "Chordotonal sensory"))$cell_name, 0.1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Chordotonal sensory") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [64]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == 
                                                       "External sensory"))$cell_name), 
           size = ifelse(df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == 
                                                       "External sensory"))$cell_name, 0.1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "External sensory") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [65]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == 
                                                       "Glutamatergic"))$cell_name), 
           size = ifelse(df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == 
                                                       "Glutamatergic"))$cell_name, 0.1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Glutamatergic") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [66]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == 
                                                       "Monoaminergic"))$cell_name), 
           size = ifelse(df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == 
                                                       "Monoaminergic"))$cell_name, 0.1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Monoaminergic") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [67]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == 
                                                       "Multidendritic sensory"))$cell_name), 
           size = ifelse(df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == 
                                                       "Multidendritic sensory"))$cell_name, 0.1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Multidendritic sensory") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [597]:
neuronal_subsubcluster  %>% count(annotation)

annotation,n
<chr>,<int>
Cholinergic 1,1562
Cholinergic 2,997
Chordotonal glia,70
Chordotonal sensory,353
External sensory,447
Glutamatergic,564
Monoaminergic,247
Multidendritic sensory,656
Peptidergic,49


In [68]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == "Peptidergic"))$cell_name), size = 
           ifelse(df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == "Peptidergic"))$cell_name, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Peptidergic") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [69]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == "Undiff. 1"))$cell_name), 
           size = ifelse(df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == "Undiff. 1"))$cell_name, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Undiff. 1") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [70]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == "Undiff. 2"))$cell_name), 
           size = ifelse(df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == "Undiff. 2"))$cell_name, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Undiff. 2") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


### Check the larger subset

In [629]:
neuronal_subcluster = read_tsv("neuronal_subcluster_annotations.modified.txt")

Rows: 59012 Columns: 3
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): cell_name, annotation
dbl (1): seurat_clusters

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [71]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% (neuronal_subcluster   
                                            %>% filter(annotation == "Head epid. /muscle"))$cell_name), size = 
           ifelse(df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == "Head epid. /muscle"))$cell_name, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Head epid. /muscle") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [72]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% (neuronal_subcluster   
                                            %>% filter(annotation == "Immature neurons"))$cell_name), size = 
           ifelse(df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == "Immature neurons"))$cell_name, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Immature neurons") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [73]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% (neuronal_subcluster   
                                            %>% filter(annotation == "Mature neurons"))$cell_name), size = 
           ifelse(df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == "Mature neurons"))$cell_name, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Mature neurons") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [74]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% (neuronal_subcluster   
                                            %>% filter(annotation == "Immature neurons"))$cell_name), size = 
           ifelse(df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == "Immature neurons"))$cell_name, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Immature neurons") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [75]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% (neuronal_subcluster   
                                            %>% filter(annotation == "Neural progenitors"))$cell_name), size = 
           ifelse(df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == "Neural progenitors"))$cell_name, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Neural progenitors") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [630]:
neuronal_subcluster  %>% count(annotation)

annotation,n
<chr>,<int>
Germline,43
Head epid. /muscle,4486
Immature neurons,15602
Mature neurons,6703
Neural progenitors,24825
Oenocytes,140
Sensory progenitors,5149
Ventral midline,2064


In [76]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% (neuronal_subcluster   
                                            %>% filter(annotation == "Oenocytes"))$cell_name), size = 
           ifelse(df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == "Oenocytes"))$cell_name, 2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Oenocytes") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [77]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% (neuronal_subcluster   
                                            %>% filter(annotation == "Sensory progenitors"))$cell_name), size = 
           ifelse(df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == "Sensory progenitors"))$cell_name, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Sensory progenitors") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [78]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cell %in% (neuronal_subcluster   
                                            %>% filter(annotation == "Ventral midline"))$cell_name), size = 
           ifelse(df_sub300$cell %in% (neuronal_subsubcluster  
                                            %>% filter(annotation == "Ventral midline"))$cell_name, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = "Ventral midline") +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [695]:
ls(pattern = "cds")

[1] "cds_sub"               "cds_sub_no_doubletons" "cds_sub_no_multitons" 
[4] "cds_sub_no_sing"

In [696]:
as.data.frame(rowData(cds_sub))  %>%  head

,gene_short_name
,<chr>
gfzf,gfzf
Osi24,Osi24
msps,msps
CG6013,CG6013
Dsk,Dsk
CG8043,CG8043


In [697]:
tf = "GATAe"
ranklb = 0
rankub = 1
## set the cluster size range
lb = 2
ub = 277
target_set = unique((fly_peaks_TF  %>% 
                            filter(TF == tf & 
                                   rank >= ranklb & rank <=rankub & 
                                   nPeaks > lb & nPeaks <= ub
                                  & Gene != tf 
                                   & grepl("emb", exp)
                                  ))$Gene)

In [698]:
target_set_GATAe = target_set

In [699]:
tf = "crol"
ranklb = 0
rankub = 1
## set the cluster size range
lb = 2
ub = 277
target_set = unique((fly_peaks_TF  %>% 
                            filter(TF == tf & 
                                   rank >= ranklb & rank <=rankub & 
                                   nPeaks > lb & nPeaks <= ub
                                  & Gene != tf 
                                   & grepl("emb", exp)
                                  ))$Gene)

In [700]:
target_set_crol = target_set

In [701]:
tf = "Camta"
ranklb = 0
rankub = 1
## set the cluster size range
lb = 2
ub = 277
target_set = unique((fly_peaks_TF  %>% 
                            filter(TF == tf & 
                                   rank >= ranklb & rank <=rankub & 
                                   nPeaks > lb & nPeaks <= ub
                                  & Gene != tf 
                                   & grepl("emb", exp)
                                  ))$Gene)

In [702]:
target_set_Camta = target_set

In [711]:
tf = "sens"
ranklb = 0
rankub = 1
## set the cluster size range
lb = 2
ub = 277
target_set = unique((fly_peaks_TF  %>% 
                            filter(TF == tf & 
                                   rank >= ranklb & rank <=rankub & 
                                   nPeaks > lb & nPeaks <= ub
                                  & Gene != tf 
                                   & grepl("emb", exp)
                                  ))$Gene)

In [712]:
target_set_sens = target_set

In [713]:
length(target_set_GATAe)
length(target_set_Camta)
length(target_set_crol)
length(target_set_sens)

[1] 1002

[1] 493

[1] 2852

[1] 407

In [709]:
options(repr.matrix.max.cols = 100)

In [710]:
fly_peaks_TF  %>% filter(nPeaks >= 277 & TF == "GATAe")  %>% head

,peakID,absDist,chromo.x,start.x,end.x,exp,score,strand.x,apexS.x,apexE.x,color.x,cluster,target,targetDist,chromo.y,start.y,end.y,nPeaks,strand.y,apexS.y,apexE.y,color.y,TranscriptCoord,Transcript,TranscriptLoc,Gene,GeneLoc,GeneExons,TranscriptNo,UniqueGeneTSSs,GeneTranscripts,TF,rank
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
1,GATAe_GATAe-GFP_embryonic_1chr2L10003641:10004001,11,chr2L,10003641,10004001,GATAe_GATAe-GFP_embryonic_1,856,.,10003819,10003823,"0,255,0",39_narrowPeak1435,Ndf-RA::5P,-11,chr2L,10003248,10004300,487,.,10003611,10003615,"0,255,0",2L:10003832:10008889:+,Ndf-RA,5P,Ndf,5P,9,1,1,3,GATAe,0.2848704
2,GATAe_GATAe-GFP_embryonic_1chr2L10010030:10010390,144,chr2L,10010030,10010390,GATAe_GATAe-GFP_embryonic_1,1000,.,10010208,10010212,"0,0,255",42_narrowPeak1395,Trp1-RB::I1,-144,chr2L,10009842,10010733,552,.,10010160,10010164,"0,0,255",2L:10010354:10013170:+,Trp1-RB,5P,Trp1,I1,5,2,2,4,GATAe,0.8562521
3,GATAe_GATAe-GFP_embryonic_1chr2L10200888:10201248,32,chr2L,10200888,10201248,GATAe_GATAe-GFP_embryonic_1,1000,.,10201066,10201070,"0,255,0",7_narrowPeak2571,RpL7-RA::E1,-32,chr2L,10200412,10201648,657,.,10200836,10200840,"0,255,0",2L:10201100:10202145:+,RpL7-RA,5P,RpL7,E1,3,2,2,2,GATAe,0.8099770
4,GATAe_GATAe-GFP_embryonic_1chr2L10334905:10335056,1109,chr2L,10334905,10335056,GATAe_GATAe-GFP_embryonic_1,1000,.,10335021,10335025,"0,255,0",13_narrowPeak2152,GATAd-RC::I1,1109,chr2L,10334370,10335803,391,.,10334963,10334967,"0,255,0",2L:10333914:10339185:+,GATAd-RC,I1,GATAd,I1,8,2,2,3,GATAe,0.9271415
5,GATAe_GATAe-GFP_embryonic_1chr2L10381942:10382274,144,chr2L,10381942,10382274,GATAe_GATAe-GFP_embryonic_1,1000,.,10382117,10382121,"0,0,255",2_narrowPeak4097,CG5096-RA::5P,-144,chr2L,10381670,10383136,429,.,10382092,10382096,"0,0,255",2L:10382263:10384723:+,CG5096-RA,5P,CG5096,5P,6,1,1,1,GATAe,0.9977027
6,GATAe_GATAe-GFP_embryonic_1chr2L10474484:10474622,165,chr2L,10474484,10474622,GATAe_GATAe-GFP_embryonic_1,1000,.,10474512,10474516,"255,0,0",2_narrowPeak4138,RluA-1-RA::5P,-165,chr2L,10473891,10475100,626,.,10474516,10474520,"255,0,0",2L:10463340:10474349:-,RluA-1-RA,5P,RluA-1,5P,11,1,1,4,GATAe,0.9606170


# Reannotate fly cells using same bins but reame with Sue's names

## Also add in subdivisions from neural annotations from Shendure paper.

In [717]:
load("data/markers for fly datasets.RData")

In [718]:
ls(pattern = "cluster.markers")

[1] "cluster.markers"                         
[2] "cluster.markers.sub300"                  
[3] "cluster.markers.sub300_no_doubletons"    
[4] "cluster.markers.sub300_no_doubletons_all"
[5] "cluster.markers.sub300_no_singletons"

In [110]:
cluster.markers.sub300_no_singletons %>% filter(!(cluster %in% c(160, 163, 103, 172, 168, 
                                                              170, 14, 8,
                                                              141, 151, 146, 165,
                                                              85, 41, 99, 110, 31, 60, 111, 169,
                                                              51, 95, 73, 1, 174, 2,
                                                              171, 20, 98, 45, 96, 92, 130, 88,
                                                              13, 55, 167, 104, 123,  25, 129,
                                                              157, 122, 58, 28, 89, 148, 159,
                                                              142, 154, 124, 149, 126, 113,
                                                              81, 139, 34, 147, 166, 107,
                                                              155, 173, 40, 80, 161, 120, 162, 
                                                               134, 156, 115, 56, 30, 128, 137, 158,
                                                              121, 109, 94, 145, 68, 86, 114, 17, 63,
                                                              47, 164, 166, 5, 66, 39, 9, 106,
                                                              10, 152,  36, 127, 12, 75, 46, 133, 136, 119, 48,
                                                              49, 64, 72, 140) ))  %>%  
head(10)

,cluster,gene,cluster.n.umi,other.n.umi,cluster.tpm,other.tpm,log2.ratio,heuristic.score
,<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG918453,54,CG9184,7147,35616,14831.3446,1522.1598978,3.283510,45.49782
CG1059153,54,CG10591,5091,23244,10405.6717,991.7346141,3.389818,45.23787
CG5715149,150,CG5715,32,17,138.1496,0.9084849,6.177659,43.98798
sr143,144,sr,591,3816,2365.5387,165.4169404,3.829295,42.92089
bbg14,15,bbg,4838,15567,7143.5268,700.3065817,3.348519,42.86983
scro56,57,scro,598,1464,1232.0042,67.3759216,4.171375,42.83152
lmd99,100,lmd,691,3111,1997.0907,132.5558449,3.902385,42.78733
Desi53,54,Desi,89,86,182.2837,2.5356873,5.688051,42.76239
ImpE253,54,ImpE2,3061,14790,6225.0365,603.3959499,3.364516,42.40670


In [721]:
cluster.markers.sub300_no_singletons  %>% filter(!(cluster %in% c(160, 163, 103, 172, 168, 
                                                              170, 14, 8,
                                                              141, 151, 146, 165,
                                                              85, 41, 99, 110, 31, 60, 111, 169,
                                                              51, 95, 73, 1, 174, 2,
                                                              171, 20, 98, 45, 96, 92, 130, 88,
                                                              13, 55, 167, 104, 123,  25, 129,
                                                              157, 122, 58, 28, 89, 148, 159,
                                                              142, 154, 124, 149, 126, 113,
                                                              81, 139, 34, 147, 166, 107,
                                                              155, 173, 40, 80, 161, 120, 162, 
                                                               134, 156, 115, 56, 30, 128, 137, 158,
                                                              121, 109, 94, 145, 68, 86, 114, 17, 63,
                                                              47, 164, 166, 5, 66, 39, 9, 106,
                                                              10, 152,  36, 127, 12, 75, 46, 133, 136, 119, 48,
                                                              49, 64, 72, 140) ))  %>%  
head(10)

,cluster,gene,cluster.n.umi,other.n.umi,cluster.tpm,other.tpm,log2.ratio,heuristic.score
,<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG918453,54,CG9184,7147,35616,14831.3446,1522.1598978,3.283510,45.49782
CG1059153,54,CG10591,5091,23244,10405.6717,991.7346141,3.389818,45.23787
CG5715149,150,CG5715,32,17,138.1496,0.9084849,6.177659,43.98798
sr143,144,sr,591,3816,2365.5387,165.4169404,3.829295,42.92089
bbg14,15,bbg,4838,15567,7143.5268,700.3065817,3.348519,42.86983
scro56,57,scro,598,1464,1232.0042,67.3759216,4.171375,42.83152
lmd99,100,lmd,691,3111,1997.0907,132.5558449,3.902385,42.78733
Desi53,54,Desi,89,86,182.2837,2.5356873,5.688051,42.76239
ImpE253,54,ImpE2,3061,14790,6225.0365,603.3959499,3.364516,42.40670


In [79]:
plot.expr_clus.br2(cds_sub_no_multitons, c("sr"), 
                   size_pos = 0.1, size_neg = 0.2, stroke = 0.02, limit = 1) 

ERROR: Error in plot.expr_clus.br2(cds_sub_no_multitons, c("sr"), size_pos = 0.1, : could not find function "plot.expr_clus.br2"


In [111]:
a = c(160)
b = c(163, 103)
c = c(172)
d = c(168)
e = c(170, 14, 8, 143)  # 8 could be broken down further
f = c(141, 151)
g = c(146)
h = c(165)
i = c(85, 41, 99, 110, 31, 60, 111)
j = c(169)
k = c(174)
l = c(51)
m = c(95, 73)
n = c(1)
o = c(2) # CG43896 is only at one tip so could be broken down further
p = c(171)
q = c(20, 98, 45, 96, 92, 130, 88) # 20 should be split with 13, 55
r = c(13, 55)
s = c(167, 104, 123,  25) # has considerable scatter
t = c(129) # possibly two branches
u = c(157)
v = c(122, 58, 28, 89)
w = c(148)
x = c(159)
y = c(142, 154) # has spur on 154 that could be removed
z = c(124)
aa = c(149, 126) # 121, 109 are spurs, tributaries
ab = c(113) # could be extension of y at least in part, but looks like much is separate
ac = c(81, 139, 34)
ad = c(147)
ae = c(166, 107)  # markers shar, but distant on map; and only a small cluster of 107, which could be broken 3 ways
af = c(173)
ag = c(155)
ah = c(40, 134 ) # 40 has spur that could be broken off
ai = c(80)
aj = c(161, 120, 162)
ak = c(156)
al = c(115, 56) # coalescent for 124, 128, 137
am = c(30)
an = c(128, 137)  # see what the markers are here
ao = c(158) # closely related to 170, 14
ap = c(121, 109) # Tsp74F lights up main trajectory of these two, but both could be broken further
aq = c(94)
ar = c(145, 68, 86)
as = c(114)  # CG13545 is only in a single part of this larger cluster, part of brain and so might be neuron
at = c(17)  # 49 is base and with part of 107 feeds into 12
au = c(63) # CG7300 is only in left spur of 63, so could be broken down further; other spur feeds 128
av = c(47, 164)
aw = c(166) # markers also light up small part of 107, but not the main cluster.  Look for better markers of bulk of 107
ax = c(5, 66) # Ptx1 marker also lights up 159 which is a spur of 5
ay = c(39, 9, 106)
az = c(10, 152, 36, 127) # basal clusters
ba = c(12) # Lim3 also in adjacent 2, 49 is base
bb = c(46) # CG44422 not specific but "CG44422", "disco-r" pair very specific
bc = c(75) # separate cluster of 75 so could be broken down further tey marker also shared with 156
bd = c(133) # Desi in small cluster of 133, part of brain, so could neuronal; shared with 121 or 54
be = c(136, 119, 48, 100) # basal cluster
bf = c(49) # with part of 107 base of 17, 12
bg = c(64) # Ccp84Ac covers it but shared with 113
bh = c(72, 57) # may be base of 124, 128, both could be broken
bi = c(140) # very poor cluster, may be with 123
## below done only on umap picture
bj = c(23, 116, 105, 153)
bk = c(52, 26, 102, 27)
bl = c(87, 70, 35, 37, 22, 59)
bm = c(132, 125, 108)
bn = c(44)
bo = c(97, 118)
bp = c(91, 144, 54, 15, 84)
bq = c(6, 117, 38, 43, 4)
br = c(138, 67, 65)
bs = c(32, 82, 131)
bt = c(11, 76, 3, 62, 50)
bu = c(71, 19)
bv = c(33, 61)
bw = c(77, 53)
bx = c(29, 135)
by = c(21)
bz = c(18) # scattered cells
ca = c(59, 74, 101)
cb = c(112, 79, 7)
cc = c(16, 93, 42, 78, 24)
cd = c(69, 83)
ce = c(90)
cf = c(150) # very scattered cells near 12

In [80]:
clus = c(150)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [81]:
clus = c(a, b, c, d, e, f, g, h, i , j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z,
        aa,ab, ac, ad, ae, af, ag, ah, ai, aj, ak, al, am, an, ao, ap, aq, ar, as, at, au, av, aw,
        ax, ay, az, ba, bb, bc, bd, be, bf, bh, bi, bj, bk, bl, bm, bn, bo, bp, bq, br, bs, bt, bu, bv, 
         bw, bx, by, bz, ca, cb, cc, cd, ce)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'a' not found


In [82]:
df_sub300$cell.type.3 = rep(NA, ncol(cds_sub))


ERROR: Error in ncol(cds_sub): object 'cds_sub' not found


In [724]:
df_sub300  %>%  head

,orig.ident,nCount_RNA,nFeature_RNA,rt_bc,cell,time,percent.mt,percent.ribo,barn_dro,barn_hu,barn_mus,RNA_snn_res.0.8,doublet_score,predicted_doublet,doublet_cluster,seurat_clusters,doublet_subcluster,RNA_snn_res.0.08,lasso_age,NNv1_age,NNv2_age,lasso_shift,NNv1_shift,NNv2_shift,annot,ident,inferred_time_window,cell.type_time.bin,Size_Factor,cluster,umap_1,umap_2,cell.type.2,cell.type.3
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<int>,<int>,<int>,<chr>,<dbl>,<chr>,<lgl>,<fct>,<lgl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<fct>,<chr>,<chr>,<dbl>,<fct>,<dbl>,<dbl>,<chr>,<lgl>
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,exp1,346,247,AAGTTGCCAT,exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,hrs_06_10,1.4450867,4.3352601,371,0,0,16,0.05276064,Singlet,FALSE,21,FALSE,21,8.402794,7.885203,8.119637,0.0000000,0.0000000,0.000000,sensory nervous system primordium,21,6-8,sensory nervous system primordium:6-8,0.8108612,89,-2.0067012,1.218446,v,NA
exp1_idx1AGACCATATC_idx2AACGAGGCTA.ACGAGGTTTTTAATTATTG,exp1,279,225,TTAATTATTG,exp1_idx1AGACCATATC_idx2AACGAGGCTA.ACGAGGTTTTTAATTATTG,hrs_08_12,0.7168459,5.0179211,331,4,5,7,0.07219402,Singlet,FALSE,11,FALSE,11,11.597093,9.940381,9.959537,0.0000000,0.0000000,0.000000,visceral muscle primordium,11,8-10,visceral muscle primordium:8-10,0.6571200,112,-4.6204820,1.096802,cb,NA
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCAGAACCTCAGGTATGGA,exp1,586,425,CAGGTATGGA,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCAGAACCTCAGGTATGGA,hrs_08_12,1.0238908,6.4846416,630,0,0,4,0.09008910,Singlet,FALSE,11,FALSE,11,11.266422,10.062633,11.156040,0.0000000,0.0000000,0.000000,brain,11,10-12,brain:10-12,1.4059888,22,-3.6831634,2.801265,bl,NA
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGAGAGATAAGTAGTCAG,exp1,292,202,AAGTAGTCAG,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGAGAGATAAGTAGTCAG,hrs_10_14,0.3424658,0.6849315,320,3,0,23,0.06585398,Singlet,FALSE,12,FALSE,12,12.544636,12.211529,13.830982,0.0000000,0.0000000,0.000000,epidermis,12,12-14,epidermis:12-14,0.6967951,165,-1.3580737,0.358204,h,NA
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGGAGTTTTGAGGTAGAA,exp1,335,243,TGAGGTAGAA,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGGAGTTTTGAGGTAGAA,hrs_14_18,4.4776119,5.6716418,386,0,0,8,0.08478501,Singlet,FALSE,11,FALSE,11,12.542527,13.739003,12.519031,-1.4574728,-0.2609968,-1.480969,sensory nervous system,11,12-14,sensory nervous system:12-14,0.7811049,102,-3.8563070,3.315827,bk,NA
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGTCTCATTCGATATTGA,exp1,354,272,TCGATATTGA,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGTCTCATTCGATATTGA,hrs_14_18,0.5649718,1.6949153,380,0,1,8,0.08257165,Singlet,FALSE,11,FALSE,11,13.458162,15.126904,14.558554,-0.5418379,0.0000000,0.000000,sensory nervous system,11,14-16,sensory nervous system:14-16,0.8306988,148,-0.6711318,-2.946210,w,NA


## Start the reannotation

In [725]:
set.cell.type = function(data_df, cell.type, x) {
       message("Assigning cells for cell type ", cell.type)
       data_df$cell.type.3 = ifelse(x, cell.type, data_df$cell.type.3)
       return(data_df)
    }


In [726]:
df_sub300_copy = df_sub300

In [1786]:
# not used, from prior
df_sub300 = df_sub300_copy

In [787]:
df_sub300 = set.cell.type(df_sub300, "proventriculus", df_sub300$cluster %in% a)  

Assigning cells for cell type proventriculus



In [1781]:
df_sub300$cell.type.3 <- 
gsub("proventriculus", NA, df_sub300$cell.type.3)

In [1785]:
df_sub300 = set.cell.type(df_sub300, "unknown_4", df_sub300$cluster %in% a &
                          df_sub300$umap_2 < 1.8*df_sub300$umap_1 -4.9
                         )

Assigning cells for cell type unknown_4



In [1788]:
df_sub300 = set.cell.type(df_sub300, "Proventriculus_older", df_sub300$cluster %in% a &
                         df_sub300$umap_2 < 1.8*df_sub300$umap_1 -4.4 &
              df_sub300$umap_2 >= 1.8*df_sub300$umap_1 -4.9)

Assigning cells for cell type Proventriculus_older



In [1789]:
df_sub300 = set.cell.type(df_sub300, "unknown_5", df_sub300$cluster %in% a &
                         df_sub300$umap_2 >= 1.8*df_sub300$umap_1 -4.4 &
              df_sub300$umap_2 < 1.8*df_sub300$umap_1 -3.95)

Assigning cells for cell type unknown_5



In [1790]:
df_sub300 = set.cell.type(df_sub300, "unknown_6", df_sub300$cluster %in% a &
                         df_sub300$umap_2 >= 1.8*df_sub300$umap_1 -3.95)

Assigning cells for cell type unknown_6



In [83]:
clus = c(a)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'a' not found


In [ ]:
clus = c(a)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

In [84]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "a" & cell.type.2.tpm > other.tpm)  %>% head(20)

ERROR: Error in cell.type.2.markers_no_doubletons %>% filter(cell.type.2 == "a" & : could not find function "%>%"


In [85]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Muc26B"), 
                   size_pos = 4, size_neg = 2, stroke = 0.8, limit = 1) +
xlim(2, 6) + ylim(0,4)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG7017"), 
                   size_pos = 4, size_neg = 2, stroke = 0.8, limit = 1) +
xlim(2, 6) + ylim(0,4)
# flybase says expressed in proventriculus
plot.expr_clus.br2(cds_sub_no_doubletons, c("Muc96D"), 
                   size_pos = 4, size_neg = 2, stroke = 0.9, limit = 1) +
xlim(2, 6) + ylim(0,4)
# flybase says expressed in proventriculus

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Muc26B"), size_pos = 4, : could not find function "plot.expr_clus.br2"


In [1723]:
two.set.differential.gene.test.3a(cds_sub, 
                                  df_sub300$cluster == a, 
                                 !(df_sub300$cluster == a),
                                 sample = 20000)  %>% head

# of cells sampled in set 1: 528

# of cells sampled in set 2: 20000



,gene,set.1.n.umi,set.2.n.umi,set.1.tpm,set.2.tpm,higher.expr,log2.ratio,heuristic.score
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
Muc26B,Muc26B,3993,41,10368.322,3.0813262,Set 1,11.310857,150.88721
CG7017,CG7017,3759,102,11685.814,9.1726831,Set 1,10.165842,137.36690
Muc96D,Muc96D,2590,53,6427.809,4.5846032,Set 1,10.168657,128.63692
CG17826,CG17826,1107,9,2902.487,0.8681700,Set 1,10.601448,121.95451
CG11300,CG11300,523,5,1123.508,0.4703003,Set 1,9.577683,97.07057
EAChm,EAChm,327,53,1601.716,4.7526624,Set 1,8.121173,86.46047


In [86]:
clus = c(a)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 < 1.8*umap_1 -4.9), 
           size =  ifelse(df_sub300$cluster %in% clus, 4, 0.2), stroke = 0.5) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 1.8, intercept = -4.4) +
geom_abline(slope = 1.8, intercept = -4.9) +
geom_abline(slope = 1.8, intercept = -5.8) +
xlim(2, 6) + ylim(0,4)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'a' not found


In [1752]:
two.set.differential.gene.test.3a(cds_sub, 
                                  df_sub300$cluster == a &
                                  df_sub300$umap_2 < 1.8*df_sub300$umap_1 -4.9, 
                                 !(df_sub300$cluster == a &
                                  df_sub300$umap_2 < 1.8*df_sub300$umap_1 -4.9) ,
                                 sample = 20000)  %>% head(20)

# of cells sampled in set 1: 156

# of cells sampled in set 2: 20000



,gene,set.1.n.umi,set.2.n.umi,set.1.tpm,set.2.tpm,higher.expr,log2.ratio,heuristic.score
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
EAChm,EAChm,326,57,5419.5933,4.63990761,Set 1,9.908297,122.90485
CG32453,CG32453,107,7,1866.6010,0.49633844,Set 1,10.284761,111.76420
CG18649,CG18649,97,11,1688.4494,0.86471948,Set 1,9.822525,105.32042
CG5402,CG5402,60,3,1067.1943,0.32416603,Set 1,9.654523,97.13376
Eip74EF,Eip74EF,488,547,8605.4569,55.99706117,Set 1,7.238220,94.61225
CG34290,CG34290,52,3,817.0746,0.23087190,Set 1,9.374643,90.70988
CG14453,CG14453,57,9,960.0928,0.87437564,Set 1,9.000620,89.18293
CG14454,CG14454,47,4,772.9593,0.48061288,Set 1,9.028054,86.63424
CG13738,CG13738,32,2,516.4712,0.09613526,Set 1,8.880118,80.05724


In [87]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("EAChm"), 
                   size_pos = 4, size_neg = 2, stroke = 0.8, limit = 1) +
xlim(2, 6) + ylim(0,4)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG32453"), 
                   size_pos = 4, size_neg = 2, stroke = 0.8, limit = 1) +
xlim(2, 6) + ylim(0,4)

plot.expr_clus.br2(cds_sub_no_doubletons, c("Eip74EF"), 
                   size_pos = 4, size_neg = 2, stroke = 0.9, limit = 1) +
xlim(2, 6) + ylim(0,4)


ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("EAChm"), size_pos = 4, : could not find function "plot.expr_clus.br2"


In [88]:
clus = c(a, as)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
              df_sub300$umap_2 < 1.8*umap_1 -4.4 &
              df_sub300$umap_2 >= 1.8*umap_1 -4.9), 
           size =  ifelse(df_sub300$cluster %in% clus, 4, 0.2), stroke = 0.5) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 1.8, intercept = -4.4) +
geom_abline(slope = 1.8, intercept = -4.9) +
geom_abline(slope = 1.8, intercept = -5.8) +
xlim(2, 6) + ylim(0,4)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'a' not found


In [1755]:
two.set.differential.gene.test.3a(cds_sub, 
                                  df_sub300$cluster == a &
                                  df_sub300$umap_2 < 1.8*df_sub300$umap_1 -4.4 &
                                  df_sub300$umap_2 >= 1.8*df_sub300$umap_1 -4.9, 
                                 !(df_sub300$cluster == a &
                                   df_sub300$umap_2 < 1.8*df_sub300$umap_1 -4.4 &
                                  df_sub300$umap_2 >= 1.8*df_sub300$umap_1 -4.9) ,
                                 sample = 20000)  %>% head(20)

# of cells sampled in set 1: 113

# of cells sampled in set 2: 20000



,gene,set.1.n.umi,set.2.n.umi,set.1.tpm,set.2.tpm,higher.expr,log2.ratio,heuristic.score
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
Muc26B,Muc26B,3991,45,48387.9156,4.57977773,Set 1,13.082152,203.58953
CG7017,CG7017,3750,86,54290.7803,7.18114684,Set 1,12.696117,199.69018
Muc96D,Muc96D,2590,54,30034.3652,5.63011511,Set 1,12.145293,180.65363
CG17826,CG17826,1107,11,13562.0611,0.64822638,Set 1,13.006374,178.54364
CG10725,CG10725,144,3,2748.7208,0.25184319,Set 1,11.100491,126.82388
CG12934,CG12934,209,21,4031.9803,1.77571514,Set 1,10.504413,125.81798
CG34282,CG34282,168,7,2052.5797,0.11675219,Set 1,10.843913,119.32561
CG11672,CG11672,184,23,3202.8374,1.90727691,Set 1,10.105466,117.68407
CG7715,CG7715,1006,1737,13228.6826,60.84266392,Set 1,7.740851,105.98379


In [89]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Muc26B"), 
                   size_pos = 4, size_neg = 2, stroke = 0.8, limit = 1) 
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG7017"), 
                   size_pos = 4, size_neg = 2, stroke = 0.8, limit = 1) 

plot.expr_clus.br2(cds_sub_no_doubletons, c(""), 
                   size_pos = 4, size_neg = 2, stroke = 0.9, limit = 1) 

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Muc26B"), size_pos = 4, : could not find function "plot.expr_clus.br2"


In [90]:
clus = c(a)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
              df_sub300$umap_2 >= 1.8*umap_1 -4.4 &
              df_sub300$umap_2 < 1.8*umap_1 -3.95
              ), 
           size =  ifelse(df_sub300$cluster %in% clus, 4, 0.2), stroke = 0.5) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 1.8, intercept = -4.4) +
geom_abline(slope = 1.8, intercept = -4.9) +
geom_abline(slope = 1.8, intercept = -3.95) +
xlim(2, 6) + ylim(0,4)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'a' not found


In [1773]:
two.set.differential.gene.test.3a(cds_sub, 
                                  df_sub300$cluster == a &
                                  df_sub300$umap_2 >= 1.8*df_sub300$umap_1 -4.4 &
              df_sub300$umap_2 < 1.8*df_sub300$umap_1 -3.99, 
                                 !(df_sub300$cluster == a &
                                   df_sub300$umap_2 >= 1.8*df_sub300$umap_1 -4.4 &
              df_sub300$umap_2 < 1.8*df_sub300$umap_1 -3.99)  ,
                                 sample = 20000)  %>% head(20)

# of cells sampled in set 1: 127

# of cells sampled in set 2: 20000



,gene,set.1.n.umi,set.2.n.umi,set.1.tpm,set.2.tpm,higher.expr,log2.ratio,heuristic.score
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
Nepl15,Nepl15,377,145,6282.1009,14.8561718,Set 1,8.630059,108.88770
Mdr65,Mdr65,73,39,1456.0449,3.8260605,Set 1,8.236993,86.56116
CG32032,CG32032,139,143,2615.0025,15.3889816,Set 1,7.317942,83.08168
CG5687,CG5687,41,64,814.1171,5.4370560,Set 1,6.982691,67.52866
CG43332,CG43332,0,2401,0.0000,281.3025212,Set 2,8.135979,66.23580
nAChRalpha6,nAChRalpha6,0,2343,0.0000,279.8162283,Set 2,8.128336,66.11168
Spn43Ad,Spn43Ad,21,15,382.7064,1.1310144,Set 1,7.488554,64.28069
Pxn,Pxn,0,2064,0.0000,250.5413048,Set 2,7.968905,63.54924
CG15203,CG15203,45,52,601.0952,4.4261253,Set 1,6.791527,62.71193


In [91]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Mdr65"), 
                   size_pos = 4, size_neg = 2, stroke = 0.8, limit = 1) +
xlim(2, 6) + ylim(0,4)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG32198"), 
                   size_pos = 4, size_neg = 2, stroke = 0.8, limit = 1) +
xlim(2, 6) + ylim(0,4)

plot.expr_clus.br2(cds_sub_no_doubletons, c("CG32032"), 
                   size_pos = 4, size_neg = 2, stroke = 0.9, limit = 1) +
xlim(2, 6) + ylim(0,4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Mdr65"), size_pos = 4, : could not find function "plot.expr_clus.br2"


In [92]:
clus = c(a)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
              df_sub300$umap_2 >= 1.8*umap_1 -3.95
              ), 
           size =  ifelse(df_sub300$cluster %in% clus, 4, 0.2), stroke = 0.5) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 1.8, intercept = -4.4) +
geom_abline(slope = 1.8, intercept = -4.9) +
geom_abline(slope = 1.8, intercept = -3.95) +
xlim(2, 6) + ylim(0,4)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'a' not found


In [1775]:
two.set.differential.gene.test.3a(cds_sub, 
                                  df_sub300$cluster == a &
                                  df_sub300$umap_2 >= 1.8*df_sub300$umap_1 -3.95 , 
                                 !(df_sub300$cluster == a &
                                   df_sub300$umap_2 >= 1.8*df_sub300$umap_1 -3.95 )  ,
                                 sample = 20000)  %>% head(20)

# of cells sampled in set 1: 107

# of cells sampled in set 2: 20000



,gene,set.1.n.umi,set.2.n.umi,set.1.tpm,set.2.tpm,higher.expr,log2.ratio,heuristic.score
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
CG11300,CG11300,474,9,5447.1744,0.62539446,Set 1,11.710502,145.34557
CG32198,CG32198,217,4,2784.8835,0.49615488,Set 1,10.862142,124.30547
CG13460,CG13460,258,3,1958.5329,0.31157878,Set 1,10.544253,115.31505
CG12310,CG12310,140,1,1642.5407,0.03958828,Set 1,10.625701,113.51003
CG13135,CG13135,89,8,2140.4677,0.93511941,Set 1,10.111288,111.87517
CG17134,CG17134,183,52,2826.4112,4.01105118,Set 1,9.139642,104.78843
CG32189,CG32189,111,0,1163.3159,0.00000000,Set 1,10.184027,103.72703
CG10918,CG10918,171,10,1372.4396,0.78699221,Set 1,9.584994,99.90993
CG13461,CG13461,105,0,896.1248,0.00000000,Set 1,9.807556,96.20393


##### df_sub300 = set.cell.type(df_sub300, "Malpighian tubule", df_sub300$cluster %in% b)

In [789]:
# α-Tubulin at 85E (αTub85E) encodes the major α-Tubulin constituent of accessory and 
# attachment cells of chordotonal organs. [Date last reviewed: 2019-03-21] (FlyBase Gene Snapshot)
# 141 (f) also has chordotonal signal from alphaTub85E, CG7409
df_sub300 = set.cell.type(df_sub300, "Chordotonal_organ", df_sub300$cluster %in% c)

Assigning cells for cell type Chordotonal_organ



In [93]:
clus = c(c)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [732]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "c")  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG740954,c,CG7409,458,16,3866.9126,0.3382439,11.496625,137.00919
CG4314954,c,CG43149,436,27,3464.9822,1.1330986,10.665682,125.41827
CG1313354,c,CG13133,316,16,2682.3791,0.6332884,10.681518,121.66073
CG1444554,c,CG14445,395,65,2888.5918,1.5446626,10.148676,116.67578
alphaTub85E54,c,alphaTub85E,2602,2401,21206.5698,80.7327894,8.019381,115.25688
CG3134554,c,CG31345,154,76,1216.6799,2.8782176,8.293340,85.00607
sosie54,c,sosie,127,118,973.5354,4.2438966,7.536450,74.82618
Tektin-C54,c,Tektin-C,177,141,1197.0941,6.6733289,7.285468,74.50503
epsilonTry54,c,epsilonTry,41,371,1166.9506,8.3214057,6.967980,71.00207


In [94]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("alphaTub85E"), 
                   size_pos = 4, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG7409"), 
                   size_pos = 3, size_neg = 0.1, stroke = 0.02, limit = 1)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG13133"), 
                   size_pos = 3, size_neg = 0.2, stroke = 0.02, limit = 0.01)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("alphaTub85E"), size_pos = 4, : could not find function "plot.expr_clus.br2"


In [1719]:
df_sub300 = set.cell.type(df_sub300, "Neuron_unknown_2", df_sub300$cluster %in% d)
# based on presence of Nplp4 a neuropeptide.

Assigning cells for cell type Neuron_unknown_2



In [1718]:
df_sub300  %>% count(cell.type.3)

cell.type.3,n
<chr>,<int>
Amnioserosa,7091
Analpad,850
Analpad_large_intestine,734
Antennal_sense_organs,748
Bolwig_organ,115
Central_brain_ventral_nerve_cord_1,6846
Central_brain_ventral_nerve_cord_2,10161
Central_brain_ventral_nerve_cord_early,5058
Central_brain_ventral_nerve_cord_primorium,5819


In [95]:
clus = c(d)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'd' not found


In [1715]:
cell.type.2.markers_no_doubletons  %>% head

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Muc26B,a,Muc26B,3651,121,11236.035,3.9689778,11.142896,149.9385
CG7017,a,CG7017,3415,236,12317.772,9.9390129,10.137043,137.7479
CG740954,c,CG7409,458,16,3866.913,0.3382439,11.496625,137.0092
CG1137061,d,CG11370,2890,826,16311.339,18.7725763,9.688159,135.5730
CG746563,f,CG7465,20683,922,18938.232,27.8101498,9.360509,133.0043
CG3260363,f,CG32603,9367,299,9354.678,8.3255405,9.970285,131.5243


In [1716]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "d" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG1137061,d,CG11370,2890,826,16311.3389,18.7725763,9.688159,135.57295
Nplp461,d,Nplp4,494,28,3733.8508,0.9382501,10.911694,129.48727
CG1456661,d,CG14566,407,62,4152.7877,2.2952246,10.299488,123.80202
CG1141361,d,CG11413,660,87,3416.5312,3.5819705,9.542348,112.01514
CG801261,d,CG8012,150,11,1085.4801,0.1985319,9.822849,99.06782
CG1306761,d,CG13067,353,125,2263.7980,3.9548949,8.835675,98.47507
CG1367861,d,CG13678,280,177,2273.6973,6.6786228,8.209977,91.55322
CG1304461,d,CG13044,516,318,2717.6486,10.3132017,7.908208,90.22216
CG863061,d,CG8630,381,300,2484.6331,9.0119569,7.955165,89.72947


In [96]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG11370"), 
                   size_pos = 4, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Nplp4"), 
                   size_pos = 3, size_neg = 0.1, stroke = 0.02, limit = 1)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG14566"), 
                   size_pos = 3, size_neg = 0.2, stroke = 0.02, limit = 0.01)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("CG11370"), size_pos = 4, : could not find function "plot.expr_clus.br2"


In [ ]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("syt"), 
                   size_pos = 4, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Nplp4"), 
                   size_pos = 3, size_neg = 0.1, stroke = 0.02, limit = 1)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG14566"), 
                   size_pos = 3, size_neg = 0.2, stroke = 0.02, limit = 0.01)

### e cluster

In [790]:
df_sub300 = set.cell.type(df_sub300, "Multidendritic_sensory", df_sub300$cluster %in% e &
                        df_sub300$umap_2 < -1*df_sub300$umap_1 -6.7 & 
                          df_sub300$umap_2 > -1* df_sub300$umap_1 -7.2 & df_sub300$umap_1 > -1 )

Assigning cells for cell type Multidendritic_sensory



In [791]:
df_sub300 = set.cell.type(df_sub300, "External_sensory", df_sub300$cluster %in% e &
                        df_sub300$umap_2 < -1*df_sub300$umap_1 -6.3 & 
                          df_sub300$umap_2 > -1* df_sub300$umap_1 -6.7 & df_sub300$umap_1 > 0 )

Assigning cells for cell type External_sensory



In [792]:
df_sub300 = set.cell.type(df_sub300, "Chordotonal_sensory", df_sub300$cluster %in% c(ao,e) &
                        df_sub300$umap_2 > -1*df_sub300$umap_1 -6.1 & 
                           df_sub300$umap_1 > 0.3 )

Assigning cells for cell type Chordotonal_sensory



In [97]:
clus = c(e)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = -1, intercept = -7.2) +
geom_abline(slope = -1., intercept = -6.7) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(-4, 2) + ylim(-9, -3)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'e' not found


In [98]:
clus = c(e)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &
              umap_2 < -1*umap_1 -6.7 & umap_2 > -1* umap_1 -7.2 & umap_1 > -1), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = -1, intercept = -7.2) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(-4, 2) + ylim(-9, -3)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'e' not found


In [99]:
clus = c(ao, e)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = -1, intercept = -7.2) +
geom_abline(slope = -1., intercept = -6.7) +
geom_abline(slope = -1., intercept = -6.3) +
xlim(-4, 2) + ylim(-9, -3)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ao' not found


In [100]:
clus = c(ao, e)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &
              umap_2 > -1*umap_1 -6.1 & umap_1 > 0.3), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = -1, intercept = -7.2) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(-4, 2) + ylim(-9, -3)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ao' not found


In [804]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "e")  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
pHCl-162,e,pHCl-1,2151,815,1166.77111,33.206456,5.092109,51.88626
para62,e,para,5159,4049,2547.94885,165.041233,3.939723,44.58067
ppk392,e,ppk,376,56,139.05923,2.071240,5.500734,39.21965
Gfrl62,e,Gfrl,1056,837,569.79301,35.666195,3.957915,36.24193
unc-13-4A62,e,unc-13-4A,1096,930,584.54631,37.067356,3.940691,36.22929
ppk2662,e,ppk26,256,40,92.57590,1.082549,5.474214,35.84551
cac62,e,cac,6189,10432,3502.51877,479.896483,2.864595,33.72942
eys62,e,eys,1023,1070,599.82083,46.664328,3.653550,33.72515
igl62,e,igl,542,383,269.91856,14.058442,4.163880,33.65129


In [101]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("pHCl-1"), 
                   size_pos = 4, size_neg = 0.05, stroke = 0.008, limit = 3)
plot.expr_clus.br2(cds_sub_no_doubletons, c("para"), 
                   size_pos = 2, size_neg = 0.1, stroke = 0.02, limit = 3)
plot.expr_clus.br2(cds_sub_no_doubletons, c("ppk"), 
                   size_pos = 3, size_neg = 0.2, stroke = 0.02, limit = 0.01)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("pHCl-1"), size_pos = 4, : could not find function "plot.expr_clus.br2"


In [812]:
df_sub300 = set.cell.type(df_sub300, "unknown_2", df_sub300$cluster %in% f)

Assigning cells for cell type unknown_2



In [102]:
clus = c(f)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'f' not found


In [802]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "f")  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG746563,f,CG7465,20683,922,18938.2318,27.8101498,9.360509,133.00431
CG3260363,f,CG32603,9367,299,9354.6783,8.3255405,9.970285,131.52428
CG1135063,f,CG11350,7226,169,6348.3690,4.8244082,10.090059,127.46163
CG1442163,f,CG14421,3808,49,3552.7510,1.5882504,10.422744,122.93758
CG1442463,f,CG14424,3717,73,3555.1975,2.4802559,9.996521,117.92015
CG3224166,f,CG32241,4339,150,3941.1991,3.9336622,9.641760,115.16875
CG729963,f,CG7299,9476,705,8369.3150,24.0240067,8.385653,109.27400
CG1442263,f,CG14422,1973,23,1519.9472,0.7092652,9.796429,103.55565
CG3045763,f,CG30457,8344,834,7163.4002,26.3604659,8.032408,102.86808


In [103]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG7465"), 
                   size_pos = 2, size_neg = 0.5, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG32603"), 
                   size_pos = 2, size_neg = 0.1, stroke = 0.02, limit = 3)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG11350"), 
                   size_pos = 3, size_neg = 0.2, stroke = 0.02, limit = 1)
## all label both clusters -- throughout.  
## CG11350 and CG32603 "Predicted to be part of RNA polymerase II transcription regulator complex."
## shares markers with epidermis (i)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("CG7465"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [817]:
df_sub300 = set.cell.type(df_sub300, "gut_query", df_sub300$cluster %in% g)

Assigning cells for cell type gut_query



In [104]:
clus = c(g)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'g' not found


In [814]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "g")  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Jon99Cii64,g,Jon99Cii,6060,300,7851.6008,9.6160613,9.530594,123.31593
Jon99Ciii64,g,Jon99Ciii,4035,217,5562.8124,7.4670202,9.359744,116.45261
CG1818064,g,CG18180,3716,103,3973.0577,3.8846047,9.667792,115.59196
CG729064,g,CG7290,1400,64,2807.3954,1.7413531,10.000128,114.55677
Jon99Fii64,g,Jon99Fii,2876,109,3492.1304,3.7020013,9.536617,112.24889
CG1091064,g,CG10910,766,22,1898.3644,0.6184414,10.195936,111.04701
CG968264,g,CG9682,600,39,2150.8132,1.3179139,9.857840,109.13947
Jon65Aiv64,g,Jon65Aiv,1764,99,2960.9984,3.4616372,9.374295,108.10770
Jon25Biii64,g,Jon25Biii,2186,117,3357.9222,4.8876112,9.155671,107.24754


In [105]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Jon99Cii"), 
                   size_pos = 4, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Jon99Ciii"), 
                   size_pos = 3, size_neg = 0.1, stroke = 0.02, limit = 3)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Jon65Aiv"), 
                   size_pos = 3, size_neg = 0.2, stroke = 0.02, limit = 2)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Jon99Cii"), size_pos = 4, : could not find function "plot.expr_clus.br2"


In [823]:
df_sub300 = set.cell.type(df_sub300, "Crystal_cell", df_sub300$cluster %in% h)

Assigning cells for cell type Crystal_cell



In [106]:
clus = c(h)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'h' not found


In [819]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "h")  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Pde1c65,h,Pde1c,5618,4675,44234.440995,206.433746,7.736376,119.39483
lz65,h,lz,331,131,2653.352391,5.413617,8.692462,98.86932
PPO265,h,PPO2,908,2330,7240.858646,80.280150,6.477114,83.05049
PPO165,h,PPO1,682,1477,5267.829417,53.895262,6.584383,81.40449
CG4236965,h,CG42369,167,214,1411.183520,8.616885,7.197120,75.30859
CG1374365,h,CG13743,286,570,2193.693999,20.336551,6.683892,74.18989
CG3132465,h,CG31324,506,1989,3917.574868,89.337219,5.438497,64.91451
para65,h,para,0,5394,0.000000,221.106880,7.788600,60.71300
qless65,h,qless,0,5385,0.000000,209.657456,7.711890,59.52619


In [107]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Pde1c"), 
                   size_pos = 4, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("lz"), 
                   size_pos = 3, size_neg = 0.1, stroke = 0.02, limit = 3)
plot.expr_clus.br2(cds_sub_no_doubletons, c("PPO2"), 
                   size_pos = 3, size_neg = 0.2, stroke = 0.02, limit = 8)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Pde1c"), size_pos = 4, : could not find function "plot.expr_clus.br2"


In [856]:
df_sub300 = set.cell.type(df_sub300, "Epidermis", df_sub300$cluster %in% c(i, ap) &
                         (df_sub300$umap_2 < 0.5*df_sub300$umap_1 -4.2 | df_sub300$umap_2 < 0.9* df_sub300$umap_1 -5.4 & df_sub300$umap_1 > -1)  &
              df_sub300$umap_2 > 0.9*df_sub300$umap_1 -6.7)

Assigning cells for cell type Epidermis



In [108]:
clus = c(i)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'i' not found


In [825]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "i")  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ELOVL66,i,ELOVL,7058,4681,2239.4818,168.48327,3.723950,41.44605
Osi666,i,Osi6,63628,81136,20941.4140,3251.02531,2.686948,38.56883
frm66,i,frm,4034,2856,1200.3364,97.60995,3.605562,36.88643
CG3269466,i,CG32694,3694,2363,1070.9823,90.18310,3.554024,35.77504
Cht666,i,Cht6,4982,3946,1552.5921,148.92165,3.372398,35.75212
dyl66,i,dyl,10725,11990,3456.0333,455.42317,2.920673,34.33344
CG532666,i,CG5326,1800,1053,498.1302,34.38688,3.815236,34.19700
CG1021166,i,CG10211,7358,7622,2395.9431,290.61937,3.038435,34.11245
CG941166,i,CG9411,10035,12260,3191.8854,437.69971,2.863103,33.32837


In [109]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("ELOVL"), 
                   size_pos = 3, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Cht6"), 
                   size_pos = 3, size_neg = 0.1, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("frm"), 
                   size_pos = 3, size_neg = 0.2, stroke = 0.02, limit = 3)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("ELOVL"), size_pos = 3, : could not find function "plot.expr_clus.br2"


In [110]:
clus = c(i, ap)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.5, intercept = -4.2) +
geom_abline(slope = 0.9, intercept = -6.7) +
geom_abline(slope = 0.9, intercept = -5.4) # +
# xlim(-4, 2) + ylim(-9, -3)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'i' not found


In [111]:
clus = c(i, ap)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &
              (umap_2 < 0.5*umap_1 -4.2 | umap_2 < 0.9* umap_1 -5.4 & umap_1 > -1)  &
              umap_2 > 0.9*umap_1 -6.7), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = -1, intercept = -7.2) +
geom_abline(slope = -1., intercept = -6.7) # +
# xlim(-4, 2) + ylim(-9, -3)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'i' not found


### j cluster

In [862]:
df_sub300 = set.cell.type(df_sub300, "Epidermis_late", df_sub300$cluster %in% j)

Assigning cells for cell type Epidermis_late



In [112]:
clus = c(j)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'j' not found


In [860]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "j")  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG1158167,j,CG11581,257,44,2564.3011,1.1160558,10.242972,116.00076
Tb67,j,Tb,814,491,8391.4464,19.2476737,8.695020,113.33850
CG975767,j,CG9757,390,524,4011.9120,12.9308165,8.169866,97.79684
CG335587,j,CG3355,282,228,2712.3344,6.7587166,8.449501,96.37375
TwdlR67,j,TwdlR,89,9,1051.6910,0.3729667,9.581199,96.19395
Cpr47Ec67,j,Cpr47Ec,99,46,1186.7214,1.5841392,8.843082,90.32307
CG3162667,j,CG31626,1007,2460,9432.7732,86.1951177,6.757291,89.22069
TwdlQ67,j,TwdlQ,114,86,1332.2345,3.1993661,8.309461,86.25814
CG1158467,j,CG11584,412,1078,3697.6084,38.4811689,6.549284,77.62714


In [113]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Tb"), 
                   size_pos = 4, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("TwdlR"), 
                   size_pos = 3, size_neg = 0.1, stroke = 0.02, limit = 1)
plot.expr_clus.br2(cds_sub_no_doubletons, c("TwdlQ"), 
                   size_pos = 3, size_neg = 0.2, stroke = 0.02, limit = 0.01)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Tb"), size_pos = 4, : could not find function "plot.expr_clus.br2"


In [865]:
df_sub300 = set.cell.type(df_sub300, "uncertain_1", df_sub300$cluster %in% k)
## very small; markers specific but very low, might be artifact

Assigning cells for cell type uncertain_1



In [114]:
clus = c(k)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'k' not found


In [864]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "k")  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG3195468,k,CG31954,28,11,1255.6939,0.3485554,9.862854,101.54220
CG3444768,k,CG34447,27,6,1141.3501,0.1320903,9.977537,101.34971
CG525568,k,CG5255,20,13,1073.4703,0.4351591,9.546856,96.13120
CG401768,k,CG4017,12,6,613.0370,0.2387753,8.950916,82.90501
fng68,k,fng,0,11165,0.0000,524.1208112,9.033756,81.63358
CG4333268,k,CG43332,0,7448,0.0000,354.2582008,8.468657,71.75260
Nipped-A68,k,Nipped-A,0,8224,0.0000,322.5693846,8.333466,69.48386
Ser99,k,Ser,0,6688,0.0000,304.5744894,8.250651,68.11226
MYPT-75D68,k,MYPT-75D,0,6518,0.0000,291.5150520,8.187427,67.07440


In [115]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG31954"), 
                   size_pos = 4, size_neg = 0.05, stroke = 0.008, limit = 1)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG34447"), 
                   size_pos = 3, size_neg = 0.1, stroke = 0.02, limit = 1)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG5255"), 
                   size_pos = 3, size_neg = 0.2, stroke = 0.02, limit = 0.01)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("CG31954"), size_pos = 4, : could not find function "plot.expr_clus.br2"


In [894]:
df_sub300 = set.cell.type(df_sub300, "Amnioserosa", df_sub300$cluster %in% c(l, m, n))

Assigning cells for cell type Amnioserosa



In [116]:
clus = c(l, m,n)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'l' not found


In [868]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "l")  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG1201169,l,CG12011,45157,23686,8.132876e+04,938.912102,6.435096,104.96604
CG4333269,l,CG43332,9720,4780,1.654825e+04,208.926590,6.300650,88.30033
CG1808869,l,CG18088,2885,1749,4.975342e+03,73.930836,6.053092,74.33724
CG799769,l,CG7997,2571,2881,4.458669e+03,121.222249,5.189034,62.90521
Muc30E69,l,Muc30E,3517,4295,5.627461e+03,184.366425,4.924032,61.34618
CG1573169,l,CG15731,0,5739,0.000000e+00,193.252562,7.594344,57.73060
CG1373169,l,CG13731,0,5381,0.000000e+00,165.166416,7.367777,54.34829
CG296269,l,CG2962,16,31237,2.663353e+01,1109.593103,5.327467,53.89860
Osi1569,l,Osi15,10,20217,1.856681e+01,804.191644,5.361059,51.75131


In [117]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG43332"), 
                   size_pos = 4, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG18088"), 
                   size_pos = 3, size_neg = 0.1, stroke = 0.02, limit = 2)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG7997"), 
                   size_pos = 3, size_neg = 0.2, stroke = 0.02, limit = 2)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("CG43332"), size_pos = 4, : could not find function "plot.expr_clus.br2"


In [883]:
df_sub300_no_doubletons = make_df(cds_sub_no_doubletons)

In [888]:
two.set.differential.gene.test.3a(cds_sub_no_doubletons, df_sub300_no_doubletons$cell.type.2 == "l", 
                                 df_sub300_no_doubletons$cell.type.2 == "m")  %>% head

# of cells sampled in set 1: 1562

# of cells sampled in set 2: 2664



,gene,set.1.n.umi,set.2.n.umi,set.1.tpm,set.2.tpm,higher.expr,log2.ratio,heuristic.score
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
fipi,fipi,177,4823,282.94543,4931.2569,Set 2,4.118270,50.52307
CG43861,CG43861,15,548,22.67978,551.8351,Set 2,4.542509,41.38547
Orcokinin,Orcokinin,7,307,12.35747,353.6630,Set 2,4.726656,40.03622
CG31145,CG31145,79,1430,108.17657,1438.6929,Set 2,3.720023,39.02879
a,a,45,958,76.79683,1008.1222,Set 2,3.695815,36.88012
CG12011,CG12011,44881,18632,80845.62026,18782.3481,Set 1,2.105715,34.32927


In [118]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("fipi"), 
                   size_pos = 4, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG43861"), 
                   size_pos = 3, size_neg = 0.1, stroke = 0.02, limit = 2)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Orcokinin"), 
                   size_pos = 3, size_neg = 0.2, stroke = 0.02, limit = 2)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("fipi"), size_pos = 4, : could not find function "plot.expr_clus.br2"


### l cluster not big enough differences below to warrant separating them!

In [889]:
two.set.differential.gene.test.3a(cds_sub_no_doubletons, df_sub300_no_doubletons$cell.type.2 == "l", 
                                 df_sub300_no_doubletons$cell.type.2 == "m")  %>% 
filter(higher.expr == "Set 1")  %>% head

# of cells sampled in set 1: 1562

# of cells sampled in set 2: 2664



,gene,set.1.n.umi,set.2.n.umi,set.1.tpm,set.2.tpm,higher.expr,log2.ratio,heuristic.score
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
CG12011,CG12011,44881,18632,80845.62026,18782.34808,Set 1,2.105715,34.32927
CG6753,CG6753,77,14,142.72037,10.01414,Set 1,3.695763,26.48798
Sr-CII,Sr-CII,17,0,33.16107,0.00000,Set 1,5.051419,25.73335
bnb,bnb,5124,2795,8638.59911,2572.09225,Set 1,1.747294,22.84893
CG18417,CG18417,26,2,46.44776,1.92734,Set 1,3.987947,22.20596
link,link,271,86,421.45603,71.55207,Set 1,2.538293,22.14066


In [119]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG6753"), 
                   size_pos = 4, size_neg = 0.05, stroke = 0.008, limit = 1)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Sr-CII"), 
                   size_pos = 3, size_neg = 0.1, stroke = 0.02, limit = 2)
plot.expr_clus.br2(cds_sub_no_doubletons, c("link"), 
                   size_pos = 3, size_neg = 0.2, stroke = 0.02, limit = 2)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("CG6753"), size_pos = 4, : could not find function "plot.expr_clus.br2"


df_sub300 = set.cell.type(df_sub300, "m", df_sub300$cluster %in% m)

df_sub300 = set.cell.type(df_sub300, "n", df_sub300$cluster %in% n)

In [899]:
df_sub300 = set.cell.type(df_sub300, "Proventriculus", df_sub300$cluster %in% o)

Assigning cells for cell type Proventriculus



In [120]:
clus = c(o)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'o' not found


In [896]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "o")  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG4389672,o,CG43896,5171,116,2550.7945,4.2726933,8.918191,100.92981
CG771572,o,CG7715,5047,648,2729.6245,20.1938883,7.008910,80.00682
CG1380672,o,CG13806,1300,780,1851.1054,30.5982426,5.872399,63.74460
Cda972,o,Cda9,523,15,268.3877,0.5763477,7.411589,59.83776
obst-F72,o,obst-F,412,12,206.2760,0.4334067,7.168984,55.16827
CG1581872,o,CG15818,665,609,991.2771,25.0539270,5.249716,52.25882
Kaz-m172,o,Kaz-m1,443,313,620.6277,13.0898354,5.461001,50.67758
Lim372,o,Lim3,1878,2662,2678.8615,122.1671318,4.442931,50.59584
CG3041372,o,CG30413,595,492,826.1740,21.6057364,5.191685,50.31805


In [121]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG43896"), 
                   size_pos = 3, size_neg = 0.05, stroke = 0.008, limit = 1)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG7715"), 
                   size_pos = 3, size_neg = 0.1, stroke = 0.02, limit = 3)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Cda9"), 
                   size_pos = 3, size_neg = 0.2, stroke = 0.02, limit = 1)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("CG43896"), size_pos = 3, : could not find function "plot.expr_clus.br2"


In [905]:
df_sub300 = set.cell.type(df_sub300, "Peptidergic_neuron", df_sub300$cluster %in% p) 
# from neuron subset

Assigning cells for cell type Peptidergic_neuron



In [122]:
clus = c(p)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'p' not found


In [900]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "p")  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG457773,p,CG4577,146,117,2994.3931,5.1710612,8.922529,103.04209
CG4254173,p,CG42541,318,1414,7257.9265,61.4327884,6.861110,87.99744
AstC73,p,AstC,62,42,667.1167,0.8393817,8.502574,79.78779
CG4333273,p,CG43332,0,7374,0.0000,354.7325993,8.470588,71.78526
CG3438673,p,CG34386,14,16,443.0765,0.7154520,8.012823,70.47009
Nplp173,p,Nplp1,43,236,1266.2370,10.5808130,6.772667,69.80907
Mip73,p,Mip,36,223,999.5280,7.8848033,6.813763,67.90968
MsR173,p,MsR1,39,165,962.1831,7.3577280,6.847057,67.86574
cas73,p,cas,0,5896,0.0000,272.4327174,8.089756,65.48692


In [123]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG4577"), 
                   size_pos = 3, size_neg = 0.05, stroke = 0.008, limit = 1)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG42541"), 
                   size_pos = 3, size_neg = 0.1, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("AstC"), 
                   size_pos = 3, size_neg = 0.2, stroke = 0.02, limit = 1)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("CG4577"), size_pos = 3, : could not find function "plot.expr_clus.br2"


In [925]:
df_sub300 = set.cell.type(df_sub300, "Cholinergic", df_sub300$cluster %in% c(q, r) &
                          df_sub300$umap_2 < -1.3*df_sub300$umap_1 -8.7
                         )

Assigning cells for cell type Cholinergic



In [926]:
df_sub300 = set.cell.type(df_sub300, "Glutamatergic", df_sub300$cluster %in% c(q, r) &
                          df_sub300$umap_2 >= -1.3*df_sub300$umap_1 -8.7
                         )

Assigning cells for cell type Glutamatergic



In [124]:
clus = c(q)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [906]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "q")  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
nAChRalpha674,q,nAChRalpha6,9146,3318,2902.6524,122.78563,4.551456,52.35837
Rdl74,q,Rdl,6730,2886,2103.3193,99.88408,4.381897,48.37237
Cngl74,q,Cngl,4977,1781,1546.7907,64.83379,4.554306,48.25740
Nckx30C74,q,Nckx30C,8208,3813,2576.2272,138.91031,4.202686,47.62317
Syt194,q,Syt1,8071,4526,2546.2870,175.92800,3.847161,43.52965
Rbp674,q,Rbp6,15601,11186,4905.8670,418.31458,3.548403,43.50550
mmd74,q,mmd,4766,2269,1504.2977,86.22446,4.108213,43.36562
cac74,q,cac,13342,8632,4243.3721,372.97304,3.504205,42.23035
GluClalpha74,q,GluClalpha,3455,1495,1077.0762,59.66785,4.150044,41.80856


In [125]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("nAChRalpha6"), 
                   size_pos = 3, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Rdl"), 
                   size_pos = 3, size_neg = 0.1, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Cngl"), 
                   size_pos = 3, size_neg = 0.2, stroke = 0.02, limit = 3)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("nAChRalpha6"), size_pos = 3, : could not find function "plot.expr_clus.br2"


In [126]:
clus = c(r,q)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = -1.3, intercept = -8.7) +
geom_abline(slope = 0.9, intercept = -6.7) +
geom_abline(slope = 0.9, intercept = -5.4) +
 xlim(-4, 2) + ylim(-9, -3)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'r' not found


In [ ]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &
              df_sub300$umap_2 < -1.3*umap_1 -8.7), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = -1.3, intercept = -8.7) +
geom_abline(slope = 0.9, intercept = -6.7) +
geom_abline(slope = 0.9, intercept = -5.4) +
 xlim(-4, 2) + ylim(-9, -3)
print(peedeef)

In [ ]:
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &
              df_sub300$umap_2 >= -1.3*umap_1 -8.7), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = -1.3, intercept = -8.7) +
geom_abline(slope = 0.9, intercept = -6.7) +
geom_abline(slope = 0.9, intercept = -5.4) +
 xlim(-4, 2) + ylim(-9, -3)
print(peedeef)

In [136]:
df_sub300 = set.cell.type(df_sub300, "r", df_sub300$cluster %in% r)

Assigning cells for cell type r



### s cluster

In [933]:
df_sub300 = set.cell.type(df_sub300, "Fat_body", df_sub300$cluster %in% s)

Assigning cells for cell type Fat_body



In [127]:
clus = c(s)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 's' not found


In [930]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "s")  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
fon76,s,fon,3297,647,1916.1924,21.780420,6.394304,69.72848
apolpp76,s,apolpp,52336,66912,34062.8642,2123.758505,4.002829,60.26641
CG223376,s,CG2233,1722,326,915.3464,12.929236,6.038130,59.41368
NimB276,s,NimB2,2470,1051,1639.1646,39.603412,5.335216,56.97811
CG273676,s,CG2736,1281,379,839.0123,14.842270,5.726841,55.63206
Glt76,s,Glt,5086,3177,3462.6335,135.959769,4.660046,54.79315
CG722776,s,CG7227,3191,1839,2214.8709,81.024817,4.755018,52.84565
Idgf276,s,Idgf2,2374,1458,1643.7176,56.854241,4.828396,51.58477
teq76,s,teq,1727,856,1167.4463,35.371768,5.004393,50.99665


In [128]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("fon"), 
                   size_pos = 3, size_neg = 0.05, stroke = 0.008, limit = 3)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG2233"), 
                   size_pos = 3, size_neg = 0.1, stroke = 0.02, limit = 3)
plot.expr_clus.br2(cds_sub_no_doubletons, c("NimB2"), 
                   size_pos = 3, size_neg = 0.2, stroke = 0.02, limit = 3)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("fon"), size_pos = 3, : could not find function "plot.expr_clus.br2"


In [950]:
df_sub300 = set.cell.type(df_sub300, "Monoaminergic", df_sub300$cluster %in% t &
                          df_sub300$umap_2 < -1*df_sub300$umap_1 -9.8
                         )

Assigning cells for cell type Monoaminergic



In [952]:
df_sub300 = set.cell.type(df_sub300, "Neuron_unknown_1", df_sub300$cluster %in% t &
                          df_sub300$umap_2 >= -1*df_sub300$umap_1 -9.8
                         )

Assigning cells for cell type Neuron_unknown_1



In [953]:
df_sub300  %>% count(cell.type.3)

cell.type.3,n
<chr>,<int>
Amnioserosa,7091
Cholinergic,8949
Chordotonal_organ,121
Chordotonal_sensory,750
Crystal_cell,432
Epidermis,10373
Epidermis_late,184
External_sensory,647
Fat_body,4732


In [129]:
clus = c(t)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [935]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "t")  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Vmat77,t,Vmat,3511,1741,10124.6978,60.350716,7.366583,98.01779
Tbh77,t,Tbh,582,179,1643.1032,6.388548,7.796917,83.29512
Tdc277,t,Tdc2,203,119,603.9666,4.077435,6.894225,63.70754
CG3119184,t,CG31191,224,213,679.0681,8.786192,6.116665,57.55498
CG226977,t,CG2269,252,239,715.7692,10.506789,5.958937,56.52269
CG4500277,t,CG45002,725,1555,2033.4604,69.040869,4.859596,53.40905
SLO277,t,SLO2,504,1064,1383.0233,45.940672,4.880843,50.92990
GluRIB77,t,GluRIB,330,631,936.9447,27.792948,5.024176,49.60550
DAT77,t,DAT,126,184,399.6778,6.840168,5.671809,49.04016


In [130]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Vmat"), 
                   size_pos = 1, size_neg = 0.05, stroke = 0.008, limit = 4)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Tbh"), 
                   size_pos = 1, size_neg = 0.1, stroke = 0.02, limit = 4)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Tdc2"), 
                   size_pos = 3, size_neg = 0.2, stroke = 0.02, limit = 3)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Vmat"), size_pos = 1, : could not find function "plot.expr_clus.br2"


In [131]:
clus = c(t)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = -1, intercept = -9.8) +
geom_abline(slope = -1., intercept = -6.7) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(-4, 2) + ylim(-9, -3)
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [132]:
clus = c(t)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus & 
              df_sub300$umap_2 >= -1*umap_1 -9.8), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = -1, intercept = -9.8) +
geom_abline(slope = -1., intercept = -6.7) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(-4, 2) + ylim(-9, -3)
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [133]:
clus = c(t)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus & 
              df_sub300$umap_2 < -1*umap_1 -9.8), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = -1, intercept = -9.8) +
geom_abline(slope = -1., intercept = -6.7) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(-4, 2) + ylim(-9, -3)
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [943]:
two.set.differential.gene.test.3a(cds_sub_no_doubletons, 
                                  df_sub300_no_doubletons$cell.type.2 == "t" & 
                                  df_sub300_no_doubletons$umap_2 < -1*df_sub300_no_doubletons$umap_1 -9.8 , 
                                 df_sub300_no_doubletons$cell.type.2 == "t" & 
                                  df_sub300_no_doubletons$umap_2 >= -1*df_sub300_no_doubletons$umap_1 -9.8)  %>% 
head(20)

# of cells sampled in set 1: 414

# of cells sampled in set 2: 547



,gene,set.1.n.umi,set.2.n.umi,set.1.tpm,set.2.tpm,higher.expr,log2.ratio,heuristic.score
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
CG14459,CG14459,0,34,0.000000,144.18660,Set 2,7.171793,51.50613
Tbh,Tbh,555,43,3433.693516,216.95094,Set 1,3.977686,46.72176
DIP-eta,DIP-eta,6,209,39.463102,945.39880,Set 2,4.546244,44.94555
Tdc2,Tdc2,199,12,1313.603998,65.46006,Set 1,4.304899,44.60053
CG1688,CG1688,2,98,18.724488,491.28494,Set 2,4.638500,41.48373
odd,odd,0,16,0.000000,84.05808,Set 2,6.393315,40.98355
CG6154,CG6154,1,33,2.181986,149.58077,Set 2,5.554853,40.18600
TwdlW,TwdlW,11,0,77.280046,0.00000,Set 1,6.272024,39.45462
Hmr,Hmr,0,13,0.000000,69.88530,Set 2,6.126917,37.66470


In [134]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("DIP-eta"),  # Is expressed in optic lobe intrinsic columnar neurons
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 4)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG1688"), 
                   size_pos = 2, size_neg = 0.1, stroke = 0.02, limit = 4)
plot.expr_clus.br2(cds_sub_no_doubletons, c("SLO2"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 8)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("DIP-eta"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [139]:
df_sub300 = set.cell.type(df_sub300, "u", df_sub300$cluster %in% u)

Assigning cells for cell type u



In [135]:
clus = c(u)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'u' not found


In [961]:
df_sub300  %>% count(cluster, cell.type.2)  %>% filter(cell.type.2 == "u")

cluster,cell.type.2,n
<fct>,<chr>,<int>
157,u,610


In [957]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "u")  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG1409578,u,CG14095,678,363,3563.2732,13.9285223,7.898988,93.20326
Ccp84Ae78,u,Ccp84Ae,302,93,1612.0340,3.1663426,8.595885,91.59398
Irk278,u,Irk2,258,36,915.8146,1.0524391,8.801572,86.61175
CG854378,u,CG8543,191,10,770.4581,0.4802334,9.023748,86.55077
CG854178,u,CG8541,150,18,586.4125,0.6465243,8.476348,77.96740
CG1367478,u,CG13674,253,180,1334.0015,6.5432938,7.466350,77.52032
CG1305178,u,CG13051,196,93,930.2568,3.1252837,7.816992,77.09927
CG507078,u,CG5070,212,120,912.3066,4.5339374,7.365069,72.43512
CG1829478,u,CG18294,120,65,605.7769,2.0218477,7.647212,70.69864


In [136]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG14095"), 
                   size_pos = 3, size_neg = 0.05, stroke = 0.008, limit = 3)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Ccp84Ae"), 
                   size_pos = 3, size_neg = 0.1, stroke = 0.02, limit = 3)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Irk2"), 
                   size_pos = 3, size_neg = 0.2, stroke = 0.02, limit = 1)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("CG14095"), size_pos = 3, : could not find function "plot.expr_clus.br2"


In [965]:
df_sub300 = set.cell.type(df_sub300, "Plasmatocytes", df_sub300$cluster %in% v)

Assigning cells for cell type Plasmatocytes



In [137]:
clus = c(v)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'v' not found


In [963]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "v")  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Ppn79,v,Ppn,70207,14001,35737.5046,517.641752,6.106557,92.36284
Pxn79,v,Pxn,12373,2288,6346.7825,85.559596,6.196188,78.27047
Col4a179,v,Col4a1,32538,12541,16628.8867,493.331004,5.072070,71.11799
vkg79,v,vkg,20725,8194,10547.7162,322.180377,5.028447,67.20409
NtR79,v,NtR,1936,444,1010.1096,17.426035,5.776622,57.66065
NimC479,v,NimC4,5948,2861,3117.9010,105.681643,4.869191,56.51584
Ance-579,v,Ance-5,920,230,478.2397,8.224957,5.696048,50.72105
Plod79,v,Plod,14578,11757,7469.3525,495.350830,3.911551,50.32977
Karl79,v,Karl,845,194,434.3417,7.252419,5.717870,50.12287


In [138]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Ppn"), 
                   size_pos = 1, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Pxn"), 
                   size_pos = 1, size_neg = 0.1, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Col4a1"), 
                   size_pos = 1, size_neg = 0.2, stroke = 0.02, limit = 8)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Ppn"), size_pos = 1, : could not find function "plot.expr_clus.br2"


In [969]:
df_sub300 = set.cell.type(df_sub300, "Antennal_sense_organs", df_sub300$cluster %in% w)

Assigning cells for cell type Antennal_sense_organs



In [139]:
clus = c(w)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'w' not found


In [966]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "w")  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
nompA80,w,nompA,1553,1351,7555.3387,53.6086022,7.112224,91.63014
CG1291180,w,CG12911,825,597,3894.4554,23.3972386,7.318560,87.29268
CG1331880,w,CG13318,193,94,928.5511,3.5219592,7.681890,75.74643
CG1286980,w,CG12869,198,168,958.5543,6.5469492,6.988823,69.23282
CG1250780,w,CG12507,158,123,759.2947,4.6364366,7.073733,67.69856
CG1427780,w,CG14277,94,105,477.8394,2.1491981,7.245398,64.50866
sv80,w,sv,910,2515,4167.8641,108.4272849,5.251264,63.14875
CG1330580,w,CG13305,81,141,411.1573,5.3392267,6.019240,52.28945
CG1445680,w,CG14456,44,131,226.0664,1.9646874,6.252723,48.93988


In [140]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("nompA"), 
                   size_pos = 1, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG12911"), 
                   size_pos = 1, size_neg = 0.1, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("sv"), 
                   size_pos = 3, size_neg = 0.2, stroke = 0.02, limit = 3)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("nompA"), size_pos = 1, : could not find function "plot.expr_clus.br2"


In [976]:
df_sub300 = set.cell.type(df_sub300, "Cuprophilic_cells", df_sub300$cluster %in% x)

Assigning cells for cell type Cuprophilic_cells



In [141]:
clus = c(x)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'x' not found


In [971]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "x")  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Vha100-481,x,Vha100-4,616,289,3170.249334,12.3098817,7.895955,91.83655
lab81,x,lab,1815,2588,9129.744553,119.9039820,6.238641,82.07878
CG3144681,x,CG31446,157,34,876.870940,1.7843818,8.298864,81.14517
CG746581,x,CG7465,0,5957,0.000000,159.9967739,7.321899,53.67602
Pgcl81,x,Pgcl,59,137,304.959681,5.2740117,5.603086,46.26579
Ugt307A181,x,Ugt307A1,18,3,110.561474,0.1240624,6.619983,45.02710
CG577081,x,CG5770,23,17,126.194380,0.6870172,6.225029,43.51850
CG1368101,x,CG1368,0,3034,0.000000,96.1787791,6.587647,43.49539
elB81,x,elB,1000,10092,5006.104342,444.5248346,3.490111,42.89262


In [142]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Vha100-4"), 
                   size_pos = 1, size_neg = 0.05, stroke = 0.008, limit = 3)
plot.expr_clus.br2(cds_sub_no_doubletons, c("lab"), 
                   size_pos = 3, size_neg = 0.1, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG31446"), 
                   size_pos = 1, size_neg = 0.2, stroke = 0.02, limit = 1)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Vha100-4"), size_pos = 1, : could not find function "plot.expr_clus.br2"


In [1005]:
df_sub300 = set.cell.type(df_sub300, "Pharynx", df_sub300$cluster %in% y & 
                         df_sub300$umap_2 >= 1*df_sub300$umap_1 -8.1)

Assigning cells for cell type Pharynx



In [143]:
clus = c(y)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 4, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
xlim(2, 6) + ylim(-6, -2)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'y' not found


In [144]:
clus = c(y)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus & 
              df_sub300$umap_2 >= 1*umap_1 -8.1), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 1, intercept = -8.1) +
geom_abline(slope = -1., intercept = -6.7) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(1, 5) + ylim(-6, -2)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'y' not found


In [978]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "y")  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
dy82,y,dy,3438,1838,6602.9940,81.644936,6.320050,80.19589
TwdlT82,y,TwdlT,1425,1350,2800.1017,48.255267,5.829057,66.75307
CG856882,y,CG8568,283,93,500.8593,3.358475,6.844438,61.40241
CAH682,y,CAH6,200,37,386.8225,1.840266,7.089502,60.96442
CG4233182,y,CG42331,1854,2499,3570.6358,116.127642,4.930028,58.18601
jv82,y,jv,832,844,1550.9703,36.343657,5.376164,56.98672
Muc91C82,y,Muc91C,1434,2484,2686.0894,94.865583,4.808351,54.77591
CG3117682,y,CG31176,1750,2747,3210.0636,123.954414,4.683128,54.55299
ImpE182,y,ImpE1,4556,11123,8564.7561,511.768632,4.062032,53.06787


In [145]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("dy"), 
                   size_pos = 1, size_neg = 0.05, stroke = 0.008, limit = 3)
plot.expr_clus.br2(cds_sub_no_doubletons, c("TwdlT"), 
                   size_pos = 1, size_neg = 0.1, stroke = 0.02, limit = 3)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CAH6"), 
                   size_pos = 1, size_neg = 0.2, stroke = 0.02, limit = 3)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("dy"), size_pos = 1, : could not find function "plot.expr_clus.br2"


In [1587]:
saveRDS(df_sub300, "data/df_cell.type.3_reannotation.rds")

In [1087]:
df_sub300 = set.cell.type(df_sub300, "Tracheal_system_1", df_sub300$cluster %in% z)

Assigning cells for cell type Tracheal_system_1



In [146]:
clus = c(z)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'z' not found


In [1014]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "z" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
rdo83,z,rdo,2479,1147,7369.2398,50.271102,7.167226,92.08091
wdp83,z,wdp,498,1738,1502.3538,71.338587,4.376315,46.18750
Osi1583,z,Osi15,2805,19199,8206.9273,747.288410,3.455176,44.92697
Gasp83,z,Gasp,2714,18108,7960.3058,727.413577,3.449994,44.70775
stumps83,z,stumps,1092,5538,3235.5433,226.520022,3.829943,44.65804
CG583583,z,CG5835,245,722,743.7968,30.093228,4.580236,43.69867
apn83,z,apn,478,2319,1415.4810,88.213053,3.987894,41.74565
CG1178683,z,CG11786,186,582,569.8222,27.995683,4.296602,39.34354
sano83,z,sano,1441,9710,4116.7097,428.608211,3.260398,39.14964


In [147]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("rdo"), 
                   size_pos = 1, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("wdp"), 
                   size_pos = 1, size_neg = 0.1, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("stumps"), 
                   size_pos = 1, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("rdo"), size_pos = 1, : could not find function "plot.expr_clus.br2"


In [148]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("apn"), 
                   size_pos = 1, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("sano"), 
                   size_pos = 1, size_neg = 0.1, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG11786"), 
                   size_pos = 1, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("apn"), size_pos = 1, : could not find function "plot.expr_clus.br2"


In [1112]:
df_sub300 = set.cell.type(df_sub300, "large_intestine_hindgut_analpad", df_sub300$cluster %in% aa | 
                         df_sub300$cluster %in% c(109) &  
              df_sub300$umap_2 < 0.7*df_sub300$umap_1 -4 &
              df_sub300$umap_2 >= 0.5*df_sub300$umap_1 -3.8)

Assigning cells for cell type large_intestine_hindgut_analpad



In [149]:
clus = c(aa)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'aa' not found


In [150]:
clus = c(109)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1089]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "aa" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
TwdlX1,aa,TwdlX,1401,415,1986.60117,13.5208150,7.096036,77.74994
CG129201,aa,CG12920,1056,395,1942.94248,17.0922640,6.746726,73.70643
Cyp49a11,aa,Cyp49a1,1544,819,2722.95141,34.0529600,6.279491,71.65832
CG64091,aa,CG6409,1251,860,1986.82260,31.4038439,5.938154,65.06420
CG149491,aa,CG14949,232,44,341.61005,1.1380942,7.319881,61.63650
CG422961,aa,CG42296,800,915,1422.55436,37.6468204,5.201990,54.49231
CG325641,aa,CG32564,795,1494,1193.28700,43.7231524,4.737775,48.42923
CG61251,aa,CG6125,202,145,304.84822,4.2440423,5.861268,48.39457
NKCC1,aa,NKCC,258,202,389.71288,7.4734855,5.523312,47.55553


In [151]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("TwdlX"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG12920"), 
                   size_pos = 2, size_neg = 0.1, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Cyp49a1"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("TwdlX"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [152]:
clus = c(109)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
              df_sub300$umap_2 < 0.7*umap_1 -4 &
              df_sub300$umap_2 >= 0.5*umap_1 -3.8), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -4) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(1, 5) + ylim(-3, 1)
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1131]:
df_sub300 = set.cell.type(df_sub300, "Pharynx_2", df_sub300$cluster %in% ab &
                         df_sub300$umap_2 >= 0.7*df_sub300$umap_1 -6.3)

Assigning cells for cell type Pharynx_2



In [1132]:
df_sub300 = set.cell.type(df_sub300, "Pharynx", df_sub300$cluster %in% ab &
                         df_sub300$umap_2 < 0.7*df_sub300$umap_1 -6.3)

Assigning cells for cell type Pharynx



In [153]:
clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ab' not found


In [154]:
clus = c(113)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1116]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "ab" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
TwdlC2,ab,TwdlC,1284,866,3105.3762,28.978450,6.694698,77.66531
CG125232,ab,CG12523,826,414,1943.2159,13.532165,7.063053,77.16366
TwdlE2,ab,TwdlE,1509,1199,3559.5434,42.234625,6.363361,75.07418
CG21502,ab,CG2150,3308,3453,7091.4851,124.019957,5.825857,74.52481
CG130412,ab,CG13041,296,121,717.5571,3.944247,7.181199,68.14210
Cpr56F2,ab,Cpr56F,826,806,1955.1389,25.921788,6.182353,67.59657
CG130592,ab,CG13059,566,463,1466.4702,16.134325,6.419315,67.52551
Cpr62Bc2,ab,Cpr62Bc,774,805,1912.4850,27.177162,6.084778,66.33617
Cpr50Cb2,ab,Cpr50Cb,258,92,649.5951,3.890549,7.053401,65.91837


In [155]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("TwdlC"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("TwdlE"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Cpr56F"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("TwdlC"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [156]:
clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(3, 7) + ylim(-5, -1)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ab' not found


### ac cluster

In [1139]:
df_sub300 = set.cell.type(df_sub300, "Salivary_gland", df_sub300$cluster %in% ac)

Assigning cells for cell type Salivary_gland



In [157]:
clus = c(ac)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ac' not found


In [158]:
clus = c(113)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1136]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "ac" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
pip3,ac,pip,47027,6925,32792.0852,264.990019,6.945832,104.19514
nur3,ac,nur,26613,9706,19927.8068,355.439960,5.804980,82.91002
CG147563,ac,CG14756,14481,5763,10818.1132,201.054867,5.742558,76.95771
CG131593,ac,CG13159,6748,2596,4972.8202,104.269209,5.561909,68.30101
CG341903,ac,CG34190,914,134,618.1632,4.290539,6.868429,63.69902
PH4alphaSG13,ac,PH4alphaSG1,1719,504,1247.2033,17.378873,6.084505,62.58301
PH4alphaSG23,ac,PH4alphaSG2,1097,293,789.6215,9.326634,6.256719,60.23246
CrzR3,ac,CrzR,952,222,676.5130,9.290377,6.038750,56.78904
GlcAT-P3,ac,GlcAT-P,5727,4643,4109.0061,205.663831,4.313432,51.78242


In [159]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("pip"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("nur"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("PH4alphaSG1"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("pip"), size_pos = 2, : could not find function "plot.expr_clus.br2"


clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(3, 7) + ylim(-5, -1)
print(peedeef)

In [1143]:
df_sub300 = set.cell.type(df_sub300, "Analpad", df_sub300$cluster %in% ad)

Assigning cells for cell type Analpad



In [160]:
clus = c(ad)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ad' not found


In [161]:
clus = c(113)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1141]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "ad" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG177824,ad,CG17782,340,90,1323.3694,3.3047155,8.264082,85.70754
CG57684,ad,CG5768,211,33,809.5932,1.2391984,8.498071,82.11545
CG136154,ad,CG13615,191,27,737.4114,1.0560434,8.486455,80.86133
CG330034,ad,CG33003,632,745,2356.7963,34.0934208,6.069483,67.99777
CG177844,ad,CG17784,101,14,373.1821,0.5367777,7.923828,67.72968
CG344444,ad,CG34444,113,29,421.8502,1.0059735,7.716285,67.31689
CG67854,ad,CG6785,361,443,1392.6316,17.8752324,6.205175,64.81078
CG344424,ad,CG34442,110,37,428.0240,1.6051126,7.360202,64.36434
CG177804,ad,CG17780,305,342,1159.1576,15.0652428,6.172990,62.84169


In [162]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG17782"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG5768"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG13615"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("CG17782"), size_pos = 2, : could not find function "plot.expr_clus.br2"


clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(3, 7) + ylim(-5, -1)
print(peedeef)

In [1602]:
df_sub300_copy = df_sub300

In [1603]:
df_sub300 = set.cell.type(df_sub300, "Garland_cell_nephrocyte", df_sub300$cluster %in% ae &
                         df_sub300$umap_2 >= -0.1*df_sub300$umap_1 -0.7)  

Assigning cells for cell type Garland_cell_nephrocyte



In [1186]:
df_sub300 = set.cell.type(df_sub300, "Midgut_primordium", df_sub300$cluster %in% 107 &  
              df_sub300$umap_2 > -0.1*df_sub300$umap_1 -1.3 &
              df_sub300$umap_2 < -0.1*df_sub300$umap_1 -0.7)

Assigning cells for cell type Midgut_primordium



In [1200]:
df_sub300 = set.cell.type(df_sub300, "Copper_cells", df_sub300$cluster %in% 107 & 
                          df_sub300$umap_2 <= -0.1*df_sub300$umap_1 -1.3)

Assigning cells for cell type Copper_cells



In [163]:
clus = c(ae)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ae' not found


In [164]:
clus = c(166)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [165]:
clus = c( 107)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1151]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "ae" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Cubn5,ae,Cubn,634,843,1629.77914,35.53650,5.479194,58.47038
CG422555,ae,CG42255,266,278,684.74277,11.27696,5.801537,54.65932
Amnionless5,ae,Amnionless,156,288,391.58419,10.82043,5.049968,43.51486
CG343475,ae,CG34347,1911,10156,5039.30873,484.25921,3.376398,41.52732
HHEX5,ae,HHEX,140,495,373.62416,22.94760,3.963635,33.88631
wb5,ae,wb,1183,10438,3307.68454,465.56124,2.825683,33.03801
Bx5,ae,Bx,442,3359,1193.80577,149.31455,2.989512,30.56046
Ptx15,ae,Ptx1,434,3232,1149.58829,142.96691,2.997308,30.47710
grn5,ae,grn,1572,17793,4405.42469,788.64217,2.480010,30.02150


In [166]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Cubn"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG42255"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Amnionless"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Cubn"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [ ]:
clus = c(107)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
              df_sub300$umap_2 <= -0.1*umap_1 -1.3, #&
             # df_sub300$umap_2 < -0.1*umap_1 -0.7
              ), 
           size =  ifelse(df_sub300$cluster %in% clus, 0.5, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = -0.1, intercept = -1.3) +
geom_abline(slope = -0.1, intercept = -0.7) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(-4, 0) + ylim(-3, 1)
print(peedeef)

In [ ]:
clus = c(107)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= -0.1*umap_1 -0.7), 
           size =  ifelse(df_sub300$cluster %in% clus, 0.5, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = -0.1, intercept = -0.7) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(-4, 0) + ylim(-3, 1)
print(peedeef)

In [1170]:
two.set.differential.gene.test.3a(cds_sub, 
                                  df_sub300$cluster == 107 & 
                                  df_sub300$umap_2 >= -0.1*df_sub300$umap_1 -0.7 , 
                                 df_sub300$cluster == 166  
                                )  %>% 
head(20)
## these are all background genes so no real differences between this part of 107 and 166; call one cell type

# of cells sampled in set 1: 165

# of cells sampled in set 2: 371



,gene,set.1.n.umi,set.2.n.umi,set.1.tpm,set.2.tpm,higher.expr,log2.ratio,heuristic.score
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
Osi9,Osi9,0,59,0.0000,430.8151,Set 2,8.750925,76.60796
GlyP,GlyP,0,48,0.0000,339.6609,Set 2,8.407951,70.72931
Muc91C,Muc91C,0,49,0.0000,329.5218,Set 2,8.364230,69.99691
TwdlD,TwdlD,0,44,0.0000,328.2649,Set 2,8.358716,69.90482
Pgk,Pgk,0,50,0.0000,311.7071,Set 2,8.284047,68.66372
Vajk4,Vajk4,0,37,0.0000,289.8846,Set 2,8.179335,66.94216
CG12723,CG12723,0,38,0.0000,277.4163,Set 2,8.115909,65.91010
Prx2540-1,Prx2540-1,0,37,0.0000,263.1085,Set 2,8.039514,64.67779
Pdss2,Pdss2,0,38,0.0000,258.1015,Set 2,8.011795,64.23355


In [167]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Osi9"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("GlyP"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Pvf2"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Osi9"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [1206]:
df_sub300 = set.cell.type(df_sub300, "Bolwig_organ", df_sub300$cluster %in% af)

Assigning cells for cell type Bolwig_organ



In [168]:
clus = c(af)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'af' not found


In [169]:
clus = c(113)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1204]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "af" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Arr16,af,Arr1,24,8,577.94757,0.22720383,8.879420,81.48900
Gbeta76C6,af,Gbeta76C,21,9,521.41372,0.29412247,8.654311,78.14019
Rh66,af,Rh6,20,24,567.42528,0.62850855,8.444735,77.27631
ninaB6,af,ninaB,11,12,357.48074,0.37504383,8.022244,68.07477
trpl6,af,trpl,11,10,293.60030,0.60410093,7.515945,61.65040
LysD6,af,LysD,2,4,83.76612,0.15613995,6.178979,39.57893
CG134036,af,CG13403,3,26,111.98208,1.38586363,5.552613,37.86854
ninaA6,af,ninaA,3,5,76.31583,0.21632321,5.971384,37.45665
elfless6,af,elfless,2,3,71.30338,0.07666857,6.049324,37.36057


In [170]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Arr1"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Gbeta76C"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Rh6"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Arr1"), size_pos = 2, : could not find function "plot.expr_clus.br2"


clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(3, 7) + ylim(-5, -1)
print(peedeef)

In [1211]:
df_sub300 = set.cell.type(df_sub300, "lateral_cord_glia", df_sub300$cluster %in% ag)

Assigning cells for cell type lateral_cord_glia



In [171]:
clus = c(ag)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ag' not found


In [172]:
clus = c(113)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1208]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "ag" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
naz7,ag,naz,323,240,1938.9989,8.535027,7.667859,83.74713
CG404707,ag,CG40470,432,388,2515.9861,14.455873,7.346825,83.00062
Gat7,ag,Gat,328,471,1927.7464,18.637423,6.617166,72.21609
CG112417,ag,CG1124,133,312,777.3491,12.969334,5.798228,55.68776
zyd7,ag,zyd,308,1156,1780.0444,49.574808,5.137351,55.47572
CG316637,ag,CG31663,153,396,843.6520,15.985689,5.634256,54.77744
CG427417,ag,CG42741,393,1699,2248.2818,75.792200,4.871719,54.24781
Cyp4g157,ag,Cyp4g15,52,69,311.3520,2.613807,6.428883,53.27634
alrm7,ag,alrm,181,705,1030.1346,25.257338,5.293968,52.99271


In [173]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("naz"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG40470"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Gat"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("naz"), size_pos = 2, : could not find function "plot.expr_clus.br2"


clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(3, 7) + ylim(-5, -1)
print(peedeef)

In [1215]:
df_sub300 = set.cell.type(df_sub300, "Somatic_muscle", df_sub300$cluster %in% ah)

Assigning cells for cell type Somatic_muscle



In [174]:
clus = c(ah)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ah' not found


In [175]:
clus = c(113)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1213]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "ah" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
bt8,ah,bt,62060,32330,35937.1751,1342.014822,4.741930,71.76071
Mp208,ah,Mp20,7028,1021,2651.2943,36.565323,6.141152,69.84347
Zasp668,ah,Zasp66,3442,434,1621.1014,16.385339,6.542959,69.77182
Mf8,ah,Mf,14532,3325,6044.8988,128.154749,5.548546,69.69939
CG450788,ah,CG45078,3351,404,1392.4439,15.018880,6.441702,67.27997
sls8,ah,sls,35880,25607,21292.0738,925.880292,4.521790,65.01473
CG50238,ah,CG5023,1101,68,456.0565,2.330025,7.097536,62.71545
CG438978,ah,CG43897,9334,4324,5307.7064,163.281674,5.013845,62.04204
CG92978,ah,CG9297,2277,359,1068.6102,14.595883,6.098427,61.36767


In [176]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("bt"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Mp20"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Mf"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("bt"), size_pos = 2, : could not find function "plot.expr_clus.br2"


clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(3, 7) + ylim(-5, -1)
print(peedeef)

In [1222]:
df_sub300 = set.cell.type(df_sub300, "large_intestine_hindgut_analpad_2", df_sub300$cluster %in% ai)

Assigning cells for cell type large_intestine_hindgut_analpad_2



In [177]:
clus = c(ai)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ai' not found


In [178]:
clus = c(113)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1217]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "ai" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG177819,ai,CG17781,662,172,1549.72237,7.1383187,7.573063,80.26481
CG330039,ai,CG33003,962,649,2191.35950,28.1453376,6.232417,69.16904
CG130829,ai,CG13082,1653,2094,3855.55731,89.6252499,5.410883,64.46037
CG177809,ai,CG17780,409,330,952.78345,14.7350480,5.920095,58.59425
CG136169,ai,CG13616,170,83,398.11669,2.9610686,6.651158,57.47044
CG67859,ai,CG6785,449,451,997.81458,19.9419961,5.574301,55.54274
CG147209,ai,CG14720,137,62,310.64695,2.5760936,6.440748,53.35366
ine9,ai,ine,606,1002,1350.15923,41.7728514,4.980290,51.79493
otp9,ai,otp,1875,4849,4288.71618,222.8642733,4.259850,51.40218


In [179]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG17781"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG33003"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("ine"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("CG17781"), size_pos = 2, : could not find function "plot.expr_clus.br2"


clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(3, 7) + ylim(-5, -1)
print(peedeef)

In [1228]:
df_sub300 = set.cell.type(df_sub300, "Ventral_midline", df_sub300$cluster %in% aj)

Assigning cells for cell type Ventral_midline



In [180]:
clus = c(aj)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'aj' not found


In [181]:
clus = c(113)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1224]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "aj" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
wrapper10,aj,wrapper,1811,797,2624.7089,31.255734,6.346458,72.08619
sim10,aj,sim,1298,623,1799.1418,25.258318,6.098391,65.94735
Oatp26F10,aj,Oatp26F,1311,737,1839.9327,29.395739,5.919640,64.20573
tbc10,aj,tbc,840,476,1223.6856,20.034026,5.862364,60.13728
Ect310,aj,Ect3,1297,1301,1930.5687,52.079066,5.184739,56.59432
NetB10,aj,NetB,9778,19799,13787.3998,838.776003,4.037202,55.51624
Epac10,aj,Epac,1647,2254,2384.6223,90.432179,4.704915,52.78985
hbs10,aj,hbs,3840,8003,5332.0549,343.383250,3.952605,48.93620
CG3114510,aj,CG31145,5738,13541,7752.9059,631.753902,3.615021,46.70863


In [182]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("sim"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("rho"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("wrapper"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("sim"), size_pos = 2, : could not find function "plot.expr_clus.br2"


clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(3, 7) + ylim(-5, -1)
print(peedeef)

In [1271]:
#also same genes in lower part of 75/bc added to this cell type.
df_sub300 = set.cell.type(df_sub300, "Longitudinal_visceral_muscle", df_sub300$cluster %in% ak)

Assigning cells for cell type Longitudinal_visceral_muscle



In [183]:
clus = c(ak)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ak' not found


In [184]:
clus = c(bc)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bc' not found


In [1230]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "ak" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
tey11,ak,tey,1065,1710,6114.6290,71.99142,6.388394,80.35504
AdamTS-A11,ak,AdamTS-A,912,3531,5263.4964,158.82552,5.041452,62.32283
beat-IIb11,ak,beat-IIb,643,2318,3694.9865,106.34885,5.105190,60.50541
Syn211,ak,Syn2,166,429,973.9816,17.11328,5.748775,57.08091
CG1518611,ak,CG15186,169,590,979.2745,25.50536,5.207358,51.74573
OtopLa11,ak,OtopLa,974,7605,5450.5802,335.41754,4.018085,49.87431
Shaw11,ak,Shaw,202,926,1176.6363,40.63772,4.820634,49.17855
HLH54F11,ak,HLH54F,104,394,607.0405,16.11352,5.148585,47.61423
beat-IIa11,ak,beat-IIa,254,1656,1433.7905,73.57266,4.265044,44.72591


In [185]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("tey"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("AdamTS-A"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("beat-IIb"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("tey"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [186]:
clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(3, 7) + ylim(-5, -1)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ab' not found


### al cell type

In [1279]:
df_sub300 = set.cell.type(df_sub300, "Tracheal_system_older", df_sub300$cluster %in% al)

Assigning cells for cell type Tracheal_system_older



In [187]:
clus = c(al)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'al' not found


In [188]:
clus = c(bc)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bc' not found


In [1277]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "al" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Osi1512,al,Osi15,22898,13527,23154.0409,494.415436,5.546481,80.41865
Osi912,al,Osi9,10784,5385,11256.0215,199.378375,5.811826,78.21868
Osi2012,al,Osi20,9176,5191,9648.3967,180.703142,5.730634,75.85195
CG1272312,al,CG12723,3357,1306,3528.6900,39.895201,6.431057,75.79211
Osi1912,al,Osi19,5121,2634,5270.5818,88.219825,5.884454,72.75551
Osi1812,al,Osi18,4771,2927,4940.4450,104.274374,5.552415,68.13211
CG4274912,al,CG42749,1220,387,1354.3149,16.345314,6.286873,65.41122
CG3254812,al,CG32548,1580,740,1607.3770,23.476831,6.037148,64.30401
CG1200912,al,CG12009,5579,4749,5940.4636,175.985507,5.068872,63.54644


In [189]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Osi15"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Osi9"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Osi20"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Osi15"), size_pos = 2, : could not find function "plot.expr_clus.br2"


clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(3, 7) + ylim(-5, -1)
print(peedeef)

In [1284]:
df_sub300 = set.cell.type(df_sub300, "Olfactory_neurons", df_sub300$cluster %in% am)

Assigning cells for cell type Olfactory_neurons



In [ ]:
flybase says pdm3 "encodes a POU domain transcription factor that acts in odor receptor gene expression and axon targeting of olfactory neurons"

In [190]:
clus = c(am)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'am' not found


In [191]:
clus = c(bc)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bc' not found


In [1281]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "am" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
pdm313,am,pdm3,2526,1368,4691.1372,57.753709,6.319114,77.06810
CG1163513,am,CG11635,186,58,346.1046,2.234541,6.741503,56.89307
Drgx13,am,Drgx,473,592,872.6917,26.292682,4.998886,48.84402
fipi13,am,fipi,1147,2284,1994.3340,106.570141,4.212557,46.17980
CG4377813,am,CG43778,241,244,352.6458,9.573613,5.059680,42.83606
CG1167013,am,CG11670,115,109,205.2060,3.800027,5.417886,41.65240
CG3203213,am,CG32032,213,326,386.1673,13.007378,4.784967,41.13547
prc13,am,prc,589,2155,1127.2325,81.560291,3.771193,38.23933
zyd13,am,zyd,411,1135,760.0021,48.369396,3.944315,37.75402


In [192]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("pdm3"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Drgx"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("fipi"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("pdm3"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [193]:
clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(3, 7) + ylim(-5, -1)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ab' not found


In [1291]:
df_sub300 = set.cell.type(df_sub300, "Tracheal_system_2", df_sub300$cluster %in% an)

Assigning cells for cell type Tracheal_system_2



In [194]:
clus = c(an)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'an' not found


In [195]:
# 63 needs to be split into tracheal system with this and something else!
clus = c(63)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1286]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "an" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Gasp14,an,Gasp,6637,16886,10797.0949,676.031956,3.995275,53.53065
CG1200914,an,CG12009,2732,5432,4415.8814,210.039506,4.387116,53.12277
wat14,an,wat,1189,1810,1911.6972,77.414342,4.607593,50.22918
Osi1514,an,Osi15,5594,18190,8954.5910,704.462065,3.665987,48.12918
Cht514,an,Cht5,725,1060,1191.3066,45.882300,4.667358,47.69824
Spn43Aa14,an,Spn43Aa,2692,7049,4456.2354,302.007359,3.878401,47.01372
CG1327214,an,CG13272,653,1165,1042.3299,49.389527,4.370544,43.82336
CG1178614,an,CG11786,386,529,627.8403,26.457797,4.515110,41.97494
CG1367614,an,CG13676,664,2015,1107.1511,81.282257,3.750127,37.92856


In [196]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Gasp"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("wat"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Osi15"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Gasp"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [197]:
clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(3, 7) + ylim(-5, -1)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ab' not found


In [159]:
## already assigned as chordotonal organ by Shendure paper, but this is adult antenna organ so more likel
## some kind of touch neuron in larva, based on nompC and Tektin-C (microtubule associated)
#df_sub300 = set.cell.type(df_sub300, "ao", df_sub300$cluster %in% ao)

Assigning cells for cell type ao



In [198]:
clus = c(ao)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ao' not found


In [199]:
clus = c(bc)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bc' not found


In [1322]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "ao" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG3103615,ao,CG31036,276,254,1487.4286,9.9684644,7.083315,74.65512
nompC15,ao,nompC,192,135,1021.9035,5.0475182,7.400700,73.99556
CG4333815,ao,CG43338,135,122,716.5711,3.8692931,7.201254,68.31813
Tektin-C15,ao,Tektin-C,156,169,796.6092,5.8833210,6.854624,66.07541
CG949215,ao,CG9492,89,67,494.9308,2.3213741,7.219303,64.64160
CG4510515,ao,CG45105,224,459,1136.7793,16.4365020,6.026698,61.18307
Dhc98D15,ao,Dhc98D,49,9,259.0329,0.3557092,7.577944,60.79444
CG3186915,ao,CG31869,323,926,1765.0474,41.8134116,5.365500,57.87394
CG723615,ao,CG7236,83,106,433.4727,3.5942727,6.559961,57.48574


In [200]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG31036"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("nompC"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Tektin-C"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("CG31036"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [201]:
clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(3, 7) + ylim(-5, -1)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ab' not found


In [202]:
df_sub300 = set.cell.type(df_sub300, "Oenocytes", df_sub300$cluster %in% c(121) & 
                          df_sub300$umap_2 > 0.2*df_sub300$umap_1 -3.18 &
              df_sub300$umap_1 >= 2
                         )

ERROR: Error in set.cell.type(df_sub300, "Oenocytes", df_sub300$cluster %in% : could not find function "set.cell.type"


In [1387]:
df_sub300 = set.cell.type(df_sub300, "Analpad_large_intestine", df_sub300$cluster %in% c(121, 109) & 
                          (df_sub300$umap_2 <= 0.2*df_sub300$umap_1 -3.18 | df_sub300$umap_2 <= 0.6*df_sub300$umap_1 -4.12)&
              df_sub300$umap_2 >  0.2*df_sub300$umap_1 -3.4)

Assigning cells for cell type Analpad_large_intestine



In [1399]:
df_sub300 = set.cell.type(df_sub300, "large_intestine_hindgut_analpad", df_sub300$cluster %in% c( 109) & 
(df_sub300$umap_2 < 0.34*df_sub300$umap_1 -3.)&
              df_sub300$umap_2 >=  0.34*df_sub300$umap_1 -3.3)

Assigning cells for cell type large_intestine_hindgut_analpad



In [203]:
clus = c(ap)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ap' not found


In [204]:
clus = c(121)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [205]:
clus = c(109)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [ ]:
44. Gutierrez E, Wiggins D, Fielding B, Gould AP. Specialized hepatocyte-like cells regulate Drosophila lipid metabolism. Nature. 2007;445:275–80.
[ 44. Shows that larval oenocytes express numerous lipid metabolic genes including Cyp4g1, accumulate lipid droplets upon fasting, are regulated by the fat body, and are required for molting. ] 

In [1327]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "ap" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
bond16,ap,bond,685,434,1009.17178,13.977733,6.074209,60.62294
Tsp74F16,ap,Tsp74F,1076,1007,1547.08340,36.284416,5.374834,56.95318
FASN316,ap,FASN3,1025,1945,1444.83775,47.719576,4.890262,51.33646
CG215716,ap,CG2157,411,389,606.79636,13.531372,5.383970,49.78796
CG1321616,ap,CG13216,172,158,257.57742,5.995284,5.202480,41.69503
CG717916,ap,CG7179,192,215,255.02886,7.916008,4.838119,38.70574
LpR116,ap,LpR1,464,1000,623.33351,40.240412,3.917874,36.38205
CG1860916,ap,CG18609,80,61,104.17412,1.996308,5.119667,34.38694
Cyp4g117,ap,Cyp4g1,420,1961,567.44220,45.520371,3.608539,33.02127


In [206]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("LpR1"), 
                   size_pos = 2, size_neg = 0.9, stroke = 0.8, limit = 2)+
xlim(0, 4) + ylim(-5, -1)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Cyp4g1"), 
                   size_pos = 2, size_neg = 0.9, stroke = 0.8, limit = 2)+
xlim(0, 4) + ylim(-5, -1)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG17562"), 
                   size_pos = 2, size_neg = 0.9, stroke = 0.8, limit = 1)+
xlim(0, 4) + ylim(-5, -1)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("LpR1"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [207]:
clus = c(121)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.2, intercept = -3.18) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(0, 4) + ylim(-5, -1)
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [208]:
clus = c(121)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
              df_sub300$umap_2 > 0.2*umap_1 -3.18 &
              df_sub300$umap_1 >= 2), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.2, intercept = -3.18) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(0, 4) + ylim(-5, -1)
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [209]:
clus = c(121, 109)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
              (df_sub300$umap_2 <= 0.2*umap_1 -3.18 | df_sub300$umap_2 <= 0.6*umap_1 -4.12)&
              df_sub300$umap_2 >  0.2*umap_1 -3.4), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.8), stroke = 0.5) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.2, intercept = -3.18) +
geom_abline(slope = 0.2, intercept = -3.4) +
geom_abline(slope = 0.6, intercept = -4.12) +
xlim(0, 4) + ylim(-5, -1)
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1384]:
two.set.differential.gene.test.3a(cds_sub, 
                                  df_sub300$cluster %in% c(121, 109) & 
                                  (df_sub300$umap_2 <= 0.2*df_sub300$umap_1 -3.18 | 
                                   df_sub300$umap_2 <= 0.6*df_sub300$umap_1 -4.12) &
                                      df_sub300$umap_2 >  0.2*df_sub300$umap_1 -3.4, 
                                 !(df_sub300$cluster %in% c(121, 109) & 
                                  (df_sub300$umap_2 <= 0.2*df_sub300$umap_1 -3.18 | 
                                   df_sub300$umap_2 <= 0.6*df_sub300$umap_1 -4.12) &
                                      df_sub300$umap_2 >  0.2*df_sub300$umap_1 -3.4),
                                  sample = 50000
                                )  %>% filter(higher.expr == "Set 1")  %>% 
head(20)

# of cells sampled in set 1: 734

# of cells sampled in set 2: 50000



,gene,set.1.n.umi,set.2.n.umi,set.1.tpm,set.2.tpm,higher.expr,log2.ratio,heuristic.score
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
Tsp74F,Tsp74F,1208,958,4738.9502,35.696188,Set 1,7.012793,85.63081
bond,bond,683,424,2768.9956,15.689418,Set 1,7.374285,84.32988
CG2157,CG2157,445,444,1807.1751,14.975233,Set 1,6.821756,73.81357
CG7179,CG7179,203,202,757.0008,6.949523,Set 1,6.573283,62.88039
CG13216,CG13216,184,204,726.9638,8.578737,Set 1,6.245904,59.38433
CG13285,CG13285,99,116,382.8871,3.293653,Set 1,6.478570,55.61553
CG15905,CG15905,125,198,501.0075,8.290548,Set 1,5.752925,51.61274
Osi14,Osi14,1449,8386,5573.5923,349.057598,Set 1,3.992943,49.69078
swi2,swi2,74,89,270.3415,3.447563,Set 1,5.925624,47.90254


In [210]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("bond"), 
                   size_pos = 2, size_neg = 0.9, stroke = 0.8, limit = 2)+
xlim(0, 4) + ylim(-5, -1)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Tsp74F"), 
                   size_pos = 2, size_neg = 0.9, stroke = 0.8, limit = 2)+
xlim(0, 4) + ylim(-5, -1)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG2157"), 
                   size_pos = 2, size_neg = 0.9, stroke = 0.8, limit = 1)+
xlim(0, 4) + ylim(-5, -1)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("bond"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [211]:
clus = c( 109)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
              (df_sub300$umap_2 < 0.34*umap_1 -3.)&
              df_sub300$umap_2 >=  0.34*umap_1 -3.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.8), stroke = 0.5) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.34, intercept = -3) +
geom_abline(slope = 0.34, intercept = -3.32) +
geom_abline(slope = 0.6, intercept = -4.12) +
xlim(0, 4) + ylim(-5, -1)
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1412]:
df_sub300 = set.cell.type(df_sub300, "Posterior_spiracle", df_sub300$cluster %in% c(aq, 154)  &
                         df_sub300$umap_2 < 1*df_sub300$umap_1 -8.1)

Assigning cells for cell type Posterior_spiracle



In [212]:
clus = c(aq)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'aq' not found


In [213]:
clus = c(154)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1404]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "aq" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Cad96Cb17,aq,Cad96Cb,1109,638,2632.65884,30.3855886,6.390275,72.61175
CG1757217,aq,CG17572,310,82,738.16252,2.8421750,7.585871,72.29144
CG266317,aq,CG2663,1424,1961,3366.44804,75.1547758,5.466149,64.04927
CG3423217,aq,CG34232,599,617,1471.85722,27.2318689,5.704169,60.03297
nyo17,aq,nyo,1815,3918,4267.94688,174.7398252,4.602029,55.49893
CG1281417,aq,CG12814,1332,3709,3074.37678,162.5850432,4.232181,49.03637
CG894517,aq,CG8945,91,40,202.02313,1.6574789,6.248318,47.89649
CG4252417,aq,CG42524,785,1901,1911.83185,93.7679068,4.334413,47.25158
CG1364017,aq,CG13640,60,19,137.47710,0.5987852,6.426071,45.71188


In [214]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Cad96Cb"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG17572"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG2663"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Cad96Cb"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [215]:
clus = c(154, aq)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 < 1*umap_1 -8.1), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) #+
#xlim(3, 7) + ylim(-5, -1)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'aq' not found


In [1420]:
df_sub300 = set.cell.type(df_sub300, "Peripheral_nervous_system_early", df_sub300$cluster %in% ar)

Assigning cells for cell type Peripheral_nervous_system_early



In [216]:
clus = c(ar)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [217]:
clus = c(ar)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1417]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "ar" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
nrm18,ar,nrm,6997,4384,6261.00984,170.412278,5.190853,65.46917
meru18,ar,meru,2322,1190,2083.51115,47.513687,5.424481,59.80758
sv18,ar,sv,3195,2021,2757.70946,85.460695,4.995282,57.09497
ss18,ar,ss,1729,2018,1592.92704,88.495298,4.153725,44.18886
sens18,ar,sens,930,963,807.35037,41.475929,4.248477,41.03535
sca18,ar,sca,2172,4754,1857.82833,192.542677,3.262894,35.43560
pb18,ar,pb,762,1373,692.54164,58.970166,3.529584,33.31164
cpo18,ar,cpo,10042,38399,8992.99078,1629.298230,2.463665,32.35961
Rfx18,ar,Rfx,678,1448,615.29590,60.815720,3.315235,30.72387


In [218]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("nrm"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("meru"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("sv"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("nrm"), size_pos = 2, : could not find function "plot.expr_clus.br2"


clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(3, 7) + ylim(-5, -1)
print(peedeef)

### as cluster

In [1451]:
df_sub300 = set.cell.type(df_sub300, "Oenocytes_older", df_sub300$cluster == as & 
                                  df_sub300$umap_2 < -1.5*df_sub300$umap_1 + 9.5 &
                                  df_sub300$umap_2 <1*df_sub300$umap_1 -1.7)

Assigning cells for cell type Oenocytes_older



In [1452]:
df_sub300 = set.cell.type(df_sub300, "Proventriculus_older", df_sub300$cluster == as & 
                                  df_sub300$umap_2 < -1.5*df_sub300$umap_1 + 9.5 &
                                  df_sub300$umap_2 >= 1*df_sub300$umap_1 -1.7)

Assigning cells for cell type Proventriculus_older



In [219]:
# upper half maybe proventriculus while bottom could be more eonocytes Split and check markers
clus = c(as)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [220]:
clus = c(as)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  
           ifelse(df_sub300$cluster %in% clus, 1, 0.5), stroke = 0.1) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
xlim(3, 7) + ylim(1, 5)
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1423]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "as" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG1354519,as,CG13545,848,58,759.2038,1.5836748,8.198919,78.46564
Cyp4g120,as,Cyp4g1,2793,994,2604.7806,36.4730834,6.119163,69.43721
FASN319,as,FASN3,2868,975,2658.7175,44.8012765,5.859199,66.66044
CG720319,as,CG7203,210,42,259.3241,1.1934814,6.885390,55.24950
lectin-22C19,as,lectin-22C,260,6,170.2814,0.1301897,7.235212,53.68690
CG1072519,as,CG10725,176,56,270.8826,2.0809875,6.458131,52.22587
CG771419,as,CG7714,397,426,677.8968,14.5780107,5.443483,51.20711
CG1113119,as,CG11131,229,167,386.7453,5.3453349,5.929544,50.98794
CG1304419,as,CG13044,429,283,527.6020,9.6488677,5.630678,50.93533


In [221]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG13545"), 
                   size_pos = 3, size_neg = 0.6, stroke = 0.8, limit = 8)+
xlim(3, 7) + ylim(1, 5)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Cyp4g1"), 
                   size_pos = 3, size_neg = 0.6, stroke = 0.8, limit = 8)+
xlim(3, 7) + ylim(1, 5)
plot.expr_clus.br2(cds_sub_no_doubletons, c("FASN3"), 
                   size_pos = 3, size_neg = 0.6, stroke = 0.8, limit = 4)+
xlim(3, 7) + ylim(1, 5)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("CG13545"), size_pos = 3, : could not find function "plot.expr_clus.br2"


In [222]:
clus = c(as)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
              df_sub300$umap_2 < -1.5*umap_1 + 9.5 &
              df_sub300$umap_2 >= 1*umap_1 -1.7), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 1, intercept = -1.3) +
geom_abline(slope = 1, intercept = -1.7) +
geom_abline(slope = -1.5, intercept = 9.5) +
xlim(3, 7) + ylim(1, 5)
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [223]:
clus = c(as)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
              df_sub300$umap_2 < -1.5*umap_1 + 9.5 &
              df_sub300$umap_2 <1*umap_1 -1.7), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 1, intercept = -1.3) +
geom_abline(slope = 1, intercept = -1.7) +
geom_abline(slope = -1.5, intercept = 9.5) +
xlim(3, 7) + ylim(1, 5)
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1448]:
two.set.differential.gene.test.3a(cds_sub, 
                                  df_sub300$cluster == as & 
                                  df_sub300$umap_2 < -1.5*df_sub300$umap_1 + 9.5 &
                                  df_sub300$umap_2 <1*df_sub300$umap_1 -1.7, 
                                 !(df_sub300$cluster == as & 
                                  df_sub300$umap_2 < -1.5*df_sub300$umap_1 + 9.5 &
                                  df_sub300$umap_2 <1*df_sub300$umap_1 -1.7),
                                  sample = 50000
                                )  %>% filter(higher.expr == "Set 1")  %>% 
head(20)

# of cells sampled in set 1: 250

# of cells sampled in set 2: 50000



,gene,set.1.n.umi,set.2.n.umi,set.1.tpm,set.2.tpm,higher.expr,log2.ratio,heuristic.score
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
CG13545,CG13545,909,91,3030.8452,3.6098462,Set 1,9.360786,108.26667
Cyp4g1,Cyp4g1,2999,1009,10146.7767,39.3121732,Set 1,7.975590,106.14614
FASN3,FASN3,3078,960,10394.0117,43.1288400,Set 1,7.879815,105.14513
FASN2,FASN2,307,40,1002.9520,1.4522661,Set 1,8.675921,86.51173
lectin-22C,lectin-22C,262,6,650.5100,0.1924667,Set 1,9.091478,84.98390
CG15531,CG15531,368,75,1096.3000,2.3819667,Set 1,8.340564,84.23755
CG11131,CG11131,265,213,1920.9309,8.6290798,Set 1,7.640192,83.34182
CG13044,CG13044,461,342,2080.4287,10.3588155,Set 1,7.516925,82.86175
Cpr31A,Cpr31A,261,112,1345.8269,4.4168247,Set 1,7.956830,82.71402


In [1449]:
two.set.differential.gene.test.3a(cds_sub, 
                                  df_sub300$cluster == as & 
                                  df_sub300$umap_2 < -1.5*df_sub300$umap_1 + 9.5 &
                                  df_sub300$umap_2 >= 1*df_sub300$umap_1 -1.7, 
                                 !(df_sub300$cluster == as & 
                                  df_sub300$umap_2 < -1.5*df_sub300$umap_1 + 9.5 &
                                  df_sub300$umap_2 >= 1*df_sub300$umap_1 -1.7),
                                  sample = 50000
                                )  %>% filter(higher.expr == "Set 1")  %>% 
head(20)

# of cells sampled in set 1: 253

# of cells sampled in set 2: 50000



,gene,set.1.n.umi,set.2.n.umi,set.1.tpm,set.2.tpm,higher.expr,log2.ratio,heuristic.score
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
CG7714,CG7714,562,363,4099.1844,14.1259502,Set 1,8.082167,96.99791
CG12934,CG12934,274,108,1843.1714,4.3268001,Set 1,8.434705,91.50607
CG6933,CG6933,628,593,3958.4668,19.4028040,Set 1,7.600031,90.82865
CG10725,CG10725,208,62,1368.2242,2.2740037,Set 1,8.707033,90.71982
CG5084,CG5084,199,55,1196.3735,2.5135347,Set 1,8.411529,86.01341
CG7252,CG7252,75,30,500.7664,1.0052698,Set 1,7.964198,71.44580
obst-J,obst-J,116,100,825.5549,4.2324714,Set 1,7.301728,70.76080
CG10154,CG10154,50,16,386.3770,0.5090056,Set 1,8.000267,68.78305
Ms,Ms,110,28,463.5209,1.2332980,Set 1,7.697315,68.19513


In [224]:
## these have signal in 2 which is proventriculus and flybase says
### "embryonic midgut chamber; embryonic proventriculus; and embryonic/larval midgut"
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG7714"), 
                   size_pos = 3, size_neg = 0.6, stroke = 0.8, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG12934"), 
                   size_pos = 3, size_neg = 0.6, stroke = 0.8, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG6933"), 
                   size_pos = 3, size_neg = 0.6, stroke = 0.8, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("CG7714"), size_pos = 3, : could not find function "plot.expr_clus.br2"


In [1459]:
df_sub300 = set.cell.type(df_sub300, "Midgut_4th_chamber", df_sub300$cluster %in% at)  

Assigning cells for cell type Midgut_4th_chamber



In [225]:
clus = c(at)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'at' not found


In [226]:
clus = c(at)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'at' not found


In [1457]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "at" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG310622,at,CG3106,930,630,1512.8971,20.657465,6.126306,64.71861
Mal-A320,at,Mal-A3,582,548,952.4360,18.416347,5.616278,55.58426
CG3119820,at,CG31198,278,274,449.3827,8.904097,5.503775,48.51582
Eglp220,at,Eglp2,262,266,431.4238,9.134088,5.411818,47.38751
ham20,at,ham,1822,3956,2955.8674,172.354329,4.091786,47.17769
CG1349220,at,CG13492,356,478,594.2824,16.630801,5.074979,46.77826
CG483920,at,CG4839,281,250,469.4386,11.293737,5.254941,46.65264
CG956820,at,CG9568,281,308,471.4803,13.491342,5.023934,44.63319
CG1858520,at,CG18585,124,94,220.5966,3.690665,5.555475,43.28710


In [227]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG3106"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Mal-A3"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG31198"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("CG3106"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [228]:
clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(3, 7) + ylim(-5, -1)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ab' not found


In [1315]:
df_sub300 = set.cell.type(df_sub300, "Tracheal_system_2", df_sub300$cluster %in% au &
                         df_sub300$umap_2 < -0.1*df_sub300$umap_1 -4.4)

Assigning cells for cell type Tracheal_system_2



In [1318]:
## could extend farther to right
df_sub300 = set.cell.type(df_sub300, "Maxillary_sensory_complex", df_sub300$cluster %in% au &
                         df_sub300$umap_2 >= -0.1*df_sub300$umap_1 -4.4 &
                         df_sub300$umap_2 < -0.1*df_sub300$umap_1 -4.1 )

Assigning cells for cell type Maxillary_sensory_complex



In [229]:
clus = c(au)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'au' not found


In [230]:
clus = c(63)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1294]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "au" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG730021,au,CG7300,601,538,1335.55703,19.643682,6.015597,62.46780
twz21,au,twz,254,664,544.11068,27.139776,4.273217,38.84528
Dfd21,au,Dfd,219,632,451.98694,24.202394,4.164648,36.74605
CG1336221,au,CG13362,744,3603,1652.33852,152.067612,3.432268,36.69495
CG1710821,au,CG17108,39,29,98.38083,1.187024,5.491336,36.43444
TfAP-221,au,TfAP-2,134,335,284.12911,13.168154,4.325823,35.27913
CG729621,au,CG7296,434,2883,1004.98267,116.195679,3.100179,30.92240
Spn43Aa21,au,Spn43Aa,1024,7726,2286.70824,338.882299,2.750165,30.69098
dsx-c73A21,au,dsx-c73A,1106,8072,2389.67108,363.966652,2.710976,30.42582


In [231]:
# These ar all from upper branch, not tracheal system branch which may connect with epidermis
# flybase says CG7300 "is expressed in embryonic maxillary sensory complex." and Sue agrees.
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG7300"), 
                   size_pos = 2, size_neg = 0.8, stroke = 0.08, limit = 8)+
xlim(-2, 2 ) + ylim(-6, -2)
plot.expr_clus.br2(cds_sub_no_doubletons, c("twz"), 
                   size_pos = 2, size_neg = 0.8, stroke = 0.2, limit = 8)+
xlim(-2, 2 ) + ylim(-6, -2)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Dfd"), 
                   size_pos = 2, size_neg = 0.8, stroke = 0.2, limit = 4)+
xlim(-2, 2 ) + ylim(-6, -2)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("CG7300"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [232]:
clus = c(au)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 < -0.1*umap_1 -4.4), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.8), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = -0.1, intercept = -4.1) +
geom_abline(slope = -0.1, intercept = -4.4) +
xlim(-2, 2 ) + ylim(-6, -2)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'au' not found


In [233]:
clus = c(au)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
              df_sub300$umap_2 < -0.1*umap_1 -4.1 &
              df_sub300$umap_2 >= -0.1*umap_1 -4.4), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.8), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = -0.1, intercept = -4.1) +
geom_abline(slope = -0.1, intercept = -4.4) +
xlim(-2, 2 ) + ylim(-6, -2)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'au' not found


In [1463]:
df_sub300 = set.cell.type(df_sub300, "Germ_cells_ovaries_query", df_sub300$cluster %in% av)

Assigning cells for cell type Germ_cells_ovaries_query



In [234]:
clus = c(av)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'av' not found


In [235]:
clus = c(av)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'av' not found


In [1461]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "av" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
dhd22,av,dhd,1176,590,1425.65676,23.052980,5.889267,61.71023
yl22,av,yl,245,139,303.76097,5.191471,5.616510,46.34483
Pxt22,av,Pxt,245,138,296.54945,5.202930,5.579179,45.84403
alphaTub67C22,av,alphaTub67C,197,105,249.75207,4.226743,5.578441,44.46083
Cyp6a1922,av,Cyp6a19,118,32,140.02111,1.350599,5.896472,42.09944
fs(1)Ya22,av,fs(1)Ya,119,67,143.91818,2.499293,5.362041,38.49460
CG720822,av,CG7208,88,41,109.07732,1.454002,5.474071,37.12719
Jabba22,av,Jabba,233,225,244.94891,8.689748,4.659878,37.00975
mtrm22,av,mtrm,205,237,254.67878,9.493299,4.601139,36.80077


In [236]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("dhd"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("yl"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Pxt"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("dhd"), size_pos = 2, : could not find function "plot.expr_clus.br2"


clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(3, 7) + ylim(-5, -1)
print(peedeef)

In [167]:
# already assigned with clster 107 part as nephrocyte query -- should be Garland cell nephrocyte
# df_sub300 = set.cell.type(df_sub300, "aw", df_sub300$cluster %in% aw)

Assigning cells for cell type aw



In [237]:
clus = c(aw)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'aw' not found


In [238]:
clus = c(aw)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'aw' not found


In [1465]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "aw" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG4225523,aw,CG42255,254,243,2602.44085,1.097149e+01,7.764119,88.09327
Amnionless23,aw,Amnionless,245,265,2465.74382,1.007212e+01,7.798948,87.88161
Cubn23,aw,Cubn,451,1002,4443.21343,4.333554e+01,6.646996,80.54639
CG1520923,aw,CG15209,29,275,297.63488,1.134813e+01,4.591179,37.74977
CG1559923,aw,CG15599,10,34,109.51322,1.474417e+00,5.467873,37.11633
eIF4E623,aw,eIF4E6,7,0,50.77040,0.000000e+00,5.665916,32.26204
Klf1523,aw,Klf15,5,2,51.76960,4.129939e-02,5.635648,32.24512
CG911723,aw,CG9117,6,22,62.84368,1.025332e+00,4.955537,29.71574
CG460723,aw,CG4607,5,11,45.73831,2.512329e-01,5.191981,28.79750


In [239]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG42255"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Amnionless"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Cubn"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("CG42255"), size_pos = 2, : could not find function "plot.expr_clus.br2"


clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(3, 7) + ylim(-5, -1)
print(peedeef)

In [1195]:
df_sub300 = set.cell.type(df_sub300, "Copper_cells", df_sub300$cluster %in% ax)

Assigning cells for cell type Copper_cells



In [240]:
clus = c(ax)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ax' not found


clus = c(113)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

In [1191]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "ax" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Ptx124,ax,Ptx1,4491,2232,3311.0703,94.006046,5.123135,59.90746
HHEX24,ax,HHEX,942,336,723.9475,15.219785,5.480058,52.07005
5-HT724,ax,5-HT7,2123,1126,1497.3070,53.128577,4.789836,50.52854
lab24,ax,lab,2838,2318,2035.5743,104.666432,4.267847,46.91187
dve24,ax,dve,2524,3176,1834.8667,137.924853,3.723299,40.36892
kcc24,ax,kcc,2112,2379,1410.7723,101.076906,3.788757,39.64287
fa2h24,ax,fa2h,882,821,662.1973,35.256650,4.190944,39.28295
CG3434724,ax,CG34347,5626,9291,4085.3241,439.022408,3.214802,38.56665
rgr24,ax,rgr,943,1064,660.9783,44.802330,3.851110,36.08737


In [241]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Ptx1"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("HHEX"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("5-HT7"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Ptx1"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [242]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("dve"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("kcc"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("fa2h"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("dve"), size_pos = 2, : could not find function "plot.expr_clus.br2"


clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(3, 7) + ylim(-5, -1)
print(peedeef)

In [1472]:
df_sub300 = set.cell.type(df_sub300, "Muscle_system", df_sub300$cluster %in% ay)

Assigning cells for cell type Muscle_system



In [243]:
clus = c(ay)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ay' not found


In [244]:
clus = c(ay)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ay' not found


In [1470]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "ay" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
up25,ay,up,22358,25378,13769.3911,982.45115,3.807468,52.34994
bt25,ay,bt,28283,41951,16887.2548,1395.97036,3.595562,50.49510
Sh25,ay,Sh,8966,8593,5475.7467,373.11780,3.871491,48.08045
wupA25,ay,wupA,13792,18894,8349.8766,704.90658,3.564206,46.43345
Mhc25,ay,Mhc,22946,47636,13896.7641,1556.24510,3.157681,43.45779
CG3352125,ay,CG33521,2446,2291,1491.0209,91.90377,4.004420,42.21880
Nlg125,ay,Nlg1,930,631,564.4749,26.27962,4.371014,39.96557
Neto25,ay,Neto,1302,1094,801.4888,47.08634,4.058983,39.16244
CG167445,ay,CG1674,1746,2079,1065.0881,75.70511,3.795506,38.17562


In [245]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("up"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("bt"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Sh"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("up"), size_pos = 2, : could not find function "plot.expr_clus.br2"


clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(3, 7) + ylim(-5, -1)
print(peedeef)

In [1477]:
df_sub300 = set.cell.type(df_sub300, "Central_brain_ventral_nerve_cord_primorium", df_sub300$cluster %in% az)

Assigning cells for cell type Central_brain_ventral_nerve_cord_primorium



In [246]:
clus = c(az)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'az' not found


In [247]:
clus = c(az)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'az' not found


In [1473]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "az" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
cas26,az,cas,6276,4634,3616.4622,193.18429,4.219081,49.87275
jigr126,az,jigr1,3737,4019,2109.8951,157.73247,3.732502,41.22041
nab26,az,nab,1002,764,626.5084,29.41010,4.364711,40.56340
klu26,az,klu,10109,15169,5795.1566,624.97172,3.210678,40.13631
E2f126,az,E2f1,9740,19125,5419.8189,740.67100,2.869393,35.59280
ktub26,az,ktub,1103,1077,602.1780,41.65751,3.819318,35.27690
Sp126,az,Sp1,2483,3837,1392.8530,161.27798,3.101504,32.39478
Pex726,az,Pex7,500,476,321.8340,21.07623,3.865751,32.21967
pros26,az,pros,2908,6103,1699.4675,221.06116,2.936054,31.50889


In [248]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("cas"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("jigr1"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("nab"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("cas"), size_pos = 2, : could not find function "plot.expr_clus.br2"


clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(3, 7) + ylim(-5, -1)
print(peedeef)

In [1484]:
df_sub300 = set.cell.type(df_sub300, "Digestive_system", df_sub300$cluster %in% ba)

Assigning cells for cell type Digestive_system



In [249]:
clus = c(ba)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ba' not found


In [250]:
clus = c(ba)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ba' not found


In [1479]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "ba" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Lim328,ba,Lim3,1891,2733,2978.4440,119.979833,4.621720,53.33848
CG456228,ba,CG4562,622,711,959.1802,26.773968,5.109997,50.62556
Jheh328,ba,Jheh3,971,1860,1572.7622,80.372302,4.272619,45.37522
CG473428,ba,CG4734,153,149,263.8954,6.117015,5.212550,41.95727
CG1534728,ba,CG15347,1347,3348,2160.6806,157.565114,3.768339,41.74542
CG3399328,ba,CG33993,1506,4527,2408.0650,204.686708,3.549354,39.87436
grn28,ba,grn,4268,17145,6592.5322,753.401411,3.127428,39.67717
CG1536528,ba,CG15365,917,2851,1441.2191,134.747304,3.408294,35.76689
Lfg28,ba,Lfg,502,1467,800.1733,65.912156,3.579972,34.53231


In [251]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Lim3"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG4562"), # Mrp2
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Jheh3"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Lim3"), size_pos = 2, : could not find function "plot.expr_clus.br2"


### cell.type.3 counts

In [1784]:
df_sub300  %>% count(cell.type.3)

cell.type.3,n
<chr>,<int>
Amnioserosa,7091
Analpad,850
Analpad_large_intestine,734
Antennal_sense_organs,748
Bolwig_organ,115
Central_brain_ventral_nerve_cord_1,6846
Central_brain_ventral_nerve_cord_2,10161
Central_brain_ventral_nerve_cord_early,5058
Central_brain_ventral_nerve_cord_primorium,5819


clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(3, 7) + ylim(-5, -1)
print(peedeef)

In [1505]:
df_sub300 = set.cell.type(df_sub300, "Visceral_muscle", df_sub300$cluster %in% bb)

Assigning cells for cell type Visceral_muscle



In [252]:
clus = c(bb)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bb' not found


In [253]:
clus = c(18,46)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1485]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "bb" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG4442229,bb,CG44422,1744,3467,3507.6104,149.341023,4.544178,53.51535
disco-r29,bb,disco-r,2741,7077,5473.9731,305.776910,4.157327,51.62833
CG3325329,bb,CG33253,298,488,598.5538,20.605880,4.791985,44.21922
side-IV29,bb,side-IV,1335,4172,2685.4678,185.489971,3.848003,43.83451
bru329,bb,bru3,7363,41873,14429.4171,1850.352999,2.962361,40.93043
CG482929,bb,CG4829,1074,3934,2155.1859,171.532618,3.642871,40.34213
NetA29,bb,NetA,1683,7445,3400.1073,327.692188,3.370771,39.54518
sqa29,bb,sqa,498,1575,973.8925,63.322972,3.920357,38.92561
CG3132329,bb,CG31323,232,499,449.5786,20.165243,4.408804,38.86641


In [254]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("side-IV"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("disco-r"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("bru3"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 10)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("side-IV"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [255]:
clus = c(18)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.7), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -3.3) +
geom_abline(slope = 1, intercept = -3.3) +
geom_abline(slope = -1., intercept = -6.7) +
 xlim(0, 4) + ylim(-3, 1)
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


### bc cluster

In [173]:
df_sub300 = set.cell.type(df_sub300, "bc", df_sub300$cluster %in% bc)

Assigning cells for cell type bc



In [1669]:
df_sub300 = set.cell.type(df_sub300, "Longitudinal_visceral_muscle", df_sub300$cluster %in% bc &
                          df_sub300$umap_2 < -0.1*df_sub300$umap_1 -0.1
                         )

Assigning cells for cell type Visceral_mesoderm



In [1673]:
df_sub300 = set.cell.type(df_sub300, "Visceral_mesoderm", df_sub300$cluster %in% bc &
                          df_sub300$umap_2 >= -0.1*df_sub300$umap_1 -0.1
                         )
# based on Kah expression  doi: 10.1242/dev.199465 

Assigning cells for cell type Visceral_mesoderm



In [256]:
clus = c(bc)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bc' not found


In [257]:
clus = c(113)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1241]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "bc" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
tey30,bc,tey,854,1707,1979.00447,71.1894839,4.776842,52.31258
sns30,bc,sns,1716,5586,4245.56777,237.1779838,4.155846,50.08659
HLH54F30,bc,HLH54F,228,343,534.43234,13.2222349,5.231787,47.42385
Kah30,bc,Kah,486,2153,1242.70110,89.1600192,3.784848,38.90984
CG3437730,bc,CG34377,151,379,360.51816,15.2306256,4.473281,38.01361
beat-IIb30,bc,beat-IIb,602,2393,1368.69188,109.0783068,3.636196,37.88783
NK7.130,bc,NK7.1,1408,8264,3497.71352,375.6909445,3.214959,37.84845
gol30,bc,gol,517,2626,1255.99837,113.6723025,3.453246,35.55381
beat-IIa30,bc,beat-IIa,379,1656,891.28705,72.1371877,3.607213,35.35561


In [258]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("tey"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("sns"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("HLH54F"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("tey"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [259]:
clus = c(75, 69)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 < -0.1*umap_1 -0.1), 
           size =  ifelse(df_sub300$cluster %in% clus, 0.3, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = -0.1, intercept = -.1) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(-5, -1) + ylim(-2, 2)
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1270]:
two.set.differential.gene.test.3a(cds_sub, 
                                  df_sub300$cluster == 75 & 
                                  df_sub300$umap_2 < -0.1*df_sub300$umap_1 -0.1 , 
                                 !(df_sub300$cluster == 75 & 
                                  df_sub300$umap_2 < -0.1*df_sub300$umap_1 -0.1),
                                  sample = 50000
                                )  %>% filter(higher.expr == "Set 1")  %>% 
head(20)

# of cells sampled in set 1: 579

# of cells sampled in set 2: 50000



,gene,set.1.n.umi,set.2.n.umi,set.1.tpm,set.2.tpm,higher.expr,log2.ratio,heuristic.score
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
tey,tey,967,1836,4611.02394,78.52727,Set 1,5.857494,71.29264
HLH54F,HLH54F,251,336,1223.38930,13.18234,Set 1,6.430644,65.96456
beat-IIb,beat-IIb,641,2319,3062.79663,105.41408,Set 1,4.847089,56.13464
beat-IIa,beat-IIa,423,1642,2054.21747,72.15760,Set 1,4.811437,52.95023
bru3,bru3,3545,41521,16857.46794,1860.61444,Set 1,3.178762,44.63358
tok,tok,234,1776,1139.77013,78.64160,Set 1,3.839077,38.98887
H2.0,H2.0,87,543,409.42302,22.36169,Set 1,4.131375,35.86434
ATP8B,ATP8B,134,1051,644.78794,47.51049,Set 1,3.732456,34.84217
RapGAP1,RapGAP1,2368,44286,11055.15800,1997.87862,Set 1,2.467457,33.14427


[1] 20

In [1663]:
# do the other part of 75 (bc)
two.set.differential.gene.test.3a(cds_sub, 
                                  df_sub300$cluster == 75 & 
                                  df_sub300$umap_2 >= -0.1*df_sub300$umap_1 -0.1 , 
                                 !(df_sub300$cluster == 75 & 
                                  df_sub300$umap_2 >= -0.1*df_sub300$umap_1 -0.1),
                                  sample = 50000
                                )  %>% filter(higher.expr == "Set 1")  %>% 
head(20)

# of cells sampled in set 1: 916

# of cells sampled in set 2: 50000



,gene,set.1.n.umi,set.2.n.umi,set.1.tpm,set.2.tpm,higher.expr,log2.ratio,heuristic.score
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
sns,sns,1882,5610,6312.26540,240.7767416,Set 1,4.706410,59.41453
NK7.1,NK7.1,1562,8219,5208.74440,378.4342898,Set 1,3.779014,46.65947
CG34377,CG34377,169,394,547.81018,14.8422673,Set 1,5.111825,46.51845
Kah,Kah,502,2284,1727.39389,97.2727810,Set 1,4.135661,44.47993
gol,gol,561,2645,1850.24954,113.1379020,Set 1,4.018870,43.62196
Vrp1,Vrp1,789,5064,2640.68498,199.4421657,Set 1,3.719654,42.28221
MCU,MCU,386,1974,1334.45571,85.1868301,Set 1,3.952640,41.04072
lmd,lmd,502,3225,1692.77080,140.8134240,Set 1,3.577321,38.37042
HEATR2,HEATR2,171,774,562.66661,31.7782450,Set 1,4.101470,37.48210


In [260]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("sns"), 
                   size_pos = 2, size_neg = 0.5, stroke = 0.008, limit = 2)
plot.expr_clus.br2(cds_sub_no_doubletons, c("gol"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 2)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Kah"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("sns"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [1535]:
df_sub300 = set.cell.type(df_sub300, "unknown_3", df_sub300$cluster %in% bd &
                         df_sub300$umap_2 < 0*df_sub300$umap_1 + 2.1 )

Assigning cells for cell type unknown_3



In [261]:
clus = c(bd)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bd' not found


In [262]:
clus = c(bd)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bd' not found


In [1521]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "bd" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Desi31,bd,Desi,95,68,266.87731,2.539683e+00,6.236413,50.29934
CG1372731,bd,CG13727,54,36,137.35042,1.072172e+00,6.050574,43.03279
CG1331031,bd,CG13310,20,3,56.93845,6.838945e-02,5.735894,33.59197
atk31,bd,atk,30,96,105.84229,4.012126e+00,4.400350,29.65545
CG3204031,bd,CG32040,27,55,82.46591,2.383886e+00,4.607045,29.40730
CG4507231,bd,CG45072,12,4,34.62260,2.417838e-01,4.801228,24.74899
pAbp31,bd,pAbp,2969,111562,10308.58419,3.401275e+03,1.599275,21.32105
CG3446131,bd,CG34461,106,1339,262.29102,4.705980e+01,2.448266,19.68532
Ccp84Ab31,bd,Ccp84Ab,17,135,54.60077,4.264440e+00,3.374569,19.56249


In [263]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Desi"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG13727"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG32040"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Desi"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [264]:
clus = c(133)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 < 0*umap_1 + 2.1), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0, intercept = 2.1) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(4, 8) + ylim(1, 5)
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1534]:
## no strong differences but keep them separate by name
two.set.differential.gene.test.3a(cds_sub, 
                                  df_sub300$cluster == 133 & 
                                   df_sub300$umap_2 < 0*df_sub300$umap_1 + 2.1 , 
                                 !(df_sub300$cluster == 18 & 
                                   df_sub300$umap_2 < 0*df_sub300$umap_1 + 2.1),
                                  sample = 50000
                                )  %>% filter(higher.expr == "Set 1")  %>% 
head(20)

# of cells sampled in set 1: 180

# of cells sampled in set 2: 50000



,gene,set.1.n.umi,set.2.n.umi,set.1.tpm,set.2.tpm,higher.expr,log2.ratio,heuristic.score
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
Desi,Desi,118,109,1462.15337,4.5872114,Set 1,8.031751,84.45277
CG13727,CG13727,64,56,713.38143,2.4560272,Set 1,7.689415,72.89989
CG13310,CG13310,23,12,262.28598,0.3391824,Set 1,7.613644,61.21741
CG32040,CG32040,32,73,430.07555,3.2541828,Set 1,6.659564,58.28315
CG45072,CG45072,14,28,177.20362,1.2591345,Set 1,6.293494,47.05886
atk,atk,18,121,275.12744,4.8699808,Set 1,5.550600,45.01088
CG13386,CG13386,13,13,116.89719,0.5098887,Set 1,6.274654,43.17831
Ptth,Ptth,3,20,60.60606,0.9442888,Set 1,4.962148,29.49997
CG3301,CG3301,3,42,64.85084,1.4013055,Set 1,4.755234,28.72699


In [1540]:
df_sub300 = set.cell.type(df_sub300, "Somatic_muscle_primordium", df_sub300$cluster %in% be)

Assigning cells for cell type Somatic_muscle_primordium



In [265]:
clus = c(be)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'be' not found


In [266]:
clus = c(be)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'be' not found


In [1536]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "be" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
lmd32,be,lmd,2521,2688,1915.68285,109.8841438,4.110734,44.82507
Kah32,be,Kah,1015,2002,734.89371,82.0770682,3.145013,29.95107
sns32,be,sns,2329,5668,1696.66298,246.0002192,2.780116,29.82879
Rim232,be,Rim2,1671,4736,1201.20687,178.2374103,2.744541,28.08069
Gyc32E32,be,Gyc32E,306,487,218.19646,17.8360264,3.534061,27.48114
kirre32,be,kirre,4464,15028,3243.79946,690.6964813,2.229474,26.00439
sing32,be,sing,201,289,145.87586,11.7929025,3.511326,25.27611
Mef232,be,Mef2,3141,12961,2278.45077,504.5078466,2.172248,24.23027
Or69a32,be,Or69a,48,22,37.67684,0.7583816,4.421358,23.31558


In [267]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("lmd"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Kah"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("sns"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("lmd"), size_pos = 2, : could not find function "plot.expr_clus.br2"


clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(3, 7) + ylim(-5, -1)
print(peedeef)

In [1545]:
df_sub300 = set.cell.type(df_sub300, "Midgut", df_sub300$cluster %in% bf)

Assigning cells for cell type Midgut



In [268]:
clus = c(bf)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bf' not found


In [269]:
clus = c(bf)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bf' not found


In [1542]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "bf" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ham33,bf,ham,1272,4031,2670.4435,180.00901,3.882946,44.20114
CG3399333,bf,CG33993,1174,4459,2451.1117,201.33156,3.598643,40.52003
FucTC33,bf,FucTC,220,587,456.7397,23.89620,4.197375,37.09801
vnd33,bf,vnd,328,1035,663.1536,42.55709,3.928364,36.82987
grn33,bf,grn,2636,17294,5582.3982,757.87278,2.878955,35.83415
sens-233,bf,sens-2,379,1720,799.3552,75.55306,3.384305,32.63992
CG1534733,bf,CG15347,634,3388,1299.8011,159.46232,3.017984,31.22161
bru233,bf,bru2,1776,13742,3696.2302,633.97719,2.541278,30.11981
CG430133,bf,CG4301,197,781,404.9602,37.03425,3.412409,29.56919


In [270]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("tey"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("AdamTS-A"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("beat-IIb"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("tey"), size_pos = 2, : could not find function "plot.expr_clus.br2"


clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(3, 7) + ylim(-5, -1)
print(peedeef)

In [1551]:
df_sub300 = set.cell.type(df_sub300, "Late_cells", df_sub300$cluster %in% bg)
## top hits connected to pharynx but only at very tip

Assigning cells for cell type Late_cells



In [271]:
clus = c(bg)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bg' not found


In [272]:
clus = c(bg)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bg' not found


In [1548]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "bg" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Ccp84Ac34,bg,Ccp84Ac,475,1028,972.66793,38.9462062,4.605817,45.72326
CG754834,bg,CG7548,94,110,192.96764,3.3584222,5.468409,41.55812
Cpr56F34,bg,Cpr56F,329,960,646.56628,31.5248357,4.313185,40.28033
Cpr92F34,bg,Cpr92F,113,164,237.90139,6.3026669,5.025796,39.70516
CG3446134,bg,CG34461,328,1240,682.72649,44.2191448,3.916302,36.88090
CG507034,bg,CG5070,88,145,172.43531,4.7561169,4.904816,36.48326
Cpr97Ea34,bg,Cpr97Ea,86,156,167.73288,5.1451985,4.770562,35.29547
CG926934,bg,CG9269,72,107,141.84387,3.7619523,4.896607,35.05136
TwdlC34,bg,TwdlC,319,1186,584.75983,43.3012171,3.722426,34.22460


In [273]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Ccp84Ac"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG7548"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Cpr56F"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Ccp84Ac"), size_pos = 2, : could not find function "plot.expr_clus.br2"


clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(3, 7) + ylim(-5, -1)
print(peedeef)

In [1573]:
df_sub300 = set.cell.type(df_sub300, "Pharynx_primordium_query", df_sub300$cluster %in% bh &
                         df_sub300$umap_2 >= -.65*df_sub300$umap_1 -4.4)

Assigning cells for cell type Pharynx_primordium_query



In [1577]:
df_sub300 = set.cell.type(df_sub300, "Tracheal_system_early", df_sub300$cluster %in% bh &
        df_sub300$umap_2 < -.65*df_sub300$umap_1 -4.4)

Assigning cells for cell type Tracheal_system_early



In [274]:
clus = c(bh)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bh' not found


In [275]:
clus = c(72, 57)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1555]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "bh" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
scro35,bh,scro,744,1378,862.6576,60.78208,3.803528,37.10081
btl35,bh,btl,450,937,505.3887,37.92671,3.698561,33.22825
knrl35,bh,knrl,1603,5043,1722.9310,211.93745,3.016363,32.43039
CG3244735,bh,CG32447,923,2460,1002.0158,111.02878,3.160964,31.51522
CG4274135,bh,CG42741,563,1708,648.4632,75.03016,3.092381,28.89244
trh35,bh,trh,916,4081,995.5577,163.61051,2.596448,25.86273
ds35,bh,ds,2748,16270,3066.7078,683.64488,2.163263,25.05696
dpy35,bh,dpy,4564,28379,4978.7536,1209.65845,2.039993,25.05490
Vsx135,bh,Vsx1,370,1426,422.2196,59.81775,2.795429,24.39085


In [276]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("scro"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("btl"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("knrl"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("scro"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [277]:
clus = c(bh)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= -.65*umap_1 -4.4), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -5) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -0.65, intercept = -4.4) +
xlim(-4, 0) + ylim(-5, -1)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bh' not found


In [1570]:
two.set.differential.gene.test.3a(cds_sub, 
                                  df_sub300$cluster %in% bh & 
                                  df_sub300$umap_2 >= -.65*df_sub300$umap_1 -4.4 , 
                                 !(df_sub300$cluster  %in% bh & 
                                  df_sub300$umap_2 >= -.65*df_sub300$umap_1 -4.4),
                                  sample = 50000
                                )  %>% filter(higher.expr == "Set 1")  %>% 
head(20)

# of cells sampled in set 1: 1371

# of cells sampled in set 2: 50000



,gene,set.1.n.umi,set.2.n.umi,set.1.tpm,set.2.tpm,higher.expr,log2.ratio,heuristic.score
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
scro,scro,783,1437,1698.2191,62.68135,Set 1,4.737008,50.83120
CG42741,CG42741,606,1711,1316.4031,77.60279,Set 1,4.065877,42.13664
Vsx1,Vsx1,401,1508,865.0783,61.65082,Set 1,3.787425,36.95904
Ser,Ser,1145,6547,2411.8339,289.66732,Set 1,3.052690,34.30159
sr,sr,504,3868,1049.1706,169.86971,Set 1,2.618281,26.27813
ds,ds,1559,16288,3303.1970,701.84804,Set 1,2.232578,26.09903
CG42747,CG42747,1132,10810,2445.5860,501.24278,Set 1,2.283723,25.70686
CG13300,CG13300,936,9532,1986.4469,411.59973,Set 1,2.267375,24.84295
Fas3,Fas3,3793,50129,8052.7295,2235.61297,Set 1,1.848162,23.98072


In [278]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("scro"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG42741"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Vsx1"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("scro"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [279]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Blimp-1"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Blimp-1"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [1574]:
two.set.differential.gene.test.3a(cds_sub, 
                                  df_sub300$cluster %in% bh & 
                                  df_sub300$umap_2 < -.65*df_sub300$umap_1 -4.4 , 
                                 !(df_sub300$cluster  %in% bh & 
                                  df_sub300$umap_2 < -.65*df_sub300$umap_1 -4.4),
                                  sample = 50000
                                )  %>% filter(higher.expr == "Set 1")  %>% 
head(20)

# of cells sampled in set 1: 1797

# of cells sampled in set 2: 50000



,gene,set.1.n.umi,set.2.n.umi,set.1.tpm,set.2.tpm,higher.expr,log2.ratio,heuristic.score
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
btl,btl,528,954,842.55653,38.76600,Set 1,4.405166,42.81971
knrl,knrl,1633,5095,2512.32097,219.67868,Set 1,3.509002,39.63550
CG32447,CG32447,908,2600,1405.95938,120.72412,Set 1,3.529868,36.91665
trh,trh,1004,3895,1553.82627,154.67893,Set 1,3.319180,35.19173
Hs6st,Hs6st,2439,12907,3797.48259,598.95041,Set 1,2.662128,31.65592
stumps,stumps,1157,5616,1808.79742,239.54276,Set 1,2.910666,31.49810
dpy,dpy,4206,27964,6513.30715,1214.66647,Set 1,2.421643,30.68075
Reck,Reck,329,1122,490.36081,48.32521,Set 1,3.313447,29.62433
Pvf1,Pvf1,378,1521,618.78967,69.56386,Set 1,3.132448,29.05544


In [280]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("btl"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("knrl"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("trh"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("btl"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [1585]:
df_sub300 = set.cell.type(df_sub300, "Fat_body", df_sub300$cluster %in% bi)  

Assigning cells for cell type Fat_body



In [281]:
clus = c(bi)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bi' not found


In [282]:
clus = c(bi)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bi' not found


In [1583]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "bi" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG722736,bi,CG7227,565,2622,1864.2348,126.64160,3.868413,42.03086
Idgf236,bi,Idgf2,401,1888,1345.2300,84.99897,3.967390,41.23986
CG273636,bi,CG2736,198,680,663.5002,32.25217,4.318576,40.49152
apolpp36,bi,apolpp,6182,78013,19901.0719,2830.11553,2.813404,40.17718
NimB236,bi,NimB2,318,1533,1000.3929,68.40678,3.849346,38.36949
Gbp236,bi,Gbp2,132,664,438.7670,29.49265,3.846922,33.77826
Apoltp36,bi,Apoltp,391,3532,1308.3100,139.51681,3.218890,33.33029
Idgf436,bi,Idgf4,737,7690,2408.4207,308.87167,2.958345,33.23544
Tep436,bi,Tep4,391,3192,1325.0141,142.78726,3.204000,33.23471


In [283]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG7227"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Idgf2"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("NimB2"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("CG7227"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [284]:
clus = c(ab)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= 0.7*umap_1 -6.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -6.3) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -1., intercept = -6.7) +
xlim(3, 7) + ylim(-5, -1)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ab' not found


In [1617]:
df_sub300 = set.cell.type(df_sub300, "Central_brain_ventral_nerve_cord_early", df_sub300$cluster %in% bj)

Assigning cells for cell type Central_brain_ventral_nerve_cord_early



In [285]:
clus = c(bj)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bj' not found


In [286]:
clus = c(bj)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bj' not found


In [1612]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "bj" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Sep537,bj,Sep5,594,364,280.20712,11.69696,4.463938,36.31632
kuk37,bj,kuk,3424,4888,1587.27462,162.53511,3.278879,34.86513
CG1003537,bj,CG10035,1630,1594,764.50987,63.12637,3.575545,34.25472
CG1562837,bj,CG15628,1193,1197,531.53144,44.94570,3.532153,31.98973
dan37,bj,dan,738,616,342.95420,23.69375,3.795798,31.98367
Sr-CII37,bj,Sr-CII,700,535,343.03010,23.80041,3.789899,31.93517
CycB337,bj,CycB3,2333,3815,1098.37610,122.98774,3.147103,31.79351
Top237,bj,Top2,2767,4448,1262.64231,150.62495,3.057867,31.50634
borr37,bj,borr,1337,1974,639.46164,64.35291,3.290534,30.67755


In [287]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Sep5"), 
                   size_pos = 2, size_neg = 0.5, stroke = 0.08, limit = 2)
plot.expr_clus.br2(cds_sub_no_doubletons, c("kuk"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 2)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG10035"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 2)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Sep5"), size_pos = 2, : could not find function "plot.expr_clus.br2"


clus = c(bh)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= -.65*umap_1 -4.4), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -5) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -0.65, intercept = -4.4) +
xlim(-4, 0) + ylim(-5, -1)
print(peedeef)

In [1627]:
df_sub300 = set.cell.type(df_sub300, "Central_brain_ventral_nerve_cord_1", df_sub300$cluster %in% bk)

Assigning cells for cell type Central_brain_ventral_nerve_cord_1



In [288]:
clus = c(bk)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bk' not found


In [289]:
clus = c(bk)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bk' not found


In [1620]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "bk" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Df3138,bk,Df31,17794,23008,2804.4980,816.34329,1.778729,20.37363
stg48,bk,stg,8867,11266,1509.6917,427.30331,1.817549,19.19513
CG4223238,bk,CG42232,2999,2907,480.2010,107.32405,2.148285,19.14228
fax38,bk,fax,41605,73599,7110.4074,2601.53893,1.450012,18.55424
akirin38,bk,akirin,10507,14399,1717.0873,525.70268,1.704903,18.32189
dap38,bk,dap,4165,4629,688.4098,180.27197,1.925112,18.15230
RhoGEF438,bk,RhoGEF4,1222,1146,212.0748,42.83461,2.274431,17.59321
CycB48,bk,CycB,5319,6891,852.7267,254.10946,1.740967,16.95289
Mapmodulin38,bk,Mapmodulin,4285,5446,718.2133,208.11045,1.780147,16.89409


In [290]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Df31"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("stg"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG42232"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Df31"), size_pos = 2, : could not find function "plot.expr_clus.br2"


clus = c(bh)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= -.65*umap_1 -4.4), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -5) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -0.65, intercept = -4.4) +
xlim(-4, 0) + ylim(-5, -1)
print(peedeef)

In [1628]:
df_sub300 = set.cell.type(df_sub300, "Central_brain_ventral_nerve_cord_2", df_sub300$cluster %in% bl)

Assigning cells for cell type Central_brain_ventral_nerve_cord_2



In [291]:
clus = c(bl)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bl' not found


In [292]:
clus = c(72, 57)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1625]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "bl" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Brd40,bl,Brd,1380,766,216.0523,33.27688,2.656073,20.61617
Tom42,bl,Tom,3864,3910,696.4962,154.34260,2.164662,20.44749
HmgD39,bl,HmgD,8396,11087,1500.5803,396.69402,1.915790,20.21592
RhoGEF439,bl,RhoGEF4,1225,1217,238.8639,40.43466,2.527279,19.98085
polo39,bl,polo,1751,2043,348.0008,73.08635,2.231811,18.85230
CycB339,bl,CycB3,2974,3769,551.2583,131.36374,2.058221,18.74874
CycB49,bl,CycB,4709,7051,924.7275,248.33623,1.890936,18.63412
eEF1alpha139,bl,eEF1alpha1,92257,214249,18176.4765,7328.17816,1.310349,18.54126
borr39,bl,borr,1633,1849,307.8084,65.13914,2.218456,18.34789


In [293]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Brd"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Tom"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("HmgD"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Brd"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [294]:
clus = c(bh)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= -.65*umap_1 -4.4), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -5) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -0.65, intercept = -4.4) +
xlim(-4, 0) + ylim(-5, -1)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bh' not found


In [1632]:
df_sub300 = set.cell.type(df_sub300, "Undifferentiated_1", df_sub300$cluster %in% bm)

Assigning cells for cell type Undifferentiated_1



In [295]:
clus = c(bm)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bm' not found


In [296]:
clus = c(bm)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bm' not found


In [1629]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "bm" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
eEF240,bm,eEF2,15933,82767,8084.7580,2731.34968,1.565062,20.31634
betaTub60D40,bm,betaTub60D,3426,14535,1723.4942,515.49430,1.738512,18.69241
eEF1alpha140,bm,eEF1alpha1,37214,223504,18278.3932,7565.11548,1.272515,18.01617
Hsc70-440,bm,Hsc70-4,9909,55407,4957.9620,1859.00434,1.414441,17.36343
Non140,bm,Non1,4092,21259,1997.7208,709.18214,1.492094,16.36060
RpL450,bm,RpL4,7051,40709,3499.0902,1348.69270,1.374349,16.18045
eIF3c40,bm,eIF3c,1468,7093,750.8986,231.45128,1.691690,16.16307
RpL7A40,bm,RpL7A,10554,64427,5159.1336,2085.48360,1.306055,16.10783
eIF4A40,bm,eIF4A,7323,44396,3589.2292,1455.22444,1.301441,15.36984


In [297]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("scro"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("btl"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("knrl"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("scro"), size_pos = 2, : could not find function "plot.expr_clus.br2"


clus = c(bh)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= -.65*umap_1 -4.4), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -5) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -0.65, intercept = -4.4) +
xlim(-4, 0) + ylim(-5, -1)
print(peedeef)

In [1636]:
df_sub300 = set.cell.type(df_sub300, "Undifferentiated_2", df_sub300$cluster %in% bn)

Assigning cells for cell type Undifferentiated_2



In [298]:
clus = c(bn)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bn' not found


In [299]:
clus = c(bn)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bn' not found


In [1634]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "bn" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG517141,bn,CG5171,365,1497,342.5247,52.52668,2.677878,22.55917
eEF1alpha141,bn,eEF1alpha1,18811,227912,19434.8958,7606.12842,1.353226,19.27864
c12.141,bn,c12.1,163,1197,191.7397,39.95890,2.226900,16.90331
RpS641,bn,RpS6,4502,52001,4445.3439,1717.44532,1.371191,16.61665
RpS3A41,bn,RpS3A,3224,37862,3324.1579,1247.76134,1.412491,16.52503
Act42A41,bn,Act42A,1981,19016,1815.7351,635.21926,1.512958,16.38099
Hsp2341,bn,Hsp23,165,1497,224.0413,51.67445,2.088589,16.32033
betaTub60D41,bn,betaTub60D,1733,15134,1537.9638,528.47768,1.538380,16.28797
RpL741,bn,RpL7,2547,29001,2521.2205,955.60025,1.398134,15.79959


In [300]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("scro"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("btl"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("knrl"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("scro"), size_pos = 2, : could not find function "plot.expr_clus.br2"


clus = c(bh)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= -.65*umap_1 -4.4), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -5) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -0.65, intercept = -4.4) +
xlim(-4, 0) + ylim(-5, -1)
print(peedeef)

In [1639]:
df_sub300 = set.cell.type(df_sub300, "Undifferentiated_3", df_sub300$cluster %in% bo)

Assigning cells for cell type Undifferentiated_3



In [301]:
clus = c(bo)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bo' not found


In [302]:
clus = c(72, 57)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1638]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "bo" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
mt:CoI42,bo,mt:CoI,3939,21895,3410.85985,808.452709,2.075117,24.35428
RpL1942,bo,RpL19,5101,33700,3874.61540,1079.976481,1.841718,21.95366
RpL542,bo,RpL5,4693,31777,3792.33919,1102.369352,1.781172,21.17681
RpL1342,bo,RpL13,3897,25275,3118.73949,880.648630,1.822687,21.15631
eEF542,bo,eEF5,1780,10597,1442.71511,373.415021,1.946076,20.42518
RpL1042,bo,RpL10,3343,21503,2643.35711,763.993723,1.788852,20.33692
mt:ND442,bo,mt:ND4,904,4988,831.17402,193.966310,2.091926,20.29323
yl42,bo,yl,88,199,79.95737,7.849171,3.175617,20.13052
RpS2442,bo,RpS24,2767,18183,2210.37212,629.417643,1.809909,20.10941


In [303]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("scro"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("btl"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("knrl"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("scro"), size_pos = 2, : could not find function "plot.expr_clus.br2"


clus = c(bh)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 >= -.65*umap_1 -4.4), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -5) +
geom_abline(slope = 0.5, intercept = -3.8) +
geom_abline(slope = -0.65, intercept = -4.4) +
xlim(-4, 0) + ylim(-5, -1)
print(peedeef)

In [1698]:
df_sub300 = set.cell.type(df_sub300, "Pharynx_primordium_query", df_sub300$cluster %in% bp &
                         df_sub300$umap_2 < -.5*df_sub300$umap_1 -4)

Assigning cells for cell type Pharynx_primordium_query



In [1705]:
df_sub300 = set.cell.type(df_sub300, "Epidermis", df_sub300$cluster %in% bp &
df_sub300$umap_2 < -0.4*df_sub300$umap_1 -3.4 &
                                  df_sub300$umap_2 >= -.5*df_sub300$umap_1 -4)

Assigning cells for cell type Epidermis



In [304]:
clus = c(bp)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bp' not found


In [305]:
clus = c(bp)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bp' not found


In [1642]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "bp" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG1059143,bp,CG10591,14608,21471,6951.2411,850.50420,3.029185,38.66228
CG918443,bp,CG9184,20829,33318,9956.5393,1334.39999,2.898372,38.49495
sha43,bp,sha,2251,2523,1089.5775,106.16884,3.345811,33.76217
bbg43,bp,bbg,8374,15453,3870.8853,675.32178,2.516882,29.99826
CG1411043,bp,CG14110,2919,4963,1420.1206,202.07180,2.805952,29.38621
tyn43,bp,tyn,14960,32414,6966.1264,1427.61175,2.285743,29.18059
ImpE243,bp,ImpE2,6694,14367,3173.4271,559.89954,2.500227,29.08335
haf43,bp,haf,4511,8632,2137.3825,366.27735,2.540903,28.10825
CG3044043,bp,CG30440,2371,3631,1111.8606,163.98659,2.752555,27.85602


In [306]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG10591"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("sha"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("bbg"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("CG10591"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [ ]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG10591"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("sha"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("bbg"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

In [307]:
clus = c(bp)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
              df_sub300$umap_2 < -0.4*umap_1 -3.4 &
              df_sub300$umap_2 >= -.5*umap_1 -4), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -5) +
geom_abline(slope = -0.5, intercept = -4.0) +
geom_abline(slope = -0.4, intercept = -3.4)  +
xlim(-3, 2.5) + ylim(-6, -2)
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bp' not found


In [1696]:
two.set.differential.gene.test.3a(cds_sub, 
                                  df_sub300$cluster == bp & 
                                  df_sub300$umap_2 < -.5*df_sub300$umap_1 -4 , 
                                 !(df_sub300$cluster == bp & 
                                  df_sub300$umap_2 < -.5*df_sub300$umap_1 -4),
                                  sample = 50000
                                )  %>% filter(higher.expr == "Set 1")  %>% 
head(20)

# of cells sampled in set 1: 124

# of cells sampled in set 2: 50000



,gene,set.1.n.umi,set.2.n.umi,set.1.tpm,set.2.tpm,higher.expr,log2.ratio,heuristic.score
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
CG10348,CG10348,64,1213,1597.02130,51.5013888,Set 1,4.926884,52.43225
rn,rn,19,524,558.36504,24.3560693,Set 1,4.460806,40.71666
mt:ND4L,mt:ND4L,4,64,133.85089,2.2568238,Set 1,5.361017,37.93038
CG34398,CG34398,54,2413,1370.22387,111.5270181,Set 1,3.606068,37.57973
CG32073,CG32073,5,6,70.24840,0.3090386,Set 1,5.745886,35.36470
Sox21b,Sox21b,33,1514,815.11339,66.4012628,Set 1,3.596153,34.78424
CG13675,CG13675,5,122,146.51375,4.6136895,Set 1,4.705943,33.90493
CG13643,CG13643,10,240,209.05299,9.7319550,Set 1,4.283884,33.04849
sano,sano,118,9864,2908.10442,444.0639745,Set 1,2.707994,31.15916


In [308]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG10348"), 
                   size_pos = 2, size_neg = 0.5, stroke = 0.008, limit = 2)
plot.expr_clus.br2(cds_sub_no_doubletons, c("rn"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 2)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG34398"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("CG10348"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [1702]:
two.set.differential.gene.test.3a(cds_sub, 
                                  df_sub300$cluster == bp & 
                                  df_sub300$umap_2 < -0.4*df_sub300$umap_1 -3.4 &
                                  df_sub300$umap_2 >= -.5*df_sub300$umap_1 -4 , 
                                 !(df_sub300$cluster == bp & 
                                   df_sub300$umap_2 < -0.4*df_sub300$umap_1 -3.4 &
                                  df_sub300$umap_2 >= -.5*df_sub300$umap_1 -4),
                                  sample = 50000
                                )  %>% filter(higher.expr == "Set 1")  %>% 
head(20)

# of cells sampled in set 1: 767

# of cells sampled in set 2: 50000



,gene,set.1.n.umi,set.2.n.umi,set.1.tpm,set.2.tpm,higher.expr,log2.ratio,heuristic.score
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
bbg,bbg,1312,16165,4936.7152,743.141753,Set 1,2.729902,33.49488
CG9184,CG9184,2294,37041,8868.3756,1601.805215,Set 1,2.468071,32.36780
CG30440,CG30440,416,4143,1584.8964,195.834456,Set 1,3.009334,31.99248
sha,sha,304,2856,1169.7647,131.930527,Set 1,3.137474,31.98101
haf,haf,734,9203,2853.5429,414.269083,Set 1,2.780636,31.91904
tyn,tyn,2274,34448,8633.8552,1589.563878,Set 1,2.440467,31.91143
CG10591,CG10591,1541,24411,5941.8520,1049.633881,Set 1,2.499653,31.33799
ab,ab,1903,31425,7426.4713,1387.779482,Set 1,2.418859,31.10328
dsx-c73A,dsx-c73A,653,7932,2404.2975,359.945884,Set 1,2.735761,30.72807


In [309]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("bbg"), 
                   size_pos = 2, size_neg = 0.5, stroke = 0.008, limit = 12)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG9184"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 12)
plot.expr_clus.br2(cds_sub_no_doubletons, c("sha"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("bbg"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [ ]:
two.set.differential.gene.test.3a(cds_sub, 
                                  df_sub300$cluster == bp & 
                                  df_sub300$umap_2 >= -0.4*df_sub300$umap_1 -3.4 , 
                                 !(df_sub300$cluster == bp & 
                                   df_sub300$umap_2 < -0.4*df_sub300$umap_1 -3.4 &
                                  df_sub300$umap_2 >= -.5*df_sub300$umap_1 -4),
                                  sample = 50000
                                )  %>% filter(higher.expr == "Set 1")  %>% 
head(20)

In [1660]:
df_sub300 = set.cell.type(df_sub300, "Embryonic_body_wall", df_sub300$cluster %in% c(bq, br,bs, bt, bu, bv, bw, bx, by))

Assigning cells for cell type Embryonic_body_wall



In [1644]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == 
                                              "bq" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Act57B44,bq,Act57B,24201,49032,6408.3207,1439.01846,2.153859,27.23761
CG296244,bq,CG2962,14862,28955,3852.2991,1015.30233,1.922390,22.89928
Mlc244,bq,Mlc2,12441,28458,3299.5489,866.05437,1.928074,22.53628
eEF1alpha144,bq,eEF1alpha1,76454,218993,20543.3019,7206.71742,1.511054,21.64804
CG1573144,bq,CG15731,2958,5182,782.8430,168.63752,2.206268,21.21199
RpL23A44,bq,RpL23A,23945,63966,6321.6670,2065.07794,1.613410,20.37143
TwdlF44,bq,TwdlF,6697,13997,1776.8570,492.98094,1.846800,19.93791
RpL644,bq,RpL6,22539,62854,5951.1989,1986.58711,1.582162,19.83906
RpS644,bq,RpS6,18692,50047,4918.5066,1630.96020,1.591614,19.52003


In [1655]:
two.set.differential.gene.test.3a(cds_sub, 
                                  df_sub300$cluster %in% c(bq, br,bs, bt, bu, bv, bw, bx, by) , 
                                 !(df_sub300$cluster %in% c(bq, br,bs, bt, bu, bv, bw, bx, by)),
                                  sample = 50000
                                )  %>% filter(higher.expr == "Set 1")  %>% 
head(20)

# of cells sampled in set 1: 42392

# of cells sampled in set 2: 50000



,gene,set.1.n.umi,set.2.n.umi,set.1.tpm,set.2.tpm,higher.expr,log2.ratio,heuristic.score
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
Act57B,Act57B,148265,28363,5202.0650,877.41574,Set 1,2.566109,31.67899
CG2962,CG2962,99075,12169,3508.8653,602.85115,Set 1,2.538740,29.89924
CG15731,CG15731,19656,1894,689.3785,86.86101,Set 1,2.972001,28.02967
Mlc2,Mlc2,79663,17297,2743.0415,561.59088,Set 1,2.285618,26.10653
TwdlF,TwdlF,45244,6161,1582.2296,312.78750,Set 1,2.334099,24.80833
Ipod,Ipod,33599,4423,1166.9931,224.21123,Set 1,2.373445,24.18497
Neurochondrin,Neurochondrin,15657,2254,545.3709,93.77742,Set 1,2.524622,22.95825
CG9083,CG9083,35113,5337,1244.7225,271.68212,Set 1,2.190532,22.52473
CG15740,CG15740,10511,1200,366.7385,58.21386,Set 1,2.630745,22.42062


In [310]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Act57B"), 
                   size_pos = 2, size_neg = 0.5, stroke = 0.008, limit = 18)
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG15731"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Ipod"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Act57B"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [311]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Neurochondrin"), 
                   size_pos = 2, size_neg = 0.5, stroke = 0.008, limit = 18)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Vajk4"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Gabat"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 2)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Neurochondrin"), : could not find function "plot.expr_clus.br2"


In [188]:
df_sub300 = set.cell.type(df_sub300, "br", df_sub300$cluster %in% br)

Assigning cells for cell type br



In [1645]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == 
                                              "bq" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Act57B44,bq,Act57B,24201,49032,6408.3207,1439.01846,2.153859,27.23761
CG296244,bq,CG2962,14862,28955,3852.2991,1015.30233,1.922390,22.89928
Mlc244,bq,Mlc2,12441,28458,3299.5489,866.05437,1.928074,22.53628
eEF1alpha144,bq,eEF1alpha1,76454,218993,20543.3019,7206.71742,1.511054,21.64804
CG1573144,bq,CG15731,2958,5182,782.8430,168.63752,2.206268,21.21199
RpL23A44,bq,RpL23A,23945,63966,6321.6670,2065.07794,1.613410,20.37143
TwdlF44,bq,TwdlF,6697,13997,1776.8570,492.98094,1.846800,19.93791
RpL644,bq,RpL6,22539,62854,5951.1989,1986.58711,1.582162,19.83906
RpS644,bq,RpS6,18692,50047,4918.5066,1630.96020,1.591614,19.52003


In [189]:
df_sub300 = set.cell.type(df_sub300, "bs", df_sub300$cluster %in% bs)

Assigning cells for cell type bs



In [1646]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == 
                                              "bq" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Act57B44,bq,Act57B,24201,49032,6408.3207,1439.01846,2.153859,27.23761
CG296244,bq,CG2962,14862,28955,3852.2991,1015.30233,1.922390,22.89928
Mlc244,bq,Mlc2,12441,28458,3299.5489,866.05437,1.928074,22.53628
eEF1alpha144,bq,eEF1alpha1,76454,218993,20543.3019,7206.71742,1.511054,21.64804
CG1573144,bq,CG15731,2958,5182,782.8430,168.63752,2.206268,21.21199
RpL23A44,bq,RpL23A,23945,63966,6321.6670,2065.07794,1.613410,20.37143
TwdlF44,bq,TwdlF,6697,13997,1776.8570,492.98094,1.846800,19.93791
RpL644,bq,RpL6,22539,62854,5951.1989,1986.58711,1.582162,19.83906
RpS644,bq,RpS6,18692,50047,4918.5066,1630.96020,1.591614,19.52003


In [190]:
df_sub300 = set.cell.type(df_sub300, "bt", df_sub300$cluster %in% bt)

Assigning cells for cell type bt



In [1647]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == 
                                              "bq" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Act57B44,bq,Act57B,24201,49032,6408.3207,1439.01846,2.153859,27.23761
CG296244,bq,CG2962,14862,28955,3852.2991,1015.30233,1.922390,22.89928
Mlc244,bq,Mlc2,12441,28458,3299.5489,866.05437,1.928074,22.53628
eEF1alpha144,bq,eEF1alpha1,76454,218993,20543.3019,7206.71742,1.511054,21.64804
CG1573144,bq,CG15731,2958,5182,782.8430,168.63752,2.206268,21.21199
RpL23A44,bq,RpL23A,23945,63966,6321.6670,2065.07794,1.613410,20.37143
TwdlF44,bq,TwdlF,6697,13997,1776.8570,492.98094,1.846800,19.93791
RpL644,bq,RpL6,22539,62854,5951.1989,1986.58711,1.582162,19.83906
RpS644,bq,RpS6,18692,50047,4918.5066,1630.96020,1.591614,19.52003


In [191]:
df_sub300 = set.cell.type(df_sub300, "bu", df_sub300$cluster %in% bu)

Assigning cells for cell type bu



In [1648]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == 
                                              "bq" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Act57B44,bq,Act57B,24201,49032,6408.3207,1439.01846,2.153859,27.23761
CG296244,bq,CG2962,14862,28955,3852.2991,1015.30233,1.922390,22.89928
Mlc244,bq,Mlc2,12441,28458,3299.5489,866.05437,1.928074,22.53628
eEF1alpha144,bq,eEF1alpha1,76454,218993,20543.3019,7206.71742,1.511054,21.64804
CG1573144,bq,CG15731,2958,5182,782.8430,168.63752,2.206268,21.21199
RpL23A44,bq,RpL23A,23945,63966,6321.6670,2065.07794,1.613410,20.37143
TwdlF44,bq,TwdlF,6697,13997,1776.8570,492.98094,1.846800,19.93791
RpL644,bq,RpL6,22539,62854,5951.1989,1986.58711,1.582162,19.83906
RpS644,bq,RpS6,18692,50047,4918.5066,1630.96020,1.591614,19.52003


In [192]:
df_sub300 = set.cell.type(df_sub300, "bv", df_sub300$cluster %in% bv)

Assigning cells for cell type bv



In [1649]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == 
                                              "bq" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Act57B44,bq,Act57B,24201,49032,6408.3207,1439.01846,2.153859,27.23761
CG296244,bq,CG2962,14862,28955,3852.2991,1015.30233,1.922390,22.89928
Mlc244,bq,Mlc2,12441,28458,3299.5489,866.05437,1.928074,22.53628
eEF1alpha144,bq,eEF1alpha1,76454,218993,20543.3019,7206.71742,1.511054,21.64804
CG1573144,bq,CG15731,2958,5182,782.8430,168.63752,2.206268,21.21199
RpL23A44,bq,RpL23A,23945,63966,6321.6670,2065.07794,1.613410,20.37143
TwdlF44,bq,TwdlF,6697,13997,1776.8570,492.98094,1.846800,19.93791
RpL644,bq,RpL6,22539,62854,5951.1989,1986.58711,1.582162,19.83906
RpS644,bq,RpS6,18692,50047,4918.5066,1630.96020,1.591614,19.52003


In [193]:
df_sub300 = set.cell.type(df_sub300, "bw", df_sub300$cluster %in% bw)

Assigning cells for cell type bw



In [1650]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == 
                                              "bq" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Act57B44,bq,Act57B,24201,49032,6408.3207,1439.01846,2.153859,27.23761
CG296244,bq,CG2962,14862,28955,3852.2991,1015.30233,1.922390,22.89928
Mlc244,bq,Mlc2,12441,28458,3299.5489,866.05437,1.928074,22.53628
eEF1alpha144,bq,eEF1alpha1,76454,218993,20543.3019,7206.71742,1.511054,21.64804
CG1573144,bq,CG15731,2958,5182,782.8430,168.63752,2.206268,21.21199
RpL23A44,bq,RpL23A,23945,63966,6321.6670,2065.07794,1.613410,20.37143
TwdlF44,bq,TwdlF,6697,13997,1776.8570,492.98094,1.846800,19.93791
RpL644,bq,RpL6,22539,62854,5951.1989,1986.58711,1.582162,19.83906
RpS644,bq,RpS6,18692,50047,4918.5066,1630.96020,1.591614,19.52003


In [194]:
df_sub300 = set.cell.type(df_sub300, "bx", df_sub300$cluster %in% bx)

Assigning cells for cell type bx



In [1651]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == 
                                              "bq" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Act57B44,bq,Act57B,24201,49032,6408.3207,1439.01846,2.153859,27.23761
CG296244,bq,CG2962,14862,28955,3852.2991,1015.30233,1.922390,22.89928
Mlc244,bq,Mlc2,12441,28458,3299.5489,866.05437,1.928074,22.53628
eEF1alpha144,bq,eEF1alpha1,76454,218993,20543.3019,7206.71742,1.511054,21.64804
CG1573144,bq,CG15731,2958,5182,782.8430,168.63752,2.206268,21.21199
RpL23A44,bq,RpL23A,23945,63966,6321.6670,2065.07794,1.613410,20.37143
TwdlF44,bq,TwdlF,6697,13997,1776.8570,492.98094,1.846800,19.93791
RpL644,bq,RpL6,22539,62854,5951.1989,1986.58711,1.582162,19.83906
RpS644,bq,RpS6,18692,50047,4918.5066,1630.96020,1.591614,19.52003


In [195]:
df_sub300 = set.cell.type(df_sub300, "by", df_sub300$cluster %in% by)

Assigning cells for cell type by



In [1652]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == 
                                              "bq" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Act57B44,bq,Act57B,24201,49032,6408.3207,1439.01846,2.153859,27.23761
CG296244,bq,CG2962,14862,28955,3852.2991,1015.30233,1.922390,22.89928
Mlc244,bq,Mlc2,12441,28458,3299.5489,866.05437,1.928074,22.53628
eEF1alpha144,bq,eEF1alpha1,76454,218993,20543.3019,7206.71742,1.511054,21.64804
CG1573144,bq,CG15731,2958,5182,782.8430,168.63752,2.206268,21.21199
RpL23A44,bq,RpL23A,23945,63966,6321.6670,2065.07794,1.613410,20.37143
TwdlF44,bq,TwdlF,6697,13997,1776.8570,492.98094,1.846800,19.93791
RpL644,bq,RpL6,22539,62854,5951.1989,1986.58711,1.582162,19.83906
RpS644,bq,RpS6,18692,50047,4918.5066,1630.96020,1.591614,19.52003


In [1516]:
df_sub300 = set.cell.type(df_sub300, "Visceral_muscle_2", df_sub300$cluster %in% bz &
                         df_sub300$umap_2 < 1*df_sub300$umap_1 -3.3)

Assigning cells for cell type Visceral_muscle_2



In [1517]:
df_sub300 = set.cell.type(df_sub300, "Visceral_muscle", df_sub300$cluster %in% bz &
                         df_sub300$umap_2 >= 1*df_sub300$umap_1 -3.3)

Assigning cells for cell type Visceral_muscle



In [312]:
clus = c(bz)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bz' not found


In [313]:
clus = c(bz)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'bz' not found


In [1508]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "bz" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
up53,bz,up,6626,28694,9532.6511,1202.03845,2.986195,39.47395
bt53,bz,bt,8603,46215,11957.0915,1651.15272,2.855448,38.67903
wupA53,bz,wupA,4161,21040,5918.9464,828.25226,2.835457,35.53216
Sh53,bz,Sh,2468,9715,3405.5853,445.37387,2.931578,34.39946
Mhc53,bz,Mhc,7437,51099,10562.4954,1784.74478,2.564353,34.27719
CG3352153,bz,CG33521,744,2636,1051.7574,115.18077,3.178359,31.91059
jp53,bz,jp,427,1694,623.4231,65.70320,3.224384,29.94285
Neto53,bz,Neto,397,1337,580.8132,59.94962,3.252385,29.87125
CG4442253,bz,CG44422,795,3587,1194.7991,162.27534,2.871389,29.35639


In [314]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("up"), 
                   size_pos = 2, size_neg = 0.05, stroke = 0.008, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("bt"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("wupA"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("up"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [315]:
clus = c(18)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus &  
             # df_sub300$umap_2 < 0.7*umap_1 -6 &
              df_sub300$umap_2 < 1*umap_1 -3.3), 
           size =  ifelse(df_sub300$cluster %in% clus, 1, 0.7), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic() +
geom_abline(slope = 0.7, intercept = -3.3) +
geom_abline(slope = 1, intercept = -3.3) +
geom_abline(slope = -1., intercept = -6.7) +
 xlim(0, 4) + ylim(-3, 1)
print(peedeef)

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1514]:
## no strong differences but keep them separate by name
two.set.differential.gene.test.3a(cds_sub, 
                                  df_sub300$cluster == 18 & 
                                  df_sub300$umap_2 < 1*df_sub300$umap_1 -3.3 , 
                                 (df_sub300$cluster == 18 & 
                                  df_sub300$umap_2 >= 1*df_sub300$umap_1 -3.3),
                                  sample = 50000
                                )  %>% filter(higher.expr == "Set 1")  %>% 
head(20)

# of cells sampled in set 1: 1508

# of cells sampled in set 2: 615



,gene,set.1.n.umi,set.2.n.umi,set.1.tpm,set.2.tpm,higher.expr,log2.ratio,heuristic.score
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
CG14304,CG14304,70,1,101.56334,3.226223,Set 1,4.586867,30.64198
modSP,modSP,25,0,45.44194,0.000000,Set 1,5.505953,30.48842
Snmp2,Snmp2,23,0,41.85317,0.000000,Set 1,5.387265,29.20614
CG5835,CG5835,25,0,39.91086,0.000000,Set 1,5.318710,28.47856
Rab40,Rab40,19,0,34.55481,0.000000,Set 1,5.110815,26.33078
ph-d,ph-d,19,0,33.51014,0.000000,Set 1,5.066526,25.88462
ine,ine,43,1,78.45769,3.637620,Set 1,4.080458,25.75632
CG7011,CG7011,19,0,33.05476,0.000000,Set 1,5.046786,25.68705
Syx5,Syx5,17,0,32.19570,0.000000,Set 1,5.008796,25.30907


In [1515]:
two.set.differential.gene.test.3a(cds_sub, 
                                  df_sub300$cluster == 18 & 
                                  df_sub300$umap_2 < 1*df_sub300$umap_1 -3.3 , 
                                 (df_sub300$cluster == 18 & 
                                  df_sub300$umap_2 >= 1*df_sub300$umap_1 -3.3),
                                  sample = 50000
                                )  %>% filter(higher.expr == "Set 2")  %>% 
head(20)

# of cells sampled in set 1: 1508

# of cells sampled in set 2: 615



,gene,set.1.n.umi,set.2.n.umi,set.1.tpm,set.2.tpm,higher.expr,log2.ratio,heuristic.score
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
CG44422,CG44422,211,716,313.729354,3430.53707,Set 2,3.446251,40.47497
CG13739,CG13739,2,46,3.671678,185.01418,Set 2,5.307551,40.01505
CG31323,CG31323,11,74,14.217173,349.12730,Set 2,4.519981,38.20168
CG13954,CG13954,12,64,14.832382,321.34180,Set 2,4.343158,36.18914
Ca-alpha1T,Ca-alpha1T,106,370,170.145540,1710.53159,Set 2,3.321149,35.67270
CG33253,CG33253,16,90,25.504338,394.45680,Set 2,3.895567,33.60852
CG30116,CG30116,40,130,57.009062,640.86113,Set 2,3.465662,32.32117
Sur,Sur,30,114,44.014088,522.95874,Set 2,3.538249,31.96209
Ppn,Ppn,167,460,276.624625,2051.42136,Set 2,2.885416,31.74856


### ca cluster

In [197]:
df_sub300 = set.cell.type(df_sub300, "Primordial_cells_1", df_sub300$cluster %in% ca)

Assigning cells for cell type ca



In [316]:
clus = c(ca)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ca' not found


In [317]:
clus = c(ca)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.4, 0.05), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'ca' not found


In [1801]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "ca" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Brd56,ca,Brd,471,942,284.47076,33.60678,3.039154,24.79098
E(spl)m4-BFM55,ca,E(spl)m4-BFM,278,502,170.17356,18.09074,3.156062,23.41582
Tom58,ca,Tom,1237,4333,746.04440,156.40094,2.244817,21.42689
link55,ca,link,1337,5089,846.65940,202.42380,2.057293,20.01199
Ama55,ca,Ama,2260,9768,1408.93770,393.80348,1.835401,19.20090
Hsp70Bc55,ca,Hsp70Bc,275,803,172.64602,28.69892,2.539335,18.89266
Obp99a55,ca,Obp99a,1591,6636,963.71498,260.62622,1.881099,18.64914
lmd55,ca,lmd,826,3074,537.61124,134.07745,1.992776,18.08066
Hsp6855,ca,Hsp68,234,856,149.17504,29.72471,2.279535,16.48218


In [318]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("Brd"), 
                   size_pos = 2, size_neg = 0.5, stroke = 0.008, limit = 1)
plot.expr_clus.br2(cds_sub_no_doubletons, c("E(spl)m4-BFM"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 1)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Tom"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 1)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("Brd"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [1811]:
df_sub300 = set.cell.type(df_sub300, "Primordial_cells_2", df_sub300$cluster %in% cb)

Assigning cells for cell type Primordial_cells_2



In [319]:
clus = c(cb)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'cb' not found


In [320]:
clus = c(cb)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.4, 0.2), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'cb' not found


In [1807]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "cb" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CG1003556,cb,CG10035,1345,1607,857.47043,58.905895,3.839317,37.41654
sca56,cb,sca,3098,4564,1973.03039,186.937089,3.392091,37.13298
Abd-B56,cb,Abd-B,2988,7747,1982.32564,326.630866,2.597051,28.44733
5-HT2A56,cb,5-HT2A,279,269,173.65133,11.743355,3.768376,28.06813
Sr-CII56,cb,Sr-CII,393,617,261.48554,23.921433,3.391272,27.25258
CG788456,cb,CG7884,674,1221,441.15350,51.786011,3.063053,26.91935
cas56,cb,cas,2137,5455,1410.00316,241.624947,2.538899,26.56324
link56,cb,link,1812,5182,1199.41711,200.722742,2.571888,26.30866
bib56,cb,bib,1347,3782,873.35872,151.604983,2.516772,24.59410


In [321]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("CG10035"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.008, limit = 1)
plot.expr_clus.br2(cds_sub_no_doubletons, c("sca"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 4)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Abd-B"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("CG10035"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [199]:
df_sub300 = set.cell.type(df_sub300, "cc", df_sub300$cluster %in% cc)

Assigning cells for cell type cc



In [322]:
clus = c(cc)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'cc' not found


In [323]:
clus = c(cc)
peedeef = ggplot(df_sub300 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_sub300$cluster %in% clus), size =  ifelse(df_sub300$cluster %in% clus, 0.2, 0.02), stroke = 0.005) +
scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
theme_classic()
print(peedeef)

ERROR: Error in eval(expr, envir, enclos): object 'cc' not found


In [1815]:
cell.type.2.markers_no_doubletons  %>% filter(cell.type.2 == "cc" & cell.type.2.tpm > other.tpm)  %>% head(20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ds57,cc,ds,9963,14410,3206.9115,619.96869,2.368592,27.58798
otp57,cc,otp,3116,4680,1251.3588,218.77044,2.509426,25.82308
Pvf357,cc,Pvf3,6120,11847,2010.4898,509.92641,1.976360,21.68867
robo257,cc,robo2,6690,11692,2010.1194,525.03955,1.934038,21.22371
Notum57,cc,Notum,2219,3892,749.4497,166.00330,2.165955,20.68836
Poxm57,cc,Poxm,580,793,206.3057,32.51788,2.621780,20.17621
nkd57,cc,nkd,16948,38596,5280.4399,1706.77591,1.628539,20.13968
byn57,cc,byn,373,560,148.2083,21.97528,2.689472,19.42117
dally57,cc,dally,6530,13999,2080.4428,629.45349,1.722429,18.98697


In [324]:
plot.expr_clus.br2(cds_sub_no_doubletons, c("ds"), 
                   size_pos = 2, size_neg = 0.5, stroke = 0.08, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("otp"), 
                   size_pos = 2, size_neg = 0.4, stroke = 0.02, limit = 8)
plot.expr_clus.br2(cds_sub_no_doubletons, c("Pvf3"), 
                   size_pos = 2, size_neg = 0.2, stroke = 0.02, limit = 4)

ERROR: Error in plot.expr_clus.br2(cds_sub_no_doubletons, c("ds"), size_pos = 2, : could not find function "plot.expr_clus.br2"


In [200]:
df_sub300 = set.cell.type(df_sub300, "cd", df_sub300$cluster %in% cd)

Assigning cells for cell type cd



In [201]:
df_sub300 = set.cell.type(df_sub300, "ce", df_sub300$cluster %in% ce)

Assigning cells for cell type ce



In [202]:
df_sub300 = set.cell.type(df_sub300, "cf", df_sub300$cluster %in% cf)

Assigning cells for cell type cf



In [771]:
df_sub300  %>% count(cell.type.3, cluster)

cell.type.3,cluster,n
<chr>,<fct>,<int>
Chordotonal_organ,172,121
Chordotonal_sensory,14,144
Chordotonal_sensory,158,606
Malpighian tubule,103,1265
Malpighian tubule,163,466
multidendritic_sensory,8,929
multidendritic_sensory,14,995
multidendritic_sensory,170,145
proventriculus,160,528


In [1601]:
df_sub300  %>% count(cell.type.3)

cell.type.3,n
<chr>,<int>
Amnioserosa,7091
Analpad,850
Analpad_large_intestine,734
Antennal_sense_organs,748
Bolwig_organ,115
Central_brain_ventral_nerve_cord_primorium,5819
Cholinergic,8949
Chordotonal_organ,121
Chordotonal_sensory,750


In [ ]:
rename.cell.type = function(data_df, cell.type, x) {
       message("Reassigning cells for cell type ", cell.type)
       data_df$cell.type.3 = ifelse(x, cell.type, data_df$cell.type.3)
       return(data_df)
    }


In [1778]:
df_sub300_copy = df_sub300

In [1605]:
df_sub300_copy$cell.type.3 <- 
gsub("nephrocyte_query", "Garland_nephrocyte", df_sub300_copy$cell.type.3)

In [1779]:
df_sub300_copy$cell.type.3 <- 
gsub("proventriculus", NA, df_sub300_copy$cell.type.3)

In [1780]:
df_sub300_copy  %>% count(cell.type.3)

cell.type.3,n
<chr>,<int>
Amnioserosa,7091
Analpad,850
Analpad_large_intestine,734
Antennal_sense_organs,748
Bolwig_organ,115
Central_brain_ventral_nerve_cord_1,6846
Central_brain_ventral_nerve_cord_2,10161
Central_brain_ventral_nerve_cord_early,5058
Central_brain_ventral_nerve_cord_primorium,5819


## DE genes on new annotation

In [205]:
DEG.results = lapply(sort((unique(df_sub300$cell.type.3))),
                     function(this.cell.type) {
    message("Finding markers for cell.type ", this.cell.type)
    cbind(
        two.set.differential.gene.test.3a(
            cds_sub,
            df_sub300$cell.type.3 == this.cell.type,
            df_sub300$cell.type.3 != this.cell.type,
            sample.n.max = 50000),
        data.frame(cell.type.3 = this.cell.type))
                         })

Finding markers for cell.type a

# of cells sampled in set 1: 528

# of cells sampled in set 2: 50000

Finding markers for cell.type aa

# of cells sampled in set 1: 1798

# of cells sampled in set 2: 50000

Finding markers for cell.type ab

# of cells sampled in set 1: 1189

# of cells sampled in set 2: 50000

Finding markers for cell.type ac

# of cells sampled in set 1: 4237

# of cells sampled in set 2: 50000

Finding markers for cell.type ad

# of cells sampled in set 1: 850

# of cells sampled in set 2: 50000

Finding markers for cell.type ae

# of cells sampled in set 1: 1241

# of cells sampled in set 2: 50000

Finding markers for cell.type af

# of cells sampled in set 1: 115

# of cells sampled in set 2: 50000

Finding markers for cell.type ag

# of cells sampled in set 1: 636

# of cells sampled in set 2: 50000

Finding markers for cell.type ah

# of cells sampled in set 1: 2766

# of cells sampled in set 2: 50000

Finding markers for cell.type ai

# of cells sampled in set 

# of cells sampled in set 2: 50000

Finding markers for cell.type v

# of cells sampled in set 1: 6021

# of cells sampled in set 2: 50000

Finding markers for cell.type w

# of cells sampled in set 1: 748

# of cells sampled in set 2: 50000

Finding markers for cell.type x

# of cells sampled in set 1: 581

# of cells sampled in set 2: 50000

Finding markers for cell.type y

# of cells sampled in set 1: 1571

# of cells sampled in set 2: 50000

Finding markers for cell.type z

# of cells sampled in set 1: 1101

# of cells sampled in set 2: 50000



In [206]:
cell.type.3.markers = do.call(rbind, DEG.results)  %>%
    select(
        cell.type.3 = cell.type.3, gene, 
        cell.type.3.n.umi = set.1.n.umi,
        other.n.umi = set.2.n.umi,
        cell.type.3.tpm = set.1.tpm,
        other.tpm = set.2.tpm,
        log2.ratio, heuristic.score) %>%
    arrange(-heuristic.score)

In [207]:
head(cell.type.3.markers, 20)

,cell.type.2,gene,cell.type.2.n.umi,other.n.umi,cell.type.2.tpm,other.tpm,log2.ratio,heuristic.score
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Muc26B,a,Muc26B,3993,111,10368.322,4.1576480,10.973182,146.3826
CG740954,c,CG7409,600,24,4121.228,0.4739611,11.449160,137.4954
CG1137061,d,CG11370,3708,446,15737.252,17.5042494,9.732111,135.6850
CG746563,f,CG7465,23068,761,18549.567,23.9239377,9.539638,135.2642
CG7017,a,CG7017,3759,265,11685.814,10.6958361,9.964547,134.6469
CG1135063,f,CG11350,8044,159,6300.220,4.2137333,10.238870,129.2290
Muc96D,a,Muc96D,2590,118,6427.809,5.1071969,10.039601,127.0043
CG1313354,c,CG13133,405,14,2679.090,0.1829864,11.145094,126.9211
CG1456661,d,CG14566,518,60,3985.539,1.6730948,10.542048,126.0926


In [208]:
cell.type.3.markers  %>% filter(heuristic.score > 60)  %>% dim

[1] 399   8

In [280]:
cell.type.3.markers  %>% filter(heuristic.score > 60 & gene %in% fly_TFs)  %>% dim

[1] 10  8

In [284]:
cell.type.3.markers  %>% filter(heuristic.score > 40 & gene %in% fly_TFs)  %>% dim

[1] 96  8

In [209]:
unique((cell.type.3.markers  %>% filter(heuristic.score > 60))$cell.type.3)

[1] "a"  "c"  "d"  "f"  "h"  "g"  "j"  "k"  "o"  "l"  "b"  "ac" "p"  "t"  "u" 
[16] "x"  "z"  "w"  "v"  "m"  "aw" "ad" "ag" "n"  "af" "y"  "al" "ai" "ak" "aa"
[31] "as" "ab" "ao" "aq" "bc" "ce" "am" "aj" "ah" "s"  "cd" "ar" "au" "at" "bj"
[46] "cb" "av" "ca" "ae"

In [210]:
length(unique((cell.type.3.markers  %>% filter(heuristic.score > 60))$cell.type.3))

[1] 49

### umap

In [793]:
colData(cds_sub)$cell.type.3 = NULL

In [794]:
colData(cds_sub)$cell.type.3 = rep(NA, ncol(cds_sub))

In [1791]:
colData(cds_sub)$cell.type.3 = df_sub300$cell.type.3

In [ ]:
plot_cells(cds_sub, color_cells_by = "cell.type.3", label_groups_by_cluster = FALSE,  
           cell_size = 0.1,group_label_size = 5) + 
# theme_classic() +
theme(text = element_text(color="black", size=20, face="bold")) 

No trajectory to plot. Has learn_graph() been called yet?



In [325]:
plot_cells(cds_sub, color_cells_by = "cell.type.3", label_groups_by_cluster = FALSE,  
           cell_size = 0.1,group_label_size = 5) + 
# theme_classic() +
theme(text = element_text(color="black", size=20, face="bold")) +
xlim(0, 7.5) + ylim(-7.5, 0)

ERROR: Error in plot_cells(cds_sub, color_cells_by = "cell.type.3", label_groups_by_cluster = FALSE, : could not find function "plot_cells"


In [326]:
plot_cells(cds_sub, color_cells_by = "cell.type.3", label_groups_by_cluster = FALSE,  
           cell_size = 0.1,group_label_size = 5) + 
# theme_classic() +
theme(text = element_text(color="black", size=20, face="bold")) +
xlim(0, 7.5) + ylim(0, 7.5)

ERROR: Error in plot_cells(cds_sub, color_cells_by = "cell.type.3", label_groups_by_cluster = FALSE, : could not find function "plot_cells"


In [327]:
plot_cells(cds_sub, color_cells_by = "cell.type.3", label_groups_by_cluster = FALSE,  
           cell_size = 0.1,group_label_size = 5) + 
# theme_classic() +
theme(text = element_text(color="black", size=20, face="bold")) +
xlim(-7.5, 0) + ylim(-7.5, 0)

ERROR: Error in plot_cells(cds_sub, color_cells_by = "cell.type.3", label_groups_by_cluster = FALSE, : could not find function "plot_cells"


In [328]:
plot_cells(cds_sub, color_cells_by = "cell.type.3", label_groups_by_cluster = FALSE,  
           cell_size = 0.1,group_label_size = 5) + 
# theme_classic() +
theme(text = element_text(color="black", size=20, face="bold")) +
xlim(-8, 0) + ylim(0, 8)

ERROR: Error in plot_cells(cds_sub, color_cells_by = "cell.type.3", label_groups_by_cluster = FALSE, : could not find function "plot_cells"


In [1599]:
options(repr.plot.width=28, repr.plot.height=16)

In [329]:
plot_cells(cds_sub, color_cells_by = "cell.type.3", label_groups_by_cluster = FALSE,  
           cell_size = 0.1,group_label_size = 5) + 
# theme_classic() +
theme(text = element_text(color="black", size=20, face="bold")) 

ERROR: Error in plot_cells(cds_sub, color_cells_by = "cell.type.3", label_groups_by_cluster = FALSE, : could not find function "plot_cells"


In [330]:
plot_cells(cds_sub, color_cells_by = "inferred_time_window", cell_size = 0.05, group_label_size = 0)

ERROR: Error in plot_cells(cds_sub, color_cells_by = "inferred_time_window", : could not find function "plot_cells"


In [215]:
saveRDS(cds_sub, "data/cds_fly_sub_PCA300_older_cells_reannotated.rds")

In [216]:
saveRDS(cell.type.2.markers, "data/fly_sub300_cell.type.2.markers")

In [331]:
ggplot(df_sub300, aes(x = umap_1, y = umap_2)) +
        geom_point(aes(color = cell.type.3), size = 0.1, stroke = 0.03) +
        geom_label(aes(label = cell.type.3), color = "black", size = 2.5, data = df_sub300 %>%
            group_by(cell.type.3) %>%
            summarize(umap_1 = median(umap_1), umap_2 = median(umap_2))) +
        guides(color = F) +
# geom_text_repel() +
        monocle3:::monocle_theme_opts()

ERROR: Error in ggplot(df_sub300, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


## check Lou's fuzzy clustering

In [1039]:
cds_n21 = readRDS("data/cds_fly_pca300_no_doubletons.nn21.rds")

In [1042]:
fuzzy_percent = read_tsv("data/cds_fly_pca300_no_doubletons.nn21.fuzzy.tsv") 

Rows: 216570 Columns: 5
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (1): cell
dbl (4): z1, x, y, pct

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [1043]:
df_n21 = make_df(cds_n21)

In [1044]:
dim(df_n21)
dim(fuzzy_percent)

[1] 216570     33

[1] 216570      5

In [1045]:
head(fuzzy_percent)

cell,z1,x,y,pct
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,41,-4.1837888,0.4881140,0.26496218
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCAGAACCTCAGGTATGGA,164,-3.0353165,-2.7561928,0.15352179
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGAGAGATAAGTAGTCAG,22,-6.3859172,3.5901138,0.01011936
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGGAGTTTTGAGGTAGAA,80,-3.1897362,-2.6267887,0.11677301
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGTCTCATTCGATATTGA,141,-0.3154551,1.6942367,0.01940591
exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCTCCTGATTGCGCCAGAA,152,-6.8989954,0.6930812,0.51285236


In [1048]:
left_join(df_n21, select(fuzzy_percent, cell, pct), by = "cell")  %>%  head

,orig.ident,nCount_RNA,nFeature_RNA,rt_bc,cell,time,percent.mt,percent.ribo,barn_dro,barn_hu,barn_mus,RNA_snn_res.0.8,doublet_score,predicted_doublet,doublet_cluster,seurat_clusters,doublet_subcluster,RNA_snn_res.0.08,lasso_age,NNv1_age,NNv2_age,lasso_shift,NNv1_shift,NNv2_shift,annot,ident,inferred_time_window,cell.type_time.bin,Size_Factor,cell.type.2,cluster,umap_1,umap_2,pct
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<int>,<int>,<int>,<chr>,<dbl>,<chr>,<lgl>,<fct>,<lgl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<fct>,<chr>,<chr>,<dbl>,<chr>,<fct>,<dbl>,<dbl>,<dbl>
1,exp1,346,247,AAGTTGCCAT,exp1_idx1AGACCATATC_idx2AACGAGGCTA.AACCGCTGTAAGTTGCCAT,hrs_06_10,1.4450867,4.3352601,371,0,0,16,0.05276064,Singlet,FALSE,21,FALSE,21,8.402794,7.885203,8.119637,0.0000000,0.0000000,0.000000,sensory nervous system primordium,21,6-8,sensory nervous system primordium:6-8,0.8108612,v,89,-2.0067012,1.2184464,0.26496218
2,exp1,586,425,CAGGTATGGA,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCAGAACCTCAGGTATGGA,hrs_08_12,1.0238908,6.4846416,630,0,0,4,0.09008910,Singlet,FALSE,11,FALSE,11,11.266422,10.062633,11.156040,0.0000000,0.0000000,0.000000,brain,11,10-12,brain:10-12,1.4059888,bl,22,-3.6831634,2.8012645,0.15352179
3,exp1,292,202,AAGTAGTCAG,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGAGAGATAAGTAGTCAG,hrs_10_14,0.3424658,0.6849315,320,3,0,23,0.06585398,Singlet,FALSE,12,FALSE,12,12.544636,12.211529,13.830982,0.0000000,0.0000000,0.000000,epidermis,12,12-14,epidermis:12-14,0.6967951,h,165,-1.3580737,0.3582040,0.01011936
4,exp1,335,243,TGAGGTAGAA,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGGAGTTTTGAGGTAGAA,hrs_14_18,4.4776119,5.6716418,386,0,0,8,0.08478501,Singlet,FALSE,11,FALSE,11,12.542527,13.739003,12.519031,-1.4574728,-0.2609968,-1.480969,sensory nervous system,11,12-14,sensory nervous system:12-14,0.7811049,bk,102,-3.8563070,3.3158269,0.11677301
5,exp1,354,272,TCGATATTGA,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCGTCTCATTCGATATTGA,hrs_14_18,0.5649718,1.6949153,380,0,1,8,0.08257165,Singlet,FALSE,11,FALSE,11,13.458162,15.126904,14.558554,-0.5418379,0.0000000,0.000000,sensory nervous system,11,14-16,sensory nervous system:14-16,0.8306988,w,148,-0.6711318,-2.9462104,0.01940591
6,exp1,287,221,TGCGCCAGAA,exp1_idx1AGACCATATC_idx2AACGAGGCTA.CCTCCTGATTGCGCCAGAA,hrs_08_12,0.0000000,3.4843206,307,0,0,6,0.06023494,Singlet,FALSE,9,FALSE,9,9.715162,9.000631,11.355991,0.0000000,0.0000000,0.000000,midline primordium,9,8-10,midline primordium:8-10,0.6893560,aj,120,-7.5437593,-0.6611074,0.51285236


In [1049]:
df_n21 = left_join(df_n21, select(fuzzy_percent, cell, pct), by = "cell")

In [1064]:
colnames(df_n21)

[1] "orig.ident"           "nCount_RNA"           "nFeature_RNA"        
 [4] "rt_bc"                "cell"                 "time"                
 [7] "percent.mt"           "percent.ribo"         "barn_dro"            
[10] "barn_hu"              "barn_mus"             "RNA_snn_res.0.8"     
[13] "doublet_score"        "predicted_doublet"    "doublet_cluster"     
[16] "seurat_clusters"      "doublet_subcluster"   "RNA_snn_res.0.08"    
[19] "lasso_age"            "NNv1_age"             "NNv2_age"            
[22] "lasso_shift"          "NNv1_shift"           "NNv2_shift"          
[25] "annot"                "ident"                "inferred_time_window"
[28] "cell.type_time.bin"   "Size_Factor"          "cell.type.2"         
[31] "cluster"              "umap_1"               "umap_2"              
[34] "pct"

In [1065]:
colData(cds_sub)$fuz_pct = NULL

In [1066]:
colData(cds_n21)$fuz_pct = rep(NA, ncol(cds_n21))

In [1067]:
colData(cds_n21)$fuz_pct = df_n21$pct

In [332]:
plot.expr_clus.br2(cds_n21, c("apn"), 
                   size_pos = 1, size_neg = 0.3, stroke = 0.008, limit = 8)

ERROR: Error in plot.expr_clus.br2(cds_n21, c("apn"), size_pos = 1, size_neg = 0.3, : could not find function "plot.expr_clus.br2"


In [1069]:
plot.expr_clus.br2(cds_n21[, df_n21  %>% filter(df_n21$pct > 0.2)], c("apn"), 
                   size_pos = 1, size_neg = 0.3, stroke = 0.008, limit = 8)

ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function 'reducedDims': unable to find an inherited method for function ‘NSBS’ for signature ‘"data.frame"’


In [1070]:
df_n21  %>% filter(pct > 0.2)  %>%  dim

[1] 141573     34

In [333]:
peedeef = ggplot(df_n21 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_n21$pct), size =  0.02, stroke = 0.005) +
# scale_color_manual(values = c("grey70", "firebrick")) +
#      labs(color = cell_type) +
guides(color = guide_legend(override.aes = list(size = 4))) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_n21, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [334]:
peedeef = ggplot(df_n21  %>% filter(pct > 0.2), aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = (df_n21  %>% filter(pct > 0.2))$pct), size =  0.02, stroke = 0.005) +
# scale_color_manual(values = c("grey70", "firebrick")) +
#      labs(color = cell_type) +
guides(color = guide_legend(override.aes = list(size = 4))) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_n21 %>% filter(pct > 0.2), aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [335]:
peedeef = ggplot(df_n21  %>% filter(pct < 0.2), aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = (df_n21  %>% filter(pct < 0.2))$pct), size =  0.02, stroke = 0.005) +
# scale_color_manual(values = c("grey70", "firebrick")) +
#      labs(color = cell_type) +
guides(color = guide_legend(override.aes = list(size = 4))) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_n21 %>% filter(pct < 0.2), aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [336]:
peedeef = ggplot(df_n21 , aes(x = umap_1, y = umap_2)) +
geom_point(aes(color = df_n21$pct < 0.1), size =  0.02, stroke = 0.005) +
 scale_color_manual(values = c("grey70", "firebrick")) +
      labs(color = cell_type) +
# guides(color = guide_legend(override.aes = list(size = 4))) +
theme_classic()
print(peedeef)

ERROR: Error in ggplot(df_n21, aes(x = umap_1, y = umap_2)): could not find function "ggplot"


In [1782]:
args(plot_genes_in_pseudotime)

function (cds_subset, min_expr = NULL, cell_size = 0.75, nrow = NULL, 
    ncol = 1, panel_order = NULL, color_cells_by = "pseudotime", 
    trend_formula = "~ splines::ns(pseudotime, df=3)", label_by_short_name = TRUE, 
    vertical_jitter = NULL, horizontal_jitter = NULL) 
NULL